# AInewsbot.ipynb - Automate collecting daily AI news

1. initial scrape of front pages of tech sites
  - Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
  - Save HTML of each URL in htmldata directory  
  - Extract URLs from all files, create a pandas dataframe with url, title, src

2. Filter and clean to AI-related headlines not seen before
  - Use ChatGPT prompt to filter only AI-related headlines by sending a prompt and formatted table of headlines
  - Use SQLite to filter headlines previously seen 
  - remove duplicate URLs and headlines
  - ensure there are pretty source names for each news site

3. Topic analysis, make a list of topics for each headline
  - using a prompt, check each headline against a number of evergreen AI topics, e.g. deepfakes, regulation, AI in education
  - extract free from topics from each headline
  - combine topics into topic list for each headline
  - cluster headlines using dimensionality-reduced embeddings and DBSCAN; ask chatgpt to name each cluster
  - sort headlines by doing a traveling salesman shortest traversal in embedding space

4. Summarize individual news story pages in 3 bullets using a prompt

5. create a large markdown file with all bullet points and topics

6. give the markdown file to ChatGPT and ask it to make a list of most popular and import topics of the day

7. human should make a list of the day's topics, combining the chatgpt response to the quesion and cluster topics and 

8. Put summaries in vector store along with metadata. For each topic, retrieve all associated stories and have chatgpt write a digest of those stories in the given format.

9. assemble stories into first draft of newsletter for rewriting as necessary

todo:

use langgraph for final editing workfow
1. prompt to edit final copy for dupes, combine similar sections, copy edit
2. have a reviewer prompt check if there are any bullet points to move to a different section 
3. if so have an editor prompt remove them , return to 2. until nothing left to move dupes left
4. have a reviewer check each section, identify bullet points that are similar to other bullet points in the section and have identical links. rewrite combining so there is no duplication. 
5. identify sections that are short or similar to other sections and suggest sections that should be combined them
6. have an editor prompt merge short sections, return to 4, until no orphan sections left
7. maybe final copy-edit prompt

Original, alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [1]:
# import sys
# del sys.modules['ainb_llm']


In [2]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter
import shutil

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, get_google_news_redirects)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, 
                      fetch_openai_summary, trunc_tokens, categorize_headline)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [3]:
# before_date = '2024-07-23 18:00:00'

In [4]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [5]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-28 07:41:27,829 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-28 07:41:27,830 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-28 07:41:27,830 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-28 07:41:27,830 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-28 07:41:27,830 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-28 07:41:27,830 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-28 07:4

In [6]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [7]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [8]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))
    

2024-07-28 07:41:27,845 - AInewsbot - INFO - 0


In [9]:
log(f"{len(files)} files found")


2024-07-28 07:41:27,846 - AInewsbot - INFO - 0 files found


# Fetch and save source pages

In [10]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-07-28 07:41:27,872 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-07-28 07:41:27,881 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:41:27,882 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:41:27,882 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:41:43,180 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:41:43,180 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:41:43,180 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:41:43,181 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-28 07:41:43,182 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-28 07:41:43,186 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-28 07:42:20,467 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-28 07:42:20,473 - AInewsbot - INFO - get_dr

2024-07-28 07:43:05,890 - AInewsbot - INFO - get_files(Hacker News Page 2) - Saving Hacker News Page 2 (07_28_2024 07_43_05 AM).html as utf-8
2024-07-28 07:43:05,890 - AInewsbot - INFO - Processing NYT Tech
2024-07-28 07:43:05,891 - AInewsbot - INFO - get_files(Technology - The New York Times) - starting get_files https://www.nytimes.com/section/technology
2024-07-28 07:43:13,567 - AInewsbot - INFO - get_files(Discover and Add New Feedly AI Feeds) - Loading additional infinite scroll items
2024-07-28 07:43:16,426 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.fin

2024-07-28 07:44:05,765 - AInewsbot - INFO - get_files(Technology - The Washington Post) - Saving Technology - The Washington Post (07_28_2024 07_44_05 AM).html as utf-8
2024-07-28 07:44:05,771 - AInewsbot - INFO - Quit webdriver
2024-07-28 07:44:09,429 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-28 07:44:09,429 - AInewsbot - INFO - get_files(Technology - WSJ.com) - Saving Technology - WSJ.com (07_28_2024 07_44_09 AM).html as utf-8
2024-07-28 07:44:09,432 - AInewsbot - INFO - Quit webdriver


In [11]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log(f"{sourcename} -> {page}")
    

2024-07-28 07:44:12,076 - AInewsbot - INFO - Saved 17 pages
2024-07-28 07:44:12,081 - AInewsbot - INFO - Business Insider -> htmldata/Tech - Business Insider (07_28_2024 07_42_33 AM).html
2024-07-28 07:44:12,081 - AInewsbot - INFO - Google News -> htmldata/Google News - Technology - Artificial intelligence (07_28_2024 07_43_04 AM).html
2024-07-28 07:44:12,081 - AInewsbot - INFO - HackerNoon -> htmldata/HackerNoon - read, write and learn about any technology (07_28_2024 07_43_16 AM).html
2024-07-28 07:44:12,081 - AInewsbot - INFO - Reddit -> htmldata/top scoring links _ multi (07_28_2024 07_43_49 AM).html
2024-07-28 07:44:12,081 - AInewsbot - INFO - VentureBeat -> htmldata/AI News _ VentureBeat (07_28_2024 07_44_00 AM).html
2024-07-28 07:44:12,082 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (07_28_2024 07_42_32 AM).html
2024-07-28 07:44:12,082 - AInewsbot - INFO - Feedly AI -> htmldata/Discover and Add New Feedly AI Feeds (07_28_2024 07_43_53 AM).html
2024-07-28 07:44:12,

17


# Extract news URLs from saved pages

In [12]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-07-28 07:44:12,091 - AInewsbot - INFO - Parsing html files
2024-07-28 07:44:12,092 - AInewsbot - INFO - Business Insider -> htmldata/Tech - Business Insider (07_28_2024 07_42_33 AM).html
2024-07-28 07:44:12,093 - AInewsbot - INFO - parse loop - Business Insider
2024-07-28 07:44:12,120 - AInewsbot - INFO - parse_file - found 313 raw links
2024-07-28 07:44:12,125 - AInewsbot - INFO - parse_file - found 52 filtered links
2024-07-28 07:44:12,125 - AInewsbot - INFO - parse loop - 52 links found
2024-07-28 07:44:12,125 - AInewsbot - INFO - Google News -> htmldata/Google News - Technology - Artificial intelligence (07_28_2024 07_43_04 AM).html
2024-07-28 07:44:12,125 - AInewsbot - INFO - parse loop - Google News
2024-07-28 07:44:12,384 - AInewsbot - INFO - parse_file - found 990 raw links
2024-07-28 07:44:12,391 - AInewsbot - INFO - parse_file - found 428 filtered links
2024-07-28 07:44:12,391 - AInewsbot - INFO - parse loop - 428 links found
2024-07-28 07:44:12,392 - AInewsbot - INFO - 

,id,src,title,url
0,0,Ars Technica,US solar production soars by 25 percent in jus...,https://arstechnica.com/science/2024/07/us-sol...
1,1,Ars Technica,Scientists unlock more secrets of Rembrandt’s ...,https://arstechnica.com/science/2024/07/arseni...
2,2,Ars Technica,Ars Live recap: Pondering the complexities of ...,https://arstechnica.com/science/2024/07/ars-li...
3,3,Ars Technica,Are you a workaholic? Here’s how to spot the s...,https://arstechnica.com/science/2024/07/are-yo...
4,4,Ars Technica,A chemist explains the chemistry behind decaf ...,https://arstechnica.com/science/2024/07/a-chem...


In [13]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

# Filter URLs to new AI headlines only

In [14]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date=globals().get('before_date'))
len(filtered_df)


2024-07-28 07:44:13,433 - AInewsbot - INFO - Existing URLs: 140027
2024-07-28 07:44:13,456 - AInewsbot - INFO - New URLs: 808


808

In [15]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semantic meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based 

In [16]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-07-28 07:44:13,721 - AInewsbot - INFO - Paginated 17 pages


In [17]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-07-28 07:44:13,723 - AInewsbot - INFO - start classify
2024-07-28 07:44:13,726 - AInewsbot - INFO - Applying prompt to 17 pages using gpt-4o-mini
2024-07-28 07:44:13,729 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,749 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,749 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,750 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,750 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,750 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,750 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,751 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,751 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,751 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,751 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,752 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,752 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:13,752 - AInewsbot - INFO - sent 50 items 
2024-07-2

808


,id,isAI
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False


In [18]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,0,Ars Technica,US solar production soars by 25 percent in jus...,https://arstechnica.com/science/2024/07/us-sol...,False,2024-07-28
1,1,Ars Technica,Scientists unlock more secrets of Rembrandt’s ...,https://arstechnica.com/science/2024/07/arseni...,False,2024-07-28
2,2,Ars Technica,Ars Live recap: Pondering the complexities of ...,https://arstechnica.com/science/2024/07/ars-li...,False,2024-07-28
3,3,Ars Technica,Are you a workaholic? Here’s how to spot the s...,https://arstechnica.com/science/2024/07/are-yo...,False,2024-07-28
4,4,Ars Technica,A chemist explains the chemistry behind decaf ...,https://arstechnica.com/science/2024/07/a-chem...,False,2024-07-28


In [19]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-28 07:44:24,234 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-28 07:44:24,236 - AInewsbot - INFO - Unmatched source rows: 0


In [20]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")

AIdf

2024-07-28 07:44:24,242 - AInewsbot - INFO - Found 81 AI headlines


,id,src,title,url,isAI,date
0,0,FT Tech,ECB finds ‘shortcomings’ in banks’ ability to ...,https://www.ft.com/content/1e69ab42-813c-4729-...,True,2024-07-28
1,1,Google News,White House releases long-awaited FedRAMP mode...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28
2,2,Google News,Nvidia supplier SK Hynix to build $6.8 billion...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28
3,3,Google News,Musk mocks Harris with AI-generated parody vid...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28
4,4,Google News,Here's how artificial intelligence can help yo...,https://news.google.com/articles/CBMiZmh0dHBzO...,True,2024-07-28
...,...,...,...,...,...,...
76,76,Reddit,X’s new AI training opt-out setting draws regu...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28
77,77,Reddit,Spain is using AI for domestic violence risk a...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28
78,78,Reddit,TikTok’s algorithm is highly sensitive – and c...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28
79,79,Reddit,Autonomous delivery startup Nuro is gearing up...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28


In [21]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [22]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-07-28 07:44:24,260 - AInewsbot - INFO - Found 79 unique AI headlines


In [23]:
# map google news headlines to redirect

AIdf = get_google_news_redirects(AIdf)


2024-07-28 07:44:24,547 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:24,825 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:25,131 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:25,415 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:25,694 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:25,987 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:26,283 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:26,568 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:26,840 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:27,155 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-28 07:44:27,438 - AInewsbot - INFO - get_google_news_redirects

In [24]:
# must do this after fixing google actualurl
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf.head()

,id,src,title,url,isAI,date,actual_url,hostname
0,0,Google News,"""AI in Robotics: Revolutionizing the Future of...",https://news.google.com/articles/CBMiYWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiYWh0dHBzO...,news.google.com
1,1,Google News,2 Monster AI Growth Stocks to Buy Before They ...,https://news.google.com/articles/CBMiW2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiW2h0dHBzO...,news.google.com
2,2,Google News,"5 big analyst AI moves: PT hikes for Nvidia, A...",https://news.google.com/articles/CBMihQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMihQFodHRwc...,news.google.com
3,3,HackerNoon,A Human Writers Thoughts on AI Writing Bots,https://hackernoon.com/a-human-writers-thought...,True,2024-07-28,https://hackernoon.com/a-human-writers-thought...,hackernoon.com
4,4,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com


### Get site names and update site names based on URL

In [25]:
# get site_name
conn = sqlite3.connect('articles.db')
c = conn.cursor()
#  and timestamp > '2024-07-01' 
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}

sites_df

,id,hostname,site_name
0,1,247wallst.com,247wallst
1,2,3quarksdaily.com,3 Quarks Daily
2,3,9to5google.com,9to5Google
3,4,9to5mac.com,9to5Mac
4,5,9to5toys.com,9to5Toys
...,...,...,...
790,838,www.91mobiles.com,91mobiles
791,839,www.braintrust.dev,Braintrust
792,840,www.vldb.org,VLDB (Very Large Data Bases)
793,841,huyenchip.com,Huyen Chip


In [26]:
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))
AIdf.loc[AIdf['site_name']==""]

,id,src,title,url,isAI,date,actual_url,hostname,site_name
63,63,Hacker News 2,The Elegant Math of Machine Learning,https://nautil.us/the-elegant-math-of-machine-...,True,2024-07-28,https://nautil.us/the-elegant-math-of-machine-...,nautil.us,


In [27]:
async def get_site_name(session, row):
    cat_prompt = f"""
based on this url and your knowledge of the Web, what is the name of the site? https://{row.hostname}

return the response as a json object of the form {{"url": "www.yankodesign.com", "site_name": "Yanko Design"}}

    """
    try:
        messages=[
                  {"role": "user", "content": cat_prompt
                  }]

        payload = {"model":  LOWCOST_MODEL,
                   "response_format": {"type": "json_object"},
                   "messages": messages,
                   "temperature": 0
                   }
        response = await fetch_openai(session, payload)
        response_dict = json.loads(response["choices"][0]["message"]["content"])
        return response_dict
    except Exception as exc:
        print(exc)
                
tasks = []
async with aiohttp.ClientSession() as session:
    for row in AIdf.loc[AIdf['site_name']==""].itertuples():
        task = asyncio.create_task(get_site_name(session, row))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)

responses


[{'url': 'nautil.us', 'site_name': 'Nautilus'}]

In [28]:
# update site_dict from responses
new_urls = []
for r in responses:
    if r['url'].startswith('https://'):
        r['url'] = r['url'][8:]
    new_urls.append(r['url'])
    sites_dict[r['url']] = r['site_name']
    print(r['url'], r['site_name'])

AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))



nautil.us Nautilus


In [29]:
for url in new_urls:
    sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
    print(url, '->', sites_dict[url])
    conn.execute(sqlstr, (url, sites_dict[url]))
    conn.commit()


nautil.us -> Nautilus


In [30]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in AIdf.itertuples():
    insert_article(conn, cursor, row.src, row.hostname, row.title,
                   row.url, row.actual_url, row.isAI, row.date)


# Topic analysis
Try to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

### Fit dimensionality reduction model

In [31]:
# # train dimensionality reduction, only need to do this every few months and pickle the model to reflect new topics
# # extracts all links from history where isAI=1
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id desc limit 20000"
# ai_history_df = pd.read_sql_query(query, conn)
# len(ai_history_df)

In [32]:
# embedding_model="text-embedding-3-large"
# embedding_df_list = []
# pages = paginate_df(ai_history_df, maxpagelen=1000, max_input_tokens=8192)

# for p in pages:
#     response = client.embeddings.create(input=[obj['title'] for obj in p],
#                                         model=embedding_model)
#     embedding_df_list.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))

# embedding_df = pd.concat(embedding_df_list, axis=0, ignore_index=True)

# embedding_df.to_pickle("historical_embedding_df.pkl")


In [33]:
# # Initialize the UMAP reducer
# reducer = umap.UMAP(n_components=30)
# # Fit the reducer to the data without transforming
# reducer.fit(embedding_df)
# # Pickle the reducer
# with open('reducer.pkl', 'wb') as f:
#     pickle.dump(reducer, f)
# print("UMAP reducer pickled and saved as 'reducer.pkl'")

In [34]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [35]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

2024-07-28 07:44:42,721 - AInewsbot - INFO - start topic extraction
2024-07-28 07:44:42,722 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-07-28 07:44:42,722 - AInewsbot - INFO - sent 50 items 
2024-07-28 07:44:42,723 - AInewsbot - INFO - sent 29 items 
2024-07-28 07:44:55,319 - AInewsbot - INFO - got dict with 29 items 
2024-07-28 07:44:59,467 - AInewsbot - INFO - got dict with 50 items 
2024-07-28 07:44:59,470 - AInewsbot - INFO - Processed 79 responses.
2024-07-28 07:44:59,471 - AInewsbot - INFO - end topic extraction


In [36]:
topic_df = pd.DataFrame(response)
topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
print(len(topic_df))
topic_df.head()


79


,id,extracted_topics
0,0,"[AI, robotics, automation, future technology]"
1,1,"[AI, stocks, investment, Microsoft, Apple]"
2,2,"[AI, analyst moves, Nvidia, Apple, Tesla, inve..."
3,3,"[AI, writing, human perspective, bots]"
4,4,"[AI, robots, intelligence, boundaries]"


In [37]:
all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
item_counts = Counter(all_topics)
filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
print(len(filtered_topics))
sorted(filtered_topics)


32


['amd ryzen',
 'anthropic',
 'apple',
 'business',
 'chatbot',
 'chatgpt',
 'chip plant',
 'data scraping',
 'elon musk',
 'google',
 'hp',
 'investment',
 'laptops',
 'machine learning',
 'meta',
 'mistral large 2',
 'msi',
 'neural networks',
 'nio',
 'nvidia',
 'openai',
 'robotics',
 'robots',
 'safety',
 'science',
 'sk hynix',
 'startups',
 'stocks',
 'training',
 'voice mode',
 'white house',
 'x']

In [38]:
topic_df['extracted_topics'] = topic_df['extracted_topics'].apply(lambda l: [t.title() for t in l if t.lower() in filtered_topics])

In [39]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
for c in categories:
    print(c)

AI doom
AMD
Agriculture
Amazon
Anthropic
Apple
Art, Design
Artificial General Intelligence
Authors, Writing
Autonomous vehicles
Baidu
Big Tech
Bill Gates
Books, Publishing
Bubble
ChatGPT
China
Claude
Climate
Cloudflare
Code assistants
Cognitive Science
Cohere
Consciousness
Copilot
Cryptocurrency
Customer service
Cybersecurity
Data
Deals
Deepfakes
Disinformation
Disney
Economics
Education
Elon Musk
Energy
Entertainment
Ethics
European Union
Facial Recognition
Fashion
Finance
Fintech
Food, Drink
Funding
Gaming
Gemini
Gen AI
Geoffrey Hinton
Google
Health, Fitness
Healthcare
History
Hollywood
IPOs
India
Intellectual Property
Japan
Jobs, Careers, Labor Market
Korea
LLMs
Language Models
Laws
Lifestyle, Travel
Machine learning
Manufacturing
Mark Zuckerberg
Mergers and acquisitions
Meta
Microsoft
Military
Music
Mustafa Suleyman
Netflix
Nuclear
Nvidia
Open Source
OpenAI
Opinion
Perplexity
Policy and regulation
Politics
Privacy
Products
RAG
Recommendation systems
Reddit
Reinforcement Learning
Re

In [40]:
[t for t in filtered_topics if t.lower() not in [u.lower() for u in CANONICAL_TOPICS]]

['robotics',
 'investment',
 'startups',
 'data scraping',
 'white house',
 'safety',
 'hp',
 'voice mode',
 'x',
 'chatbot',
 'training',
 'msi',
 'amd ryzen',
 'laptops',
 'business',
 'mistral large 2',
 'nio',
 'sk hynix',
 'chip plant',
 'neural networks']

In [41]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,"""AI in Robotics: Revolutionizing the Future of...",https://news.google.com/articles/CBMiYWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiYWh0dHBzO...,news.google.com,Google News
1,1,Google News,2 Monster AI Growth Stocks to Buy Before They ...,https://news.google.com/articles/CBMiW2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiW2h0dHBzO...,news.google.com,Google News
2,2,Google News,"5 big analyst AI moves: PT hikes for Nvidia, A...",https://news.google.com/articles/CBMihQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMihQFodHRwc...,news.google.com,Google News
3,3,HackerNoon,A Human Writers Thoughts on AI Writing Bots,https://hackernoon.com/a-human-writers-thought...,True,2024-07-28,https://hackernoon.com/a-human-writers-thought...,hackernoon.com,Hacker Noon
4,4,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News
...,...,...,...,...,...,...,...,...,...
74,74,Google News,White House releases long-awaited FedRAMP mode...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMib2h0dHBzO...,news.google.com,Google News
75,75,Google News,"With Digital Twins, The Doctor Will See You Now",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News
76,76,Reddit,Xs new AI training opt-out setting draws regul...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit
77,77,Google News,iFixit CEO takes shots at Anthropic for 'hitti...,https://news.google.com/articles/CBMirQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMirQFodHRwc...,news.google.com,Google News


In [42]:
# maybe try to set timeout in categorize_headline
# https://platform.openai.com/settings/organization/limits
# rate limit is 10,000 rpm , we are sending like 100 at a time and waiting for response so should be OK

catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_headline(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        
catdict

2024-07-28 07:44:59,527 - AInewsbot - INFO - Categorizing headline 1 of 79
2024-07-28 07:44:59,528 - AInewsbot - INFO - "AI in Robotics: Revolutionizing the Future of Automation"
2024-07-28 07:45:00,747 - AInewsbot - INFO - ['Robots', 'Science']
2024-07-28 07:45:00,748 - AInewsbot - INFO - Categorizing headline 2 of 79
2024-07-28 07:45:00,748 - AInewsbot - INFO - 2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies
2024-07-28 07:45:01,769 - AInewsbot - INFO - ['Big Tech', 'Finance', 'Gen AI', 'Stocks']
2024-07-28 07:45:01,770 - AInewsbot - INFO - Categorizing headline 3 of 79
2024-07-28 07:45:01,771 - AInewsbot - INFO - 5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com
2024-07-28 07:45:02,989 - AInewsbot - INFO - ['Apple', 'Big Tech', 'Finance', 'Nvidia', 'Stocks']
2024-07-28 07:45:02,991 - AInewsbot - INFO - Categorizing headline 4 of 79
2024-07-28 07:45:02,991 - AInewsbot - INFO - A Human Writer

2024-07-28 07:45:36,959 - AInewsbot - INFO - Here's how artificial intelligence can help you sell your home
2024-07-28 07:45:38,361 - AInewsbot - INFO - ['Gen AI']
2024-07-28 07:45:38,362 - AInewsbot - INFO - Categorizing headline 27 of 79
2024-07-28 07:45:38,362 - AInewsbot - INFO - How to join the SearchGPT waitlist  heres how to get early access
2024-07-28 07:45:39,329 - AInewsbot - INFO - ['Gen AI', 'Products']
2024-07-28 07:45:39,330 - AInewsbot - INFO - Categorizing headline 28 of 79
2024-07-28 07:45:39,331 - AInewsbot - INFO - MSI Unveils High-End AMD Ryzen AI 300 Strix Powered Laptops For Gamers & Creators: Alpha 17 & Titan 18 Pro Detailed
2024-07-28 07:45:40,438 - AInewsbot - INFO - ['AMD', 'Gaming', 'Products']
2024-07-28 07:45:40,439 - AInewsbot - INFO - Categorizing headline 29 of 79
2024-07-28 07:45:40,440 - AInewsbot - INFO - MSI unveils fleet of new high-end AMD Ryzen AI 300 'Strix Point' APU-powered laptops
2024-07-28 07:45:41,631 - AInewsbot - INFO - ['AMD', 'Big Tech'

2024-07-28 07:46:19,749 - AInewsbot - INFO - ['Big Tech', 'ChatGPT', 'Gen AI', 'Google', 'LLMs', 'OpenAI', 'Products', 'Testing']
2024-07-28 07:46:19,750 - AInewsbot - INFO - Categorizing headline 52 of 79
2024-07-28 07:46:19,751 - AInewsbot - INFO - PRISE: A Unique Machine Learning Method for Learning Multitask Temporal Action Abstractions Using Natural Language Processing (NLP)
2024-07-28 07:46:20,788 - AInewsbot - INFO - ['Data', 'Gen AI', 'Language Models', 'Machine learning', 'Science']
2024-07-28 07:46:20,790 - AInewsbot - INFO - Categorizing headline 53 of 79
2024-07-28 07:46:20,791 - AInewsbot - INFO - Predicting a wildfire's next move? USC researchers using AI to forecast fire's likely path
2024-07-28 07:46:22,128 - AInewsbot - INFO - ['Climate', 'Data', 'Machine learning', 'Science']
2024-07-28 07:46:22,130 - AInewsbot - INFO - Categorizing headline 54 of 79
2024-07-28 07:46:22,130 - AInewsbot - INFO - Real-time neural network training on chip
2024-07-28 07:46:23,442 - AInews

2024-07-28 07:47:12,938 - AInewsbot - INFO - ['Health, Fitness', 'Healthcare', 'Science']
2024-07-28 07:47:12,939 - AInewsbot - INFO - Categorizing headline 77 of 79
2024-07-28 07:47:12,940 - AInewsbot - INFO - Xs new AI training opt-out setting draws regulatory scrutiny
2024-07-28 07:47:14,519 - AInewsbot - INFO - ['Big Tech', 'Data', 'Ethics', 'Gen AI', 'LLMs', 'Laws', 'Machine learning', 'Policy and regulation', 'Privacy']
2024-07-28 07:47:14,520 - AInewsbot - INFO - Categorizing headline 78 of 79
2024-07-28 07:47:14,520 - AInewsbot - INFO - iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves
2024-07-28 07:47:16,389 - AInewsbot - INFO - ['Anthropic', 'Big Tech', 'Gen AI', 'LLMs', 'Machine learning', 'Opinion']
2024-07-28 07:47:16,390 - AInewsbot - INFO - Categorizing headline 79 of 79
2024-07-28 07:47:16,390 - AInewsbot - INFO - iPhone 16 coming with two new buttons, AI chatbot and surprise color

{0: ['Robots', 'Science'],
 1: ['Big Tech', 'Finance', 'Gen AI', 'Stocks'],
 2: ['Apple', 'Big Tech', 'Finance', 'Nvidia', 'Stocks'],
 3: ['Authors, Writing',
  'ChatGPT',
  'Ethics',
  'Gen AI',
  'LLMs',
  'Language Models',
  'Opinion',
  'Society, Culture'],
 4: ['Ethics', 'Gen AI', 'Opinion', 'Robots', 'Science'],
 5: ['Anthropic',
  'Big Tech',
  'Data',
  'Ethics',
  'Gen AI',
  'Intellectual Property',
  'LLMs',
  'Language Models',
  'Laws',
  'Machine learning',
  'Policy and regulation',
  'Privacy'],
 6: ['Amazon',
  'Big Tech',
  'Entertainment',
  'Gen AI',
  'Machine learning',
  'Products',
  'Recommendation systems',
  'Streaming',
  'TV, Film, Movies'],
 7: ['Anthropic',
  'Big Tech',
  'Data',
  'Ethics',
  'Gen AI',
  'Laws',
  'Policy and regulation'],
 8: ['Apple',
  'Big Tech',
  'Ethics',
  'Gen AI',
  'Policy and regulation',
  'Politics'],
 9: ['Artificial General Intelligence',
  'Big Tech',
  'Gen AI',
  'Machine learning',
  'Robots',
  'Science',
  'Singul

In [43]:
topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))
topic_df

,id,extracted_topics,assigned_topics
0,0,[Robotics],"[Robots, Science]"
1,1,"[Stocks, Investment, Apple]","[Big Tech, Finance, Gen AI, Stocks]"
2,2,"[Nvidia, Apple]","[Apple, Big Tech, Finance, Nvidia, Stocks]"
3,3,[],"[Authors, Writing, ChatGPT, Ethics, Gen AI, LL..."
4,4,[Robots],"[Ethics, Gen AI, Opinion, Robots, Science]"
...,...,...,...
74,74,[White House],"[Cybersecurity, Data, Policy and regulation]"
75,75,[],"[Health, Fitness, Healthcare, Science]"
76,76,[Training],"[Big Tech, Data, Ethics, Gen AI, LLMs, Laws, M..."
77,77,[Anthropic],"[Anthropic, Big Tech, Gen AI, LLMs, Machine le..."


In [44]:
lcategories = set([c.lower() for c in categories])

In [45]:
def clean_topics(row):
    topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]
    return combined

topic_df["topics"] = topic_df.apply(clean_topics, axis=1)
topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)
topic_df

,id,extracted_topics,assigned_topics,topics,topic_str
0,0,[Robotics],"[Robots, Science]","[Robotics, Robots, Science]","Robotics, Robots, Science"
1,1,"[Stocks, Investment, Apple]","[Big Tech, Finance, Gen AI, Stocks]","[Apple, Big Tech, Finance, Gen AI, Investment,...","Apple, Big Tech, Finance, Gen AI, Investment, ..."
2,2,"[Nvidia, Apple]","[Apple, Big Tech, Finance, Nvidia, Stocks]","[Apple, Big Tech, Finance, Nvidia, Stocks]","Apple, Big Tech, Finance, Nvidia, Stocks"
3,3,[],"[Authors, Writing, ChatGPT, Ethics, Gen AI, LL...","[Authors, Writing, Chatgpt, Ethics, Gen AI, La...","Authors, Writing, Chatgpt, Ethics, Gen AI, Lan..."
4,4,[Robots],"[Ethics, Gen AI, Opinion, Robots, Science]","[Ethics, Gen AI, Opinion, Robots, Science]","Ethics, Gen AI, Opinion, Robots, Science"
...,...,...,...,...,...
74,74,[White House],"[Cybersecurity, Data, Policy and regulation]","[Cybersecurity, Data, Policy And Regulation, W...","Cybersecurity, Data, Policy And Regulation, Wh..."
75,75,[],"[Health, Fitness, Healthcare, Science]","[Health, Fitness, Healthcare, Science]","Health, Fitness, Healthcare, Science"
76,76,[Training],"[Big Tech, Data, Ethics, Gen AI, LLMs, Laws, M...","[Big Tech, Data, Ethics, Gen AI, Laws, Llms, M...","Big Tech, Data, Ethics, Gen AI, Laws, Llms, Ma..."
77,77,[Anthropic],"[Anthropic, Big Tech, Gen AI, LLMs, Machine le...","[Anthropic, Big Tech, Gen AI, Llms, Machine Le...","Anthropic, Big Tech, Gen AI, Llms, Machine Lea..."


In [46]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,"""AI in Robotics: Revolutionizing the Future of...",https://news.google.com/articles/CBMiYWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiYWh0dHBzO...,news.google.com,Google News
1,1,Google News,2 Monster AI Growth Stocks to Buy Before They ...,https://news.google.com/articles/CBMiW2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiW2h0dHBzO...,news.google.com,Google News
2,2,Google News,"5 big analyst AI moves: PT hikes for Nvidia, A...",https://news.google.com/articles/CBMihQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMihQFodHRwc...,news.google.com,Google News
3,3,HackerNoon,A Human Writers Thoughts on AI Writing Bots,https://hackernoon.com/a-human-writers-thought...,True,2024-07-28,https://hackernoon.com/a-human-writers-thought...,hackernoon.com,Hacker Noon
4,4,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News
...,...,...,...,...,...,...,...,...,...
74,74,Google News,White House releases long-awaited FedRAMP mode...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMib2h0dHBzO...,news.google.com,Google News
75,75,Google News,"With Digital Twins, The Doctor Will See You Now",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News
76,76,Reddit,Xs new AI training opt-out setting draws regul...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit
77,77,Google News,iFixit CEO takes shots at Anthropic for 'hitti...,https://news.google.com/articles/CBMirQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMirQFodHRwc...,news.google.com,Google News


In [47]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,"""AI in Robotics: Revolutionizing the Future of...",https://news.google.com/articles/CBMiYWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiYWh0dHBzO...,news.google.com,Google News
1,1,Google News,2 Monster AI Growth Stocks to Buy Before They ...,https://news.google.com/articles/CBMiW2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiW2h0dHBzO...,news.google.com,Google News
2,2,Google News,"5 big analyst AI moves: PT hikes for Nvidia, A...",https://news.google.com/articles/CBMihQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMihQFodHRwc...,news.google.com,Google News
3,3,HackerNoon,A Human Writers Thoughts on AI Writing Bots,https://hackernoon.com/a-human-writers-thought...,True,2024-07-28,https://hackernoon.com/a-human-writers-thought...,hackernoon.com,Hacker Noon
4,4,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News
...,...,...,...,...,...,...,...,...,...
74,74,Google News,White House releases long-awaited FedRAMP mode...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMib2h0dHBzO...,news.google.com,Google News
75,75,Google News,"With Digital Twins, The Doctor Will See You Now",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News
76,76,Reddit,Xs new AI training opt-out setting draws regul...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit
77,77,Google News,iFixit CEO takes shots at Anthropic for 'hitti...,https://news.google.com/articles/CBMirQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMirQFodHRwc...,news.google.com,Google News


In [48]:
try:  # for idempotency
    AIdf = AIdf.drop(columns=["title_topic_str"])
except:
    pass
try:  # for idempotency
    AIdf = AIdf.drop(columns=["topic_str"])
except:
    pass

AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="inner")
AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str
0,0,Google News,"""AI in Robotics: Revolutionizing the Future of...",https://news.google.com/articles/CBMiYWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiYWh0dHBzO...,news.google.com,Google News,"Robotics, Robots, Science","""AI in Robotics: Revolutionizing the Future of..."
1,1,Google News,2 Monster AI Growth Stocks to Buy Before They ...,https://news.google.com/articles/CBMiW2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiW2h0dHBzO...,news.google.com,Google News,"Apple, Big Tech, Finance, Gen AI, Investment, ...",2 Monster AI Growth Stocks to Buy Before They ...
2,2,Google News,"5 big analyst AI moves: PT hikes for Nvidia, A...",https://news.google.com/articles/CBMihQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMihQFodHRwc...,news.google.com,Google News,"Apple, Big Tech, Finance, Nvidia, Stocks","5 big analyst AI moves: PT hikes for Nvidia, A..."
3,3,HackerNoon,A Human Writers Thoughts on AI Writing Bots,https://hackernoon.com/a-human-writers-thought...,True,2024-07-28,https://hackernoon.com/a-human-writers-thought...,hackernoon.com,Hacker Noon,"Authors, Writing, Chatgpt, Ethics, Gen AI, Lan...",A Human Writers Thoughts on AI Writing Bots (T...
4,4,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News,"Ethics, Gen AI, Opinion, Robots, Science",AI is making robots smarter. Theyll need bound...
...,...,...,...,...,...,...,...,...,...,...,...
74,74,Google News,White House releases long-awaited FedRAMP mode...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMib2h0dHBzO...,news.google.com,Google News,"Cybersecurity, Data, Policy And Regulation, Wh...",White House releases long-awaited FedRAMP mode...
75,75,Google News,"With Digital Twins, The Doctor Will See You Now",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News,"Health, Fitness, Healthcare, Science","With Digital Twins, The Doctor Will See You No..."
76,76,Reddit,Xs new AI training opt-out setting draws regul...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-28,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit,"Big Tech, Data, Ethics, Gen AI, Laws, Llms, Ma...",Xs new AI training opt-out setting draws regul...
77,77,Google News,iFixit CEO takes shots at Anthropic for 'hitti...,https://news.google.com/articles/CBMirQFodHRwc...,True,2024-07-28,https://news.google.com/articles/CBMirQFodHRwc...,news.google.com,Google News,"Anthropic, Big Tech, Gen AI, Llms, Machine Lea...",iFixit CEO takes shots at Anthropic for 'hitti...


In [49]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(AIdf.loc[AIdf["topic_str"]==""][['title']])


,title


### Semantic sort

In [50]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

# sort of a traveling salesman sort
log(f"Sort with agglomerative cluster sort")
sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf['sort_order'] = sorted_indices

# do dimensionality reduction on embedding_df and cluster analysis
log(f"Perform dimensionality reduction")
with open("reducer.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
reduced_data = reducer.transform(embedding_df)
log(f"Cluster with DBSCAN")
dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
AIdf['cluster'] = dbscan.fit_predict(reduced_data)
AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999
    
# sort first by clusters found by DBSCAN, then by semantic ordering
AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

AIdf


2024-07-28 07:47:17,742 - AInewsbot - INFO - Fetching embeddings for 79 headlines
2024-07-28 07:47:18,170 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:47:18,634 - AInewsbot - INFO - Sort with agglomerative cluster sort
2024-07-28 07:47:18,648 - AInewsbot - INFO - Perform dimensionality reduction
2024-07-28 07:47:21,418 - AInewsbot - INFO - Cluster with DBSCAN


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster
0,0,Google News,OpenAI is testing web search features for Chat...,https://news.google.com/articles/CBMiUmh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiUmh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Google, Llms, OpenA...",OpenAI is testing web search features for Chat...,14,0
1,1,Google News,ChatGPTs Voice Mode update is coming next week...,https://news.google.com/articles/CBMiPWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiPWh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Language Models, Ll...",ChatGPTs Voice Mode update is coming next week...,20,0
2,2,Google News,Sci-fi ChatGPT Voice Mode that lets you talk t...,https://news.google.com/articles/CBMiZGh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiZGh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Entertainment, Gen AI, Lang...",Sci-fi ChatGPT Voice Mode that lets you talk t...,43,0
3,3,Google News,Google is Redesigning Chrome's Downloads using...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Cybersecurity, Gen AI, Google, Machi...",Google is Redesigning Chrome's Downloads using...,0,1
4,4,Google News,Google dangles bomber jackets and spot bonuses...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Google, Jobs, Careers, Labor...",Google dangles bomber jackets and spot bonuses...,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News,"Ethics, Gen AI, Opinion, Robots, Science",AI is making robots smarter. Theyll need bound...,74,999
75,75,Reddit,Mark Zuckerberg says there will be more AI age...,https://www.reddit.com/r/ChatGPT/comments/1ee1...,True,2024-07-28,https://www.reddit.com/r/ChatGPT/comments/1ee1...,www.reddit.com,Reddit,"Big Tech, Business, Gen AI, Llms, Mark Zuckerberg",Mark Zuckerberg says there will be more AI age...,75,999
76,76,Reddit,What are the chances that all this Marketing H...,https://www.reddit.com/r/ChatGPT/comments/1edk...,True,2024-07-28,https://www.reddit.com/r/ChatGPT/comments/1edk...,www.reddit.com,Reddit,"AI Doom, Bubble, Opinion",What are the chances that all this Marketing H...,76,999
77,77,Google News,The hot AI trade is entering a danger zone: Mo...,https://news.google.com/articles/CBMiR2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiR2h0dHBzO...,news.google.com,Google News,"AI Doom, Bubble, Economics, Finance, Gen AI, I...",The hot AI trade is entering a danger zone: Mo...,77,999


In [51]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):

    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses. 
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided 
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
            response = await fetch_openai(session, payload)
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}
        

# show clusters
cluster_topics = []
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    async with aiohttp.ClientSession() as session:

        for i in range(30):
            tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
            if len(tmpdf) ==0:
                break
            display(tmpdf)
            title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
            cluster_topic = await write_topic_name(session, title_topic_str_list)
            cluster_topics.append(cluster_topic)
            print(cluster_topic)

    


,id,title_topic_str
0,0,"OpenAI is testing web search features for ChatGPT, challenging Google (Topics: Big Tech, Chatgpt, Gen AI, Google, Llms, OpenAI, Products, Testing)"
1,1,"ChatGPTs Voice Mode update is coming next week: 3 new features subscribers will get (Topics: Big Tech, Chatgpt, Gen AI, Language Models, Llms, OpenAI, Products, Voice Mode)"
2,2,"Sci-fi ChatGPT Voice Mode that lets you talk to AI like a human lands next week trick to try it before... (Topics: Big Tech, Chatgpt, Entertainment, Gen AI, Language Models, Llms, OpenAI, Products, Science, Voice Mode)"


{'topic_title': 'ChatGPT and AI Innovations'}


,id,title_topic_str
3,3,"Google is Redesigning Chrome's Downloads using AI-powered Google Safe Browsing Tech (Topics: Big Tech, Cybersecurity, Gen AI, Google, Machine Learning, Privacy, Products)"
4,4,"Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show (Topics: Big Tech, Gen AI, Google, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning)"
5,5,"Google restates commitment to empowering businesses with AI (Topics: Big Tech, Gen AI, Google)"


{'topic_title': 'Google and AI Innovations'}


,id,title_topic_str
6,6,"Mistral AI Unveils Mistral Large 2: A Powerful New Language Model (Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science)"
7,7,"Mistral Large 2 is now available in Amazon Bedrock (Topics: Amazon, Big Tech, Gen AI, Language Models, Llms, Machine Learning, Mistral Large 2, Products)"
8,8,"Mistrals Large 2 is its answer to Meta and OpenAIs latest models (Topics: Big Tech, Gen AI, Google, Language Models, Llms, Machine Learning, Meta, Nvidia, OpenAI, Products)"


{'topic_title': 'Mistral Large 2 Language Model'}


,id,title_topic_str
9,9,"Nvidia supplier SK Hynix to build $6.8 billion chip plant in South Korea (Topics: Chip Plant, Deals, Economics, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix)"
10,10,"Nvidia supplier SK hynix to invest $6.8B to build new South Korean chip facility (Topics: Deals, Economics, Funding, Investment, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix)"
11,11,"Nvidia Supplier SK Hynix To Spend $6.8B on South Korea Plant To Meet AI Chip Demand (Topics: Big Tech, Chip Plant, Deals, Economics, Funding, Gen AI, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix)"


{'topic_title': 'Nvidia and Semiconductor Manufacturing Investments'}


In [52]:
# we could extract top words using tfidf, something like 
# vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = vectorizer.fit_transform(documents)
# feature_names = vectorizer.get_feature_names_out()
# topics = []
# for i in range(n_topics):
#     # Get the documents in this cluster
#     cluster_docs = [doc for doc, label in zip(documents, cluster_labels) if label == i]
#     cluster_metadatas = [meta for meta, label in zip(metadatas, cluster_labels) if label == i]

#     # Get the top words for this cluster based on TF-IDF scores
#     tfidf_scores = tfidf_matrix[cluster_labels == i].sum(axis=0).A1
#     top_word_indices = tfidf_scores.argsort()[-n_words_per_topic:][::-1]
#     top_words = [feature_names[index] for index in top_word_indices]


In [53]:
cluster_topic_list = [obj['topic_title'] for obj in cluster_topics]
cluster_topic_list

['ChatGPT and AI Innovations',
 'Google and AI Innovations',
 'Mistral Large 2 Language Model',
 'Nvidia and Semiconductor Manufacturing Investments']

In [54]:
AIdf['cluster_name'] = AIdf['cluster'].apply(lambda i: cluster_topic_list[i] if i<len(cluster_topic_list) else "")


# Save and email headlines


In [55]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.site_name}]({row.actual_url})")
    html_str += f'{row.Index}.<a href="{row.actual_url}">{row.title} - {row.site_name}</a><br />\n'


2024-07-28 07:47:23,490 - AInewsbot - INFO - [0. OpenAI is testing web search features for ChatGPT, challenging Google - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)
2024-07-28 07:47:23,491 - AInewsbot - INFO - [1. ChatGPTs Voice Mode update is coming next week: 3 new features subscribers will get - Google News](https://news.google.com/articles/CBMiPWh0dHBzOi8vbWFzaGFibGUuY29tL2FydGljbGUvb3BlbmFpLWNoYXRncHQtdm9pY2UtbW9kZS11cGRhdGXSAQA?hl=en-US&gl=US&ceid=US:en)
2024-07-28 07:47:23,491 - AInewsbot - INFO - [2. Sci-fi ChatGPT Voice Mode that lets you talk to AI like a human lands next week  trick to try it before... - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC

2024-07-28 07:47:23,498 - AInewsbot - INFO - [24. A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
2024-07-28 07:47:23,498 - AInewsbot - INFO - [25. What's something you use ChatGPT for that you're sure no one else does? - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/)
2024-07-28 07:47:23,498 - AInewsbot - INFO - [26. Spain is using AI for domestic violence risk assessment and its going as well as youd expect  At least 247 women have been killed by their current or former partner since 2007, as police accept VioGen risk scores about 95 percent of the time - Reddit](https://www.reddit.com/r/technology/comments/1edvhk5/spain_is_using_ai_for_domestic_violence_risk/)
2024-07-28 07:47:23,499 - AInewsbot - INFO - [27. SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwc

2024-07-28 07:47:23,504 - AInewsbot - INFO - [51. 5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-28 07:47:23,504 - AInewsbot - INFO - [52. Surreal AI Images of Basketball Courts Carved Into Ancient Canyons - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vbXltb2Rlcm5tZXQuY29tL2FpLWdlbmVyYXRlZC1iYXNrZXRiYWxsLWNvdXJ0cy1pbi1zYXVkaS1hcmFiaWEtY2FueW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)
2024-07-28 07:47:23,504 - AInewsbot - INFO - [53. The Elegant Math of Machine Learning - Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
2024-07-28 07:47:23,505 - AInewsbot - INFO - [54. Spectral characterization of intraoperative renal perfusion using hyp

2024-07-28 07:47:23,509 - AInewsbot - INFO - [73. Small neural network enables realistic rendering of woven fabrics in real-time - TechXplore](https://techxplore.com/news/2024-07-lightweight-neural-network-enables-realistic.html)
2024-07-28 07:47:23,509 - AInewsbot - INFO - [74. AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
2024-07-28 07:47:23,509 - AInewsbot - INFO - [75. Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
2024-07-28 07:47:23,509 - AInewsbot - INFO - [76. What are the chances that all this Marketing Hype? And AI remains as basic as it is now. 

In [56]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [57]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-28 07:47:23,517 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [58]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.actual_url, row.title))


2024-07-28 07:47:25,116 - AInewsbot - INFO - Queuing URLs for scraping


In [59]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-07-28 07:47:25,131 - AInewsbot - INFO - fetching 79 pages using 4 browsers
2024-07-28 07:47:25,135 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:47:25,138 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:47:25,138 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:47:25,139 - AInewsbot - INFO - get_driver - 16559 Initializing webdriver
2024-07-28 07:47:40,930 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:47:40,931 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-28 07:47:40,932 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:47:40,932 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:47:40,932 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-28 07:47:40,933 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-28 07:47:40,933 - AInewsbot - INFO -

2024-07-28 07:48:43,732 - AInewsbot - INFO - get_url(Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show) - Saving Google_dangles_bomber_jackets_and_spot_bonuses_in_search_for__Golden_Prompts__for_AI__leaked_documents_show_20240728_074843.html as utf-8
2024-07-28 07:48:43,734 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:48:43,734 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:48:46,845 - AI

2024-07-28 07:48:57,507 - AInewsbot - INFO - Processing https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/
2024-07-28 07:48:57,514 - AInewsbot - INFO - get_url(https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/) - starting get_url https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/
2024-07-28 07:49:04,181 - AInewsbot - INFO - get_url(Real-time neural network training on chip) - Saving Real-time_neural_network_training_on_chip_20240728_074904.html as utf-8
2024-07-28 07:49:04,185 - AInewsbot - INFO - Processing https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/
2024-07-28 07:49:04,186 - AInewsbot - INFO - get_url(https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/) - starting get_url https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt

2024-07-28 07:49:28,906 - AInewsbot - INFO - get_url(OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report) - Saving OpenAI__Home_Of_ChatGPT__May_Lose__5B_This_Year__Report_20240728_074928.html as utf-8
2024-07-28 07:49:28,910 - AInewsbot - INFO - Processing https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/
2024-07-28 07:49:28,912 - AInewsbot - INFO - get_url(https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/) - starting get_url https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/
2024-07-28 07:49:29,503 - AInewsbot - INFO - get_url(Autonomous delivery startup Nuro is gearing up for a comeback) - Saving Autonomous_delivery_startup_Nuro_is_gearing_up_for_a_comeback_20240728_074929.html as utf-8
2024-07-28 07:49:29,507 - AInewsbot - INFO - Processing https://www.reddit.com/r/technology/comments/1edvhk5/spain_is_using_ai_for_domestic_violence_risk/


2024-07-28 07:49:55,807 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2JlYTlkZjcxLTM3MWMtNDA0NS05Y2I0LTY0YzIyNzg5YmY3YtIBAA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2JlYTlkZjcxLTM3MWMtNDA0NS05Y2I0LTY0YzIyNzg5YmY3YtIBAA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:50:01,664 - AInewsbot - INFO - get_url(As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market) - Saving As_HPs_new_Copilot_Plus_AI_PCs_reach_India__it_sets_its_sights_on_businesses_to_break_into_the_market_20240728_075001.html as utf-8
2024-07-28 07:50:01,666 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LnRvbXNndWlkZS5jb20vYWkvaG93LXRvLWFjY2Vzcy1zZWFyY2hncHQtaGVyZXMtaG93LXRvLXNpZ24tdXAtZm9yLXRoZS13YWl0bGlzdNIBAA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:50:01,667 - AInewsbot - INFO - get_url(https://news.google.com

2024-07-28 07:50:25,648 - AInewsbot - INFO - get_url(Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst.) - Saving Nvidia_Stock_Could_Surge__Another__259__by_2030__This_Artificial_Intelligence__AI__Stock_Could_Soar_Even_Higher__According_to_1_Wall_Street_Analyst__20240728_075025.html as utf-8
2024-07-28 07:50:25,650 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMingFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL3N0cmVhbWluZy9hbWF6b24tcHJpbWUtdmlkZW8vYW1hem9uLWp1c3QtZ2F2ZS1wcmltZS12aWRlby1zdWJzY3JpYmVycy1hLWdyZWF0LWZyZWUtdXBkYXRlLXRoYXQtdXNlcy1haS10by1yZWNvbW1lbmQtbW92aWVzLWFuZC1zaG93c9IBAA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:50:25,651 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMingFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL3N0cmVhbWluZy9hbWF6b24tcHJpbWUtdmlkZW8vYW1hem9uLWp1c3QtZ2F2ZS1wcmltZS12aWRlby1zdWJzY3JpYmVycy1hLWdyZWF0LWZyZWUtdXBkYXRlLXRoYXQtdXNlcy1haS10by1yZ

2024-07-28 07:50:47,907 - AInewsbot - INFO - get_url("AI in Robotics: Revolutionizing the Future of Automation") - Saving _AI_in_Robotics__Revolutionizing_the_Future_of_Automation__20240728_075047.html as utf-8
2024-07-28 07:50:47,909 - AInewsbot - INFO - Processing https://nautil.us/the-elegant-math-of-machine-learning-727842/
2024-07-28 07:50:47,911 - AInewsbot - INFO - get_url(https://nautil.us/the-elegant-math-of-machine-learning-727842/) - starting get_url https://nautil.us/the-elegant-math-of-machine-learning-727842/
2024-07-28 07:50:48,402 - AInewsbot - INFO - get_url(5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com) - Saving 5_big_analyst_AI_moves__PT_hikes_for_Nvidia__Apple__Tesla_downside_risk_flagged_By_Investing_com_20240728_075048.html as utf-8
2024-07-28 07:50:48,410 - AInewsbot - INFO - get_url(2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies) - Saving 2_Monster_AI_Growth_Stock

2024-07-28 07:51:04,677 - AInewsbot - INFO - get_url(My new iPhone symbolises stagnation, not innovation  and a similar fate awaits AI) - Saving My_new_iPhone_symbolises_stagnation__not_innovation__and_a_similar_fate_awaits_AI_20240728_075104.html as utf-8
2024-07-28 07:51:04,683 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:51:04,683 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://ne

2024-07-28 07:51:26,890 - AInewsbot - INFO - get_url(NIO unveils new operating system, 5nm chip, and phone at an AI event in Shanghai) - Saving NIO_unveils_new_operating_system__5nm_chip__and_phone_at_an_AI_event_in_Shanghai_20240728_075126.html as utf-8
2024-07-28 07:51:26,894 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiZ2h0dHBzOi8vd2NjZnRlY2guY29tL21zaS1oaWdoLWVuZC1hbWQtcnl6ZW4tYWktMzAwLXN0cml4LWxhcHRvcHMtZ2FtZXJzLWNyZWF0b3JzLWFscGhhLTE3LXRpdGFuLTE4LXByby_SAWtodHRwczovL3djY2Z0ZWNoLmNvbS9tc2ktaGlnaC1lbmQtYW1kLXJ5emVuLWFpLTMwMC1zdHJpeC1sYXB0b3BzLWdhbWVycy1jcmVhdG9ycy1hbHBoYS0xNy10aXRhbi0xOC1wcm8vYW1wLw?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:51:26,897 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiZ2h0dHBzOi8vd2NjZnRlY2guY29tL21zaS1oaWdoLWVuZC1hbWQtcnl6ZW4tYWktMzAwLXN0cml4LWxhcHRvcHMtZ2FtZXJzLWNyZWF0b3JzLWFscGhhLTE3LXRpdGFuLTE4LXByby_SAWtodHRwczovL3djY2Z0ZWNoLmNvbS9tc2ktaGlnaC1lbmQtYW1kLXJ5emVuLWFpLTMwMC1zdHJpeC1sYXB0b3BzLWdhbWVycy1jcmVhdG9yc

2024-07-28 07:51:55,224 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en
2024-07-28 07:51:57,832 - AInewsbot - INFO - get_url(AI is making robots smarter. Theyll need boundaries.) - Saving AI_is_making_robots_smarter__Theyll_need_boundaries__20240728_075157.html as utf-8
2024-07-28 07:51:57,836 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vc2NpZW50aXN0cy1haS1kZWNvZGUtc3Blcm0td2hhbGUtc291bmRzLWRpc2NvdmVyLWFscGhhYmV0LWNvbXBsZXgtbGFuZ3VhZ2UtMjAyNC030gF1aHR0cHM6Ly93d3cuYnVzaW5lc3NpbnNpZGVyLmNvbS9zY2llbnRpc3RzLWFpLWRlY29kZS1zcGVybS13aGFsZS1zb3VuZHMtZGlzY292ZXItYWxwaGFiZXQtY29tcGxleC1sYW5ndWFnZS0yMDI0LTc

In [60]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'actual_url', 'title', 'path']
pages_df

,id,actual_url,title,path
0,3,https://news.google.com/articles/CBMiSWh0dHBzO...,Google is Redesigning Chrome's Downloads using...,htmldata/Google_is_Redesigning_Chrome_s_Downlo...
1,7,https://news.google.com/articles/CBMiYWh0dHBzO...,Mistral Large 2 is now available in Amazon Bed...,htmldata/Mistral_Large_2_is_now_available_in_A...
2,11,https://news.google.com/articles/CBMiemh0dHBzO...,Nvidia Supplier SK Hynix To Spend $6.8B on Sou...,htmldata/Nvidia_Supplier_SK_Hynix_To_Spend__6_...
3,15,https://www.reddit.com/r/technology/comments/1...,TikToks algorithm is highly sensitive and cou...,htmldata/TikToks_algorithm_is_highly_sensitive...
4,19,https://news.google.com/articles/CBMiNGh0dHBzO...,HPs New Elitebook Laptop Has the Best Battery ...,htmldata/HPs_New_Elitebook_Laptop_Has_the_Best...
...,...,...,...,...
74,63,https://news.google.com/articles/CBMib2h0dHBzO...,White House releases long-awaited FedRAMP mode...,htmldata/White_House_releases_long-awaited_Fed...
75,66,https://news.google.com/articles/CBMiXGh0dHBzO...,Mistral Large 2: The David to Big Techs Goliat...,htmldata/Mistral_Large_2__The_David_to_Big_Tec...
76,70,https://www.reddit.com/r/technology/comments/1...,Xs new AI training opt-out setting draws regul...,htmldata/Xs_new_AI_training_opt-out_setting_dr...
77,74,https://news.google.com/articles/CBMiS2h0dHBzO...,AI is making robots smarter. Theyll need bound...,htmldata/AI_is_making_robots_smarter__Theyll_n...


In [61]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [62]:
AIdf

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,OpenAI is testing web search features for Chat...,https://news.google.com/articles/CBMiUmh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiUmh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Google, Llms, OpenA...",OpenAI is testing web search features for Chat...,14,0,ChatGPT and AI Innovations,htmldata/OpenAI_is_testing_web_search_features...
1,1,Google News,ChatGPTs Voice Mode update is coming next week...,https://news.google.com/articles/CBMiPWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiPWh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Language Models, Ll...",ChatGPTs Voice Mode update is coming next week...,20,0,ChatGPT and AI Innovations,htmldata/ChatGPTs_Voice_Mode_update_is_coming_...
2,2,Google News,Sci-fi ChatGPT Voice Mode that lets you talk t...,https://news.google.com/articles/CBMiZGh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiZGh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Entertainment, Gen AI, Lang...",Sci-fi ChatGPT Voice Mode that lets you talk t...,43,0,ChatGPT and AI Innovations,htmldata/Sci-fi_ChatGPT_Voice_Mode_that_lets_y...
3,3,Google News,Google is Redesigning Chrome's Downloads using...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Cybersecurity, Gen AI, Google, Machi...",Google is Redesigning Chrome's Downloads using...,0,1,Google and AI Innovations,htmldata/Google_is_Redesigning_Chrome_s_Downlo...
4,4,Google News,Google dangles bomber jackets and spot bonuses...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Google, Jobs, Careers, Labor...",Google dangles bomber jackets and spot bonuses...,25,1,Google and AI Innovations,htmldata/Google_dangles_bomber_jackets_and_spo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News,"Ethics, Gen AI, Opinion, Robots, Science",AI is making robots smarter. Theyll need bound...,74,999,,htmldata/AI_is_making_robots_smarter__Theyll_n...
75,75,Reddit,Mark Zuckerberg says there will be more AI age...,https://www.reddit.com/r/ChatGPT/comments/1ee1...,True,2024-07-28,https://www.reddit.com/r/ChatGPT/comments/1ee1...,www.reddit.com,Reddit,"Big Tech, Business, Gen AI, Llms, Mark Zuckerberg",Mark Zuckerberg says there will be more AI age...,75,999,,htmldata/Mark_Zuckerberg_says_there_will_be_mo...
76,76,Reddit,What are the chances that all this Marketing H...,https://www.reddit.com/r/ChatGPT/comments/1edk...,True,2024-07-28,https://www.reddit.com/r/ChatGPT/comments/1edk...,www.reddit.com,Reddit,"AI Doom, Bubble, Opinion",What are the chances that all this Marketing H...,76,999,,htmldata/What_are_the_chances_that_all_this_Ma...
77,77,Google News,The hot AI trade is entering a danger zone: Mo...,https://news.google.com/articles/CBMiR2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiR2h0dHBzO...,news.google.com,Google News,"AI Doom, Bubble, Economics, Finance, Gen AI, I...",The hot AI trade is entering a danger zone: Mo...,77,999,,htmldata/The_hot_AI_trade_is_entering_a_danger...


# Summarize individual pages

In [63]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will focus on the top 3 points of the text and keep the response concise."
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will output Markdown format.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [64]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [65]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-07-28 07:52:16,250 - AInewsbot - INFO - Starting summarize
2024-07-28 07:52:16,310 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI is testing web search features, challenging Google - The Washington Post
Social card title: OpenAI is testing web search features for ChatGPT, challenging Google
Social card description: Tech experts have predicted that search could be upended by AI, but Google’s dominance remains largely untouched.

2024-07-28 07:52:16,431 - AInewsbot - INFO - fetch_all_summaries - Page title: ChatGPT’s Voice Mode update is coming next week: 3 new features subscribers will get | Mashable
Social card title: ChatGPT’s Voice Mode update is coming next week: 3 new features subscribers will get
Social card description: Live translation, sound effects, and more.

2024-07-28 07:52:16,470 - AInewsbot - INFO - fetch_all_summaries - Page title: Sci-fi ChatGPT ‘Voice Mode’ that lets you talk to AI like a human lands next week – trick to try it before everyone else 

2024-07-28 07:52:18,391 - AInewsbot - INFO - fetch_all_summaries - Page title: Autonomous delivery startup Nuro is gearing up for a comeback : r/technology

2024-07-28 07:52:18,452 - AInewsbot - INFO - fetch_all_summaries - Page title: Subscribe to read

2024-07-28 07:52:18,531 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI, Home Of ChatGPT, May Lose $5B This Year - Report
Social card title: OpenAI, Home Of ChatGPT, May Lose $5B This Year – Report
Social card description: Artificial Intelligence development is costly, as a new report on OpenAI claims.

2024-07-28 07:52:18,598 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI faces bankruptcy within a year, projecting $5B losses | Windows Central
Social card title: OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses
Social card description: OpenAI might need another round of funding to remain afloat.

2024-07-28 07:52:18,807 - AInewsbot - INFO - fetch_all_sum

2024-07-28 07:52:21,416 - AInewsbot - INFO - fetch_all_summaries - Page title: Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol
Social card title: Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol
Social card description: iFixit and Freelancer have accused Anthropic, the AI startup behind the Claude large language models, of ignoring their "do not crawl" robots.txt protocol and policy to scrape their websites' data.

2024-07-28 07:52:21,451 - AInewsbot - INFO - fetch_all_summaries - Page title: Anthropic’s crawler is ignoring websites’ anti-AI scraping policies - The Verge
Social card title: Anthropic’s crawler is ignoring websites’ anti-AI scraping policies
Social card description: iFixit’s CEO says ClaudeBot tied up its DevOps resources.

2024-07-28 07:52:21,487 - AInewsbot - INFO - fetch_all_summaries - Page title: "AI in Robotics: Revolutionizing the Future of Automation" | DC Velocity
Social card title:

2024-07-28 07:52:22,491 - AInewsbot - INFO - fetch_all_summaries - Page title: PR Week. The leading source of news, insight and analysis on the Public Relations Industry

2024-07-28 07:52:22,526 - AInewsbot - INFO - fetch_all_summaries - Page title: Mistral Large 2: The David to Big Tech's Goliath(s)
Social card title: Mistral Large 2: The David to Big Tech's Goliath(s)
Social card description: Mistral AI’s latest model, Mistral Large 2 (ML2), allegedly competes with large models from industry leaders like OpenAI, Meta, and Anthropic, despite being a fraction of their sizes.

2024-07-28 07:52:22,555 - AInewsbot - INFO - fetch_all_summaries - Page title: Wearable sensor tracks steps to detect neurological disorders - The Jerusalem Post
Social card title: Tel Aviv researchers develop wearable sensor capable of neurological disorder detection
Social card description: Imagine detecting early signs of neurological disorders just by tracking your steps. Researchers at Tel Aviv University hav

(0, {'id': 'chatcmpl-9pwjoTx8Oq4CwqQ1pmhzO0QGxaD1L', 'object': 'chat.completion', 'created': 1722167544, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI.\n- Concerns from publishers about the potential negative impact on their businesses are rising, as AI-generated answers might undercut traffic from traditional search results, despite OpenAI's efforts to collaborate with media organizations.\n- The new search tool aims to provide sources and direct links to information, while OpenAI signals future improvements that may include advertisements, further competing with Google's established business model."}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 863, 'completion_tokens': 104, 'total_tokens': 967}, 'system_fingerprint': 'fp_611b667b19'

In [66]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

79

In [67]:
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))


In [68]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,OpenAI is testing web search features for Chat...,https://news.google.com/articles/CBMiUmh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiUmh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Google, Llms, OpenA...",OpenAI is testing web search features for Chat...,14,0,ChatGPT and AI Innovations,htmldata/OpenAI_is_testing_web_search_features...
1,1,Google News,ChatGPTs Voice Mode update is coming next week...,https://news.google.com/articles/CBMiPWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiPWh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Language Models, Ll...",ChatGPTs Voice Mode update is coming next week...,20,0,ChatGPT and AI Innovations,htmldata/ChatGPTs_Voice_Mode_update_is_coming_...
2,2,Google News,Sci-fi ChatGPT Voice Mode that lets you talk t...,https://news.google.com/articles/CBMiZGh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiZGh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Entertainment, Gen AI, Lang...",Sci-fi ChatGPT Voice Mode that lets you talk t...,43,0,ChatGPT and AI Innovations,htmldata/Sci-fi_ChatGPT_Voice_Mode_that_lets_y...
3,3,Google News,Google is Redesigning Chrome's Downloads using...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Cybersecurity, Gen AI, Google, Machi...",Google is Redesigning Chrome's Downloads using...,0,1,Google and AI Innovations,htmldata/Google_is_Redesigning_Chrome_s_Downlo...
4,4,Google News,Google dangles bomber jackets and spot bonuses...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Google, Jobs, Careers, Labor...",Google dangles bomber jackets and spot bonuses...,25,1,Google and AI Innovations,htmldata/Google_dangles_bomber_jackets_and_spo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,Google News,AI is making robots smarter. Theyll need bound...,https://news.google.com/articles/CBMiS2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiS2h0dHBzO...,news.google.com,Google News,"Ethics, Gen AI, Opinion, Robots, Science",AI is making robots smarter. Theyll need bound...,74,999,,htmldata/AI_is_making_robots_smarter__Theyll_n...
75,75,Reddit,Mark Zuckerberg says there will be more AI age...,https://www.reddit.com/r/ChatGPT/comments/1ee1...,True,2024-07-28,https://www.reddit.com/r/ChatGPT/comments/1ee1...,www.reddit.com,Reddit,"Big Tech, Business, Gen AI, Llms, Mark Zuckerberg",Mark Zuckerberg says there will be more AI age...,75,999,,htmldata/Mark_Zuckerberg_says_there_will_be_mo...
76,76,Reddit,What are the chances that all this Marketing H...,https://www.reddit.com/r/ChatGPT/comments/1edk...,True,2024-07-28,https://www.reddit.com/r/ChatGPT/comments/1edk...,www.reddit.com,Reddit,"AI Doom, Bubble, Opinion",What are the chances that all this Marketing H...,76,999,,htmldata/What_are_the_chances_that_all_this_Ma...
77,77,Google News,The hot AI trade is entering a danger zone: Mo...,https://news.google.com/articles/CBMiR2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiR2h0dHBzO...,news.google.com,Google News,"AI Doom, Bubble, Economics, Finance, Gen AI, I...",The hot AI trade is entering a danger zone: Mo...,77,999,,htmldata/The_hot_AI_trade_is_entering_a_danger...


In [69]:
# make text for email and also collect data for vector store
markdown_str = ''
vectorstore_list = []
metadata_list=[]
for i, row in enumerate(AIdf.itertuples()):
    topics = []
    if row.cluster_name:
        topics.append(row.cluster_name)
    if row.topic_str:
        topics.append(row.topic_str)
    topic_str = ", ".join(topics)

    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
    # simpler version for vector store
    vectorstore_list.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
    metadata_list.append({'id': row.id, 'title': row.title, 'url': row.actual_url, 'site': row.site_name})
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. OpenAI is testing web search features for ChatGPT, challenging Google - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)  

 ChatGPT and AI Innovations, Big Tech, Chatgpt, Gen AI, Google, Llms, OpenAI, Products, Testing  

- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI.
- Concerns from publishers about the potential negative impact on their businesses are rising, as AI-generated answers might undercut traffic from traditional search results, despite OpenAI's efforts to collaborate with media organizations.
- The new search tool aims to provide sources and direct links to information, while OpenAI signals future improvements that may include advertisements, further competing with Google's established business model. 



[2. ChatGPTs Voice Mode update is coming next week: 3 new features subscribers will get - Google News](https://news.google.com/articles/CBMiPWh0dHBzOi8vbWFzaGFibGUuY29tL2FydGljbGUvb3BlbmFpLWNoYXRncHQtdm9pY2UtbW9kZS11cGRhdGXSAQA?hl=en-US&gl=US&ceid=US:en)  

 ChatGPT and AI Innovations, Big Tech, Chatgpt, Gen AI, Language Models, Llms, OpenAI, Products, Voice Mode  

- ChatGPT's Voice Mode update will be available to Plus members next week, enhancing subscription value with new features.
- The update includes live translation between languages, the ability to create sound effects for storytelling, and custom character voices.
- The rollout was delayed from June but will offer users more interactive and dynamic conversations with reduced response times. 



[3. Sci-fi ChatGPT Voice Mode that lets you talk to AI like a human lands next week  trick to try it before... - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en)  

 ChatGPT and AI Innovations, Big Tech, Chatgpt, Entertainment, Gen AI, Language Models, Llms, OpenAI, Products, Science, Voice Mode  

- OpenAI's upgraded Voice Mode for ChatGPT, powered by the new GPT-4o model, will enable more humanlike spoken conversations and is set to launch next week for Plus subscribers.
- The new Voice Mode features significantly reduced response times, averaging around 320 milliseconds compared to 5.4 seconds with the previous generation, allowing for smoother interactions.
- GPT-4o will also support real-time translation, enabling conversations between users speaking different languages, and offers the ability to adjust speaking tones and pace for personalized interactions. 



[4. Google is Redesigning Chrome's Downloads using AI-powered Google Safe Browsing Tech - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3Lmdpem1vY2hpbmEuY29tLzIwMjQvMDcvMjcvZ29vZ2xlLWNocm9tZS1kb3dubG9hZHMtYWktcG93ZXJlZC_SAQA?hl=en-US&gl=US&ceid=US:en)  

 Google and AI Innovations, Big Tech, Cybersecurity, Gen AI, Google, Machine Learning, Privacy, Products  

- Google Chrome is enhancing its download experience with AI-powered Google Safe Browsing technology to distinguish between “suspicious files” and “dangerous documents.”
- Users with Enhanced Protection mode will automatically have their downloads deeply scanned, reducing the need for manual checks and preventing malicious downloads.
- The redesign aims to improve clarity in download warnings, thereby increasing overall online safety and reducing instances of users ignoring security measures. 



[5. Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)  

 Google and AI Innovations, Big Tech, Gen AI, Google, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning  

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software, offering prizes like golden bomber jackets.
- The competition aimed to develop effective prompts for tasks such as managing customer inquiries, invoicing, and business leads to compete with Microsoft's AI capabilities in Office 365.
- Seven winning prompts were selected, including solutions for responding to Google reviews and organizing receipts, with plans to integrate them into future Workspace updates. 



...

[75. AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)  

 Ethics, Gen AI, Opinion, Robots, Science  

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage. 



[76. Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)  

 Big Tech, Business, Gen AI, Llms, Mark Zuckerberg  

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors. 



[77. What are the chances that all this Marketing Hype? And AI remains as basic as it is now. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/)  

 AI Doom, Bubble, Opinion  

- Current AI, particularly LLMs, has potential use cases but fundamentally lacks the ability to create high-quality music, journalism, or cinema, with outputs primarily summarizing existing knowledge.
- The hype surrounding AI advancements is often viewed as marketing rather than reality, with experts agreeing that the field faces significant challenges and technological plateaus within a few years.
- The financial sustainability of AI companies is in jeopardy as they continue to burn through capital without a clear revenue model, raising questions about how long they can persist before funding runs out. 



[78. The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)  

 AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks  

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization. 



[79. Scientists use AI to decode sperm whale sounds, find phonetic alphabet - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vc2NpZW50aXN0cy1haS1kZWNvZGUtc3Blcm0td2hhbGUtc291bmRzLWRpc2NvdmVyLWFscGhhYmV0LWNvbXBsZXgtbGFuZ3VhZ2UtMjAyNC030gF1aHR0cHM6Ly93d3cuYnVzaW5lc3NpbnNpZGVyLmNvbS9zY2llbnRpc3RzLWFpLWRlY29kZS1zcGVybS13aGFsZS1zb3VuZHMtZGlzY292ZXItYWxwaGFiZXQtY29tcGxleC1sYW5ndWFnZS0yMDI0LTc_YW1w?hl=en-US&gl=US&ceid=US:en)  

 Gen AI, Machine Learning, Science  

- Researchers at Project CETI are using AI to decode the complex communication patterns of sperm whales, discovering a phonetic alphabet within their clicks.  
- The AI model showed remarkable accuracy in predicting whale clicks, indicating that their communication follows structured linguistic patterns.  
- Understanding sperm whale language aims to mitigate noise pollution's impact on their behavior, ultimately aiding in their protection. 



In [70]:
display(Markdown( vectorstore_list[16]))

[ChatGPT Is Not Providing the Accurate Information I Need. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/)

Topics: Chatgpt, Customer Service, Disinformation, Gen AI, Language Models, Llms, OpenAI, Opinion, Review 

- Users are experiencing issues with ChatGPT's accuracy, particularly in obtaining detailed professional profiles and accurate industry trends, leading to challenges in their work.
- Suggestions for alternatives include using Myko Assistant, which is reported to provide more reliable information without complicated setup processes.
- ChatGPT is primarily a language generator and needs specific prompts to yield accurate results; it does not inherently understand truth and can create incorrect information instead of admitting limitations.



In [71]:
print(metadata_list[16])


{'id': 16, 'title': 'ChatGPT Is Not Providing the Accurate Information I Need.', 'url': 'https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/', 'site': 'Reddit'}


In [72]:
# Create Document objects with the paragraphs and corresponding metadata
docs = [Document(page_content=paragraph, metadata=meta) 
        for paragraph, meta in zip(vectorstore_list, metadata_list)]
len(docs)

79

In [73]:
print(docs[16])


page_content="[ChatGPT Is Not Providing the Accurate Information I Need. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/)\n\nTopics: Chatgpt, Customer Service, Disinformation, Gen AI, Language Models, Llms, OpenAI, Opinion, Review \n\n- Users are experiencing issues with ChatGPT's accuracy, particularly in obtaining detailed professional profiles and accurate industry trends, leading to challenges in their work.\n- Suggestions for alternatives include using Myko Assistant, which is reported to provide more reliable information without complicated setup processes.\n- ChatGPT is primarily a language generator and needs specific prompts to yield accurate results; it does not inherently understand truth and can create incorrect information instead of admitting limitations.\n\n" metadata={'id': 16, 'title': 'ChatGPT Is Not Providing the Accurate Information I Need.', 'url': 'https://www.reddit.com/r/ChatGPT/comments/1edq7am/chat

In [74]:
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_openai"
try:
    del vectorstore
except Exception as e:
    log(f"{e}")

try:
    shutil.rmtree(persist_directory)
    log(f"Directory '{persist_directory}' and all its contents have been removed successfully.")
except Exception as e:
    log(f"Remove directory error: {e}")
        
embeddings_openAI = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, embeddings_openAI)


2024-07-28 07:52:27,748 - AInewsbot - INFO - name 'vectorstore' is not defined
2024-07-28 07:52:27,749 - AInewsbot - INFO - Remove directory error: name 'persist_directory' is not defined
2024-07-28 07:52:28,488 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-28 07:52:29,780 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [75]:
# Perform a similarity search
query = "What is the latest with openai?"
results = vectorstore.similarity_search_with_score(query, 
                                        k=20,
                                       )  # k is the number of results to return
# Print the results
urldict = {}
for doc, score in results:
    if urldict.get(doc.metadata['url']):
        continue
    urldict[doc.metadata['url']] = 1
    if score < 1.25:
        print(f"Score:   {score}")
        print(f"Content: {doc.page_content}\n")
        print(f"Metadata: {doc.metadata}\n")
        print("---")

2024-07-28 07:52:30,326 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Score:   0.8109857439994812
Content: [OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses - Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Finance, Funding, Gen AI, OpenAI 

- OpenAI may face bankruptcy within a year, projecting losses of $5 billion due to high operational costs of $7 billion for AI model training and $1.5 billion for staffing.
- Despite generating approximately $3.5 billion in annual revenue, OpenAI's expenses significantly exceed its income, leading to financial strain.
- The company has raised over $11 billion through seven funding rounds but may need additional funding to continue operations while pursuing its goal of artificial ge

In [76]:
# # or use local embeddings with sentence_transformers
# # Initialize your embedding model
# embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the vector store with a persist_directory
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_huggingface"
# vectorstore_hf = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings_hf,
#     persist_directory=persist_directory
# )

# # Perform a similarity search
# query = "What is the latest with OpenAI?"
# results = vectorstore_hf.similarity_search(query, k=10)  # k is the number of results to return

# # Print the results
# for doc in results:
#     print(f"Content: {doc.page_content}\n")
#     print(f"Metadata: {doc.metadata}\n")
#     print("---")
    


In [77]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [78]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [79]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-28 07:52:30,429 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [80]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [81]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-07-28 07:52:36,820 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [82]:
suggested_categories = list(json.loads(response.choices[0].message.content).values())[0]
suggested_categories

['OpenAI search tool',
 'ChatGPT Voice Mode update',
 'Google Chrome AI Safe Browsing',
 'Google Workspace AI prompts',
 'Mistral Large 2 release',
 'SK Hynix $6.8B chip plant',
 'Real-time neural network training',
 'USC AI wildfire prediction',
 'HP AI accessibility tools',
 'TikTok algorithm issues',
 'ChatGPT accuracy concerns',
 'Google AI weather forecasts',
 'Daniel Loeb AI investments',
 'Nuro autonomous delivery',
 'Anthropic data scraping',
 'OpenAI financial losses',
 'AI in domestic violence risk assessment',
 'Elon Musk X AI training',
 'AI in robotics',
 'SAG-AFTRA AI strike',
 'Amazon Prime Video AI update',
 'AI in real estate',
 'Recursive self-improvement in AI',
 'Apple AI safety commitment',
 'AI-generated deepfakes in politics',
 'FedRAMP modernization',
 'Mark Zuckerberg on AI agents',
 'AI trade risks',
 'AI decoding whale sounds']

In [83]:
cluster_topic_list

['ChatGPT and AI Innovations',
 'Google and AI Innovations',
 'Mistral Large 2 Language Model',
 'Nvidia and Semiconductor Manufacturing Investments']

In [84]:
# human category edit 
my_cats = [
 'AI in drug discovery',
 'Machine learning in structural biology',
 'nference and Takeda AI partnership',
 'Ant-inspired AI for drones',
 "Fei-Fei Li's $1B AI startup",
 "Artificial Agency's $16M funding",
 "CTERA's $80M funding",
 "Anthropic's $100M AI fund",
 "Samsung's AI image generation",
 'EU antitrust probe on AI deals',
 'Salesforce AI service agent',
 "Meta's AI regulatory issues",
 'Nvidia and Mistral AI model',
 'TSMC AI chip demand surge',
 'AI search engines',
 'AI in cybersecurity',
 'AI in Healthcare & Precision Medicine',
 'Autonomous Drones',
 'AI Startup Funding',
 'Claude AI for Data Analysis',
 'AI Startup Funding',
 'AI Smartphones',
 'Big Tech Antitrust Investigations',
 'Large Language Models',
 'AI-Powered Customer Service Agents',
 'AI Ethics and Regulation',
 'AI Art by Microsoft',
 'AI Chip Demand Surge',
 'Text-To-Image Diffusion Models',
 'ChatGPT in AI Dominance'
]

In [85]:
md_str = ""
doc_list = []
docid_list = []
similarity_cutoff = 1.25
for cat in my_cats:
    docstr = f"# {cat} \n\n"
    # Perform a similarity search
    results = vectorstore.similarity_search_with_score(cat, 
                                                       k=10,
                                                      )
    if results:
        # Print the results
        urldict = {}
        for doc, score in results:
            if urldict.get(doc.metadata['url']):
                continue
            urldict[doc.metadata['url']] = 1    
            if score > similarity_cutoff:
                break
            docstr += f"{doc.page_content}\n"
            docid_list.append(doc.metadata['id'])
        doc_list.append(docstr)
        md_str += docstr
        
        
display(Markdown(md_str))
            


2024-07-28 07:52:37,022 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:37,189 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:37,469 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:37,713 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:37,928 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:38,090 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:38,326 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:38,475 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-28 07:52:38,655 - httpx - INFO - HTTP Request: POST https://api.openai.c

# AI in drug discovery 

[SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Policy And Regulation, Politics, Science 

- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits.
- The plan includes exploring AI applications in citizen services and workforce training, developing a monitoring system for AI deployment, and ensuring human oversight remains central in government processes.
- Initial implementations of AI technology are already in progress at various state agencies, with future efforts supported by a dedicated Center of Excellence and an AI Advisory Group for collaboration between government and the private sector.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[Unifying gen X, Y, Z and boomers: The overlooked secret to AI success - Google News](https://news.google.com/articles/CBMiXmh0dHBzOi8vdmVudHVyZWJlYXQuY29tL2FpL3VuaWZ5aW5nLWdlbi14LXktei1hbmQtYm9vbWVycy10aGUtb3Zlcmxvb2tlZC1zZWNyZXQtdG8tYWktc3VjY2Vzcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Business, Gen AI, Opinion, Society, Culture 

- Effective implementation of generative AI (gen AI) relies on collaboration between IT veterans and AI-native talent, leveraging their different skills and experiences.
- Organizations face challenges such as resource scarcity and the need for alignment on objectives when integrating gen AI into existing systems.
- Continuous education for IT professionals in data science can enhance understanding and cooperation with AI specialists, fostering successful gen AI initiatives.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)

Topics: Authors, Writing, Chatgpt, Ethics, Gen AI, Language Models, Llms, Opinion, Society, Culture 

- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility.
- While initially defensive and critical, the author acknowledges that AI tools can assist with mundane tasks, serving as useful aids rather than replacements for human writers.
- Despite the advancements of AI, the author maintains that human creativity and complexity still surpass AI capabilities, particularly in producing nuanced and high-quality written content.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[Predicting a wildfire's next move? USC researchers using AI to forecast fire's likely path - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Climate, Data, Machine Learning, Science 

- USC researchers are developing an AI model that uses real-time satellite data to predict wildfire behavior, including its path, intensity, and growth rate.
- The AI forecasting tool aims to enhance firefighting efforts amidst an increasing wildfire season, with California experiencing a significant rise in wildfire incidents in 2024.
- The technology is designed to support emergency response operations, aiding in decision-making for evacuation and fire containment without replacing human efforts.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


[Here's how artificial intelligence can help you sell your home - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LnR1cm50bzIzLmNvbS91cy1uZXdzL2hvdXNpbmcvaGVyZXMtaG93LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNhbi1oZWxwLXlvdS1zZWxsLXlvdXItaG9tZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI 

- Redfin has launched an AI-powered tool called Redfin Redesign to assist home sellers with design choices such as paint colors and furniture placement, making their homes more appealing to buyers.
- As of June, a significant increase in homes remained unsold for over 30 days, largely due to rising prices and mortgage rates, with 64.7% of homes on the market listed for at least 30 days.
- To improve marketability, experts recommend fair pricing, cosmetic repairs, and small curb-appeal enhancements, with the AI tool providing sellers and renovators a way to visualize potential changes.


# Machine learning in structural biology 

[The Elegant Math of Machine Learning - Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)

Topics: Data, Education, Gen AI, Machine Learning, Science 

- The author recounts their journey of learning machine learning through coding a neural network to predict planetary positions, which sparked their interest in understanding the mathematics of the field.
- Key insights include the transformation of various data types into vectors, allowing algorithms to analyze and recognize patterns in high-dimensional space.
- Certain machine learning algorithms, particularly deep neural networks, are capable of being "universal function approximators," meaning they can theoretically approximate the output for any complex function if given sufficient data and network capacity.


# nference and Takeda AI partnership 

[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


[Artificial intelligence breakthroughs create new brain for advanced robots - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2JlYTlkZjcxLTM3MWMtNDA0NS05Y2I0LTY0YzIyNzg5YmY3YtIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, Machine Learning, Robots, Science, Singularity 

- Advances in artificial intelligence are leading to the development of a new 'brain' for advanced robots.
- Registration will provide access to exclusive content, demonstrating why over a million readers choose to pay for the Financial Times.
- The subscription model indicates a growing interest in in-depth analysis of technological innovations.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


# Ant-inspired AI for drones 

[AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Robots, Science 

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage.


[A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)

Topics: Authors, Writing, Chatgpt, Ethics, Gen AI, Language Models, Llms, Opinion, Society, Culture 

- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility.
- While initially defensive and critical, the author acknowledges that AI tools can assist with mundane tasks, serving as useful aids rather than replacements for human writers.
- Despite the advancements of AI, the author maintains that human creativity and complexity still surpass AI capabilities, particularly in producing nuanced and high-quality written content.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


[PRISE: A Unique Machine Learning Method for Learning Multitask Temporal Action Abstractions Using Natural Language Processing (NLP) - Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Language Models, Machine Learning, Science 

- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics.  
- The method simplifies action representation by converting continuous actions into discrete codes, enhancing the performance of behavior cloning by extracting meaningful action primitives.  
- Empirical studies demonstrate PRISE's effectiveness in improving learning efficiency over established baselines in robotic manipulation tasks by enabling the learning of high-level skills.  


[The secret to better weather forecasts may be a dash of AI - Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS8yMDI0LzA3LzI3L2dvb2dsZV9haV93ZWF0aGVyL9IBPWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS9BTVAvMjAyNC8wNy8yNy9nb29nbGVfYWlfd2VhdGhlci8?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Machine Learning, Science 

- Google and ECMWF developed NeuralGCM, an AI-enhanced climate model that utilizes machine learning to improve the speed and accuracy of weather forecasts by replacing traditional parameterizations with neural networks.
- NeuralGCM demonstrates significant advancements, outperforming existing models by providing quicker forecasts and better predicting small-scale climate processes, while operating on modern hardware like TPUs, making it over 3,500 times faster than traditional models.
- The model's architecture and code have been made publicly available on GitHub, aiming to integrate broader climate factors in future iterations for long-term climate predictions.


[Predicting a wildfire's next move? USC researchers using AI to forecast fire's likely path - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Climate, Data, Machine Learning, Science 

- USC researchers are developing an AI model that uses real-time satellite data to predict wildfire behavior, including its path, intensity, and growth rate.
- The AI forecasting tool aims to enhance firefighting efforts amidst an increasing wildfire season, with California experiencing a significant rise in wildfire incidents in 2024.
- The technology is designed to support emergency response operations, aiding in decision-making for evacuation and fire containment without replacing human efforts.


[SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Policy And Regulation, Politics, Science 

- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits.
- The plan includes exploring AI applications in citizen services and workforce training, developing a monitoring system for AI deployment, and ensuring human oversight remains central in government processes.
- Initial implementations of AI technology are already in progress at various state agencies, with future efforts supported by a dedicated Center of Excellence and an AI Advisory Group for collaboration between government and the private sector.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[Real-time neural network training on chip - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS9yZWFsLXRpbWUtbmV1cmFsLW5ldHdvcmstdHJhaW5pbmctb24tY2hpcC0yMzM2ODY4MtIBYWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL2FtcC90ZWNobm9sb2d5L2FydGljbGUvcmVhbC10aW1lLW5ldXJhbC1uZXR3b3JrLXRyYWluaW5nLW9uLWNoaXAtMjMzNjg2ODI?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Machine Learning, Neural Networks, Science, Semiconductor Chips, Training 

- Researchers at TU Eindhoven developed a neuromorphic chip for on-chip neural network training, eliminating the need for model transfers and significantly reducing energy consumption and enhancing efficiency.
- The chip utilizes electrochemical random-access memory (EC-RAM) components to mimic neuron behavior, enabling real-time learning and adaptation.
- Innovations in AI technology are complemented by other advancements, such as biomimetic compound eyes inspired by the praying mantis for improved machine vision and semiconductors that convert waste heat into electricity.


# Fei-Fei Li's $1B AI startup 

[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses - Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Finance, Funding, Gen AI, OpenAI 

- OpenAI may face bankruptcy within a year, projecting losses of $5 billion due to high operational costs of $7 billion for AI model training and $1.5 billion for staffing.
- Despite generating approximately $3.5 billion in annual revenue, OpenAI's expenses significantly exceed its income, leading to financial strain.
- The company has raised over $11 billion through seven funding rounds but may need additional funding to continue operations while pursuing its goal of artificial general intelligence (AGI).


[OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Economics, Finance, Funding, Gen AI, Llms, OpenAI, Stocks 

- OpenAI may incur losses of up to $5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT.
- The company's expenditures include approximately $4 billion for server capacity from Microsoft and around $3 billion for training AI models with new data.
- The path to profitability is questioned amid increasing competition, raising concerns from investors about OpenAI's unique technology and sustainability in the market.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[Artificial intelligence breakthroughs create new brain for advanced robots - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2JlYTlkZjcxLTM3MWMtNDA0NS05Y2I0LTY0YzIyNzg5YmY3YtIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, Machine Learning, Robots, Science, Singularity 

- Advances in artificial intelligence are leading to the development of a new 'brain' for advanced robots.
- Registration will provide access to exclusive content, demonstrating why over a million readers choose to pay for the Financial Times.
- The subscription model indicates a growing interest in in-depth analysis of technological innovations.


[5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Nvidia, Stocks 

- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from $120 to $140 and Loop Capital from $120 to $175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch.
- Wells Fargo identifies Amazon as the preferred mega-cap stock ahead of its earnings report, predicting an 18% rise in AWS revenue and emphasizing strong investor sentiment compared to competitors like Meta.
- UBS maintains a Sell rating on Tesla, citing concerns about overvaluation and substantial downside risk, as they believe current stock prices reflect inflated expectations around future AI-driven initiatives.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


# Artificial Agency's $16M funding 

[OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses - Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Finance, Funding, Gen AI, OpenAI 

- OpenAI may face bankruptcy within a year, projecting losses of $5 billion due to high operational costs of $7 billion for AI model training and $1.5 billion for staffing.
- Despite generating approximately $3.5 billion in annual revenue, OpenAI's expenses significantly exceed its income, leading to financial strain.
- The company has raised over $11 billion through seven funding rounds but may need additional funding to continue operations while pursuing its goal of artificial general intelligence (AGI).


[OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Economics, Finance, Funding, Gen AI, Llms, OpenAI, Stocks 

- OpenAI may incur losses of up to $5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT.
- The company's expenditures include approximately $4 billion for server capacity from Microsoft and around $3 billion for training AI models with new data.
- The path to profitability is questioned amid increasing competition, raising concerns from investors about OpenAI's unique technology and sustainability in the market.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[SheerLuxe shows us AI is harming DE&I progress - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3LnByd2Vlay5jb20vYXJ0aWNsZS8xODgxNTMwL3NoZWVybHV4ZS1zaG93cy11cy1haS1oYXJtaW5nLWRlLWktcHJvZ3Jlc3PSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Ethics, Gen AI, Opinion, Society, Culture 

- Omnicom PR agencies are experiencing layoffs as reported by outgoing staff members.
- Agency founder Robert Marston has passed away at the age of 86.
- Vivendi is preparing to list Havas on Euronext Amsterdam.


[Artificial intelligence breakthroughs create new brain for advanced robots - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2JlYTlkZjcxLTM3MWMtNDA0NS05Y2I0LTY0YzIyNzg5YmY3YtIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, Machine Learning, Robots, Science, Singularity 

- Advances in artificial intelligence are leading to the development of a new 'brain' for advanced robots.
- Registration will provide access to exclusive content, demonstrating why over a million readers choose to pay for the Financial Times.
- The subscription model indicates a growing interest in in-depth analysis of technological innovations.


[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[AI start-up Anthropic accused of egregious data scraping - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzA3NjExYjc0LTNkNjktNDU3OS05MDg5LWYyZmMyYWY2MWJhYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Startups 

- Subscription options include Standard Digital and Premium Digital access to content with various features like global news, expert analysis, and exclusive journalism.
- Subscribers receive access to mobile apps, curated newsletters, videos, podcasts, gift articles, and investment insights.
- Additional features include alerts for topics, sharing capabilities, and integration tools for enhanced user experience.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


# CTERA's $80M funding 

# Anthropic's $100M AI fund 

[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses - Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Finance, Funding, Gen AI, OpenAI 

- OpenAI may face bankruptcy within a year, projecting losses of $5 billion due to high operational costs of $7 billion for AI model training and $1.5 billion for staffing.
- Despite generating approximately $3.5 billion in annual revenue, OpenAI's expenses significantly exceed its income, leading to financial strain.
- The company has raised over $11 billion through seven funding rounds but may need additional funding to continue operations while pursuing its goal of artificial general intelligence (AGI).


[OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Economics, Finance, Funding, Gen AI, Llms, OpenAI, Stocks 

- OpenAI may incur losses of up to $5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT.
- The company's expenditures include approximately $4 billion for server capacity from Microsoft and around $3 billion for training AI models with new data.
- The path to profitability is questioned amid increasing competition, raising concerns from investors about OpenAI's unique technology and sustainability in the market.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[AI start-up Anthropic accused of egregious data scraping - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzA3NjExYjc0LTNkNjktNDU3OS05MDg5LWYyZmMyYWY2MWJhYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Startups 

- Subscription options include Standard Digital and Premium Digital access to content with various features like global news, expert analysis, and exclusive journalism.
- Subscribers receive access to mobile apps, curated newsletters, videos, podcasts, gift articles, and investment insights.
- Additional features include alerts for topics, sharing capabilities, and integration tools for enhanced user experience.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[Anthropics crawler is ignoring websites anti-AI scraping policies - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Laws, Policy And Regulation 

- Anthropic's ClaudeBot reportedly scraped iFixit's website nearly a million times in 24 hours, violating its Terms of Use that prohibit unauthorized use of content for AI training.
- iFixit's CEO criticized the excessive crawling for draining their DevOps resources and confirmed that they added crawl-delay measures to the robots.txt file, which led to ClaudeBot halting its scraping activities.
- Other companies, including Read the Docs and Freelancer.com, have also reported aggressive scraping by Anthropic, raising concerns about AI crawlers' impact on web resources and the limitations of robots.txt as a control mechanism.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


# Samsung's AI image generation 

[Nvidia Supplier SK Hynix To Spend $6.8B on South Korea Plant To Meet AI Chip Demand - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chip Plant, Deals, Economics, Funding, Gen AI, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK Hynix is investing $6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips.  
- The facility will be the first of four planned plants in the Yongin Semiconductor Cluster, aligned with South Korea's push to enhance its chip market position.  
- Construction of the first plant is expected to begin next year, with completion targeted for May 2027, fostering collaboration with over 50 small tech companies.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[Nvidia supplier SK hynix to invest $6.8B to build new South Korean chip facility - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9uZXdzLzQxMjg4MjItbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWludmVzdC02OGItaW4tbmV3LXNvdXRoLWtvcmVhbi1jaGlwLWZhY2lsaXR50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Funding, Investment, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK hynix plans to invest $6.8B to construct its first chip plant in Yongin, South Korea, with a focus on next-generation HBM and DRAM production, set to begin construction in March 2024 and finish by May 2027.
- The investment will establish a Global AI semiconductor production base, consisting of four state-of-the-art semiconductor fabs and a cooperation complex for over 50 small local companies.
- SK hynix's broader investment strategy includes a total of $74.8B through 2028, with the majority allocated to HBM chips, enhancing its competitive position in the global semiconductor market.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


# EU antitrust probe on AI deals 

[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[AI start-up Anthropic accused of egregious data scraping - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzA3NjExYjc0LTNkNjktNDU3OS05MDg5LWYyZmMyYWY2MWJhYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Startups 

- Subscription options include Standard Digital and Premium Digital access to content with various features like global news, expert analysis, and exclusive journalism.
- Subscribers receive access to mobile apps, curated newsletters, videos, podcasts, gift articles, and investment insights.
- Additional features include alerts for topics, sharing capabilities, and integration tools for enhanced user experience.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.


[Apple signs the White House's commitment to AI safety - Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/)

Topics: Apple, Big Tech, Ethics, Gen AI, Policy And Regulation, Politics, Safety, White House 

- Apple has signed a non-binding commitment to AI safety initiated by the White House.
- The signing has prompted skepticism regarding the effectiveness and sincerity of such voluntary commitments.
- Commenters express frustration and disbelief over the motives behind tech companies' commitments to ethics while prioritizing profits.


[Xs new AI training opt-out setting draws regulatory scrutiny - Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/)

Topics: Big Tech, Data, Ethics, Gen AI, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Training 

- X introduces an AI training opt-out setting, which has drawn scrutiny from regulators and users express skepticism about its effectiveness.
- Concerns are raised that deleting a Twitter account is the only true way to prevent data use, as opting out may not stop X from leveraging historical posts.
- The conversation reflects broader distrust in AI companies to genuinely respect user privacy and data preferences.


[SheerLuxe shows us AI is harming DE&I progress - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3LnByd2Vlay5jb20vYXJ0aWNsZS8xODgxNTMwL3NoZWVybHV4ZS1zaG93cy11cy1haS1oYXJtaW5nLWRlLWktcHJvZ3Jlc3PSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Ethics, Gen AI, Opinion, Society, Culture 

- Omnicom PR agencies are experiencing layoffs as reported by outgoing staff members.
- Agency founder Robert Marston has passed away at the age of 86.
- Vivendi is preparing to list Havas on Euronext Amsterdam.


[Anthropics crawler is ignoring websites anti-AI scraping policies - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Laws, Policy And Regulation 

- Anthropic's ClaudeBot reportedly scraped iFixit's website nearly a million times in 24 hours, violating its Terms of Use that prohibit unauthorized use of content for AI training.
- iFixit's CEO criticized the excessive crawling for draining their DevOps resources and confirmed that they added crawl-delay measures to the robots.txt file, which led to ClaudeBot halting its scraping activities.
- Other companies, including Read the Docs and Freelancer.com, have also reported aggressive scraping by Anthropic, raising concerns about AI crawlers' impact on web resources and the limitations of robots.txt as a control mechanism.


# Salesforce AI service agent 

[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.


[Here's how artificial intelligence can help you sell your home - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LnR1cm50bzIzLmNvbS91cy1uZXdzL2hvdXNpbmcvaGVyZXMtaG93LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNhbi1oZWxwLXlvdS1zZWxsLXlvdXItaG9tZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI 

- Redfin has launched an AI-powered tool called Redfin Redesign to assist home sellers with design choices such as paint colors and furniture placement, making their homes more appealing to buyers.
- As of June, a significant increase in homes remained unsold for over 30 days, largely due to rising prices and mortgage rates, with 64.7% of homes on the market listed for at least 30 days.
- To improve marketability, experts recommend fair pricing, cosmetic repairs, and small curb-appeal enhancements, with the AI tool providing sellers and renovators a way to visualize potential changes.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Policy And Regulation, Politics, Science 

- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits.
- The plan includes exploring AI applications in citizen services and workforce training, developing a monitoring system for AI deployment, and ensuring human oversight remains central in government processes.
- Initial implementations of AI technology are already in progress at various state agencies, with future efforts supported by a dedicated Center of Excellence and an AI Advisory Group for collaboration between government and the private sector.


[Management - How machine learning and robotics are sparking a revolution in property marketing - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5idXNpbmVzcy1yZXBvcnRlci5jby51ay9tYW5hZ2VtZW50L2hvdy1tYWNoaW5lLWxlYXJuaW5nLWFuZC1yb2JvdGljcy1hcmUtc3BhcmtpbmctYS1yZXZvbHV0aW9uLWluLXByb3BlcnR5LW1hcmtldGluZ9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Machine Learning, Robotics 

- Giraffe360, an AI and robotics company, revolutionizes real estate marketing with its innovative Go Cam that simplifies capturing high-quality imaging and 3D visualizations without requiring technical expertise.  
- Their advanced backend software uses machine learning to process data, enabling personalized editing options and significant time and cost savings for real estate professionals.  
- Giraffe360 aims to enhance automation in content creation, ensuring faster delivery of marketing materials while maintaining a focus on convenience, trust, and user control over their data.  


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


[A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)

Topics: Authors, Writing, Chatgpt, Ethics, Gen AI, Language Models, Llms, Opinion, Society, Culture 

- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility.
- While initially defensive and critical, the author acknowledges that AI tools can assist with mundane tasks, serving as useful aids rather than replacements for human writers.
- Despite the advancements of AI, the author maintains that human creativity and complexity still surpass AI capabilities, particularly in producing nuanced and high-quality written content.


# Meta's AI regulatory issues 

[Xs new AI training opt-out setting draws regulatory scrutiny - Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/)

Topics: Big Tech, Data, Ethics, Gen AI, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Training 

- X introduces an AI training opt-out setting, which has drawn scrutiny from regulators and users express skepticism about its effectiveness.
- Concerns are raised that deleting a Twitter account is the only true way to prevent data use, as opting out may not stop X from leveraging historical posts.
- The conversation reflects broader distrust in AI companies to genuinely respect user privacy and data preferences.


[AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Robots, Science 

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage.


[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.


[Anthropics crawler is ignoring websites anti-AI scraping policies - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Laws, Policy And Regulation 

- Anthropic's ClaudeBot reportedly scraped iFixit's website nearly a million times in 24 hours, violating its Terms of Use that prohibit unauthorized use of content for AI training.
- iFixit's CEO criticized the excessive crawling for draining their DevOps resources and confirmed that they added crawl-delay measures to the robots.txt file, which led to ClaudeBot halting its scraping activities.
- Other companies, including Read the Docs and Freelancer.com, have also reported aggressive scraping by Anthropic, raising concerns about AI crawlers' impact on web resources and the limitations of robots.txt as a control mechanism.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[What are the chances that all this Marketing Hype? And AI remains as basic as it is now. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/)

Topics: AI Doom, Bubble, Opinion 

- Current AI, particularly LLMs, has potential use cases but fundamentally lacks the ability to create high-quality music, journalism, or cinema, with outputs primarily summarizing existing knowledge.
- The hype surrounding AI advancements is often viewed as marketing rather than reality, with experts agreeing that the field faces significant challenges and technological plateaus within a few years.
- The financial sustainability of AI companies is in jeopardy as they continue to burn through capital without a clear revenue model, raising questions about how long they can persist before funding runs out.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[Mistrals Large 2 is its answer to Meta and OpenAIs latest models - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Language Models, Llms, Machine Learning, Meta, Nvidia, OpenAI, Products 

- Mistral launched its Large 2 model, boasting performance on par with OpenAI and Meta's latest offerings in code generation, mathematics, and reasoning, despite having fewer parameters (123 billion).
- The model is designed to minimize hallucination issues and responds more discerningly, along with improved multilingual support that includes understanding 11 languages and 80 coding languages.
- Large 2 is not fully open source and requires a paid license for commercial use, but is accessible on platforms like Google Vertex AI and Amazon Bedrock for testing.


[Apple signs the White House's commitment to AI safety - Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/)

Topics: Apple, Big Tech, Ethics, Gen AI, Policy And Regulation, Politics, Safety, White House 

- Apple has signed a non-binding commitment to AI safety initiated by the White House.
- The signing has prompted skepticism regarding the effectiveness and sincerity of such voluntary commitments.
- Commenters express frustration and disbelief over the motives behind tech companies' commitments to ethics while prioritizing profits.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


# Nvidia and Mistral AI model 

[Mistral AI Unveils Mistral Large 2: A Powerful New Language Model - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, outperforming Llama 3.1 and closely following GPT-4 on benchmarks like HumanEval, MultiPL-E, and the MATH benchmark.
- The model features enhanced code generation, mathematics, and multilingual capabilities, achieving an 84.0% accuracy on the Multilingual MMLU benchmark and surpassing its competitors in multilingual performance.
- Mistral Large 2 is designed for efficiency in single-node inference, emphasizing throughput for long-context applications, and aims to set a new standard for performance-to-cost ratio in AI language models.


[Mistral Large 2: The David to Big Techs Goliath(s) - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFydGlmaWNpYWxpbnRlbGxpZ2VuY2UtbmV3cy5jb20vbmV3cy9taXN0cmFsLWxhcmdlLTItZGF2aWQtdG8tYmlnLXRlY2gtZ29saWF0aHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Mistral Large 2, Open Source, Opinion, Products, Review 

- Mistral AI's Mistral Large 2 (ML2) model competes with larger models from OpenAI, Meta, and Anthropic, offering impressive capabilities such as a 128,000 token context window and support for numerous languages, while being significantly smaller in size.
- ML2 demonstrates competitive performance in language understanding tests, scoring 84% on the MMLU benchmark, and is designed for higher efficiency and faster response generation on available hardware compared to its larger counterparts.
- The model is released under the Mistral Research License, allowing non-commercial and research use, but requiring a separate license for business applications, indicating a strategic shift in accessibility compared to previous models.


[Mistrals Large 2 is its answer to Meta and OpenAIs latest models - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Language Models, Llms, Machine Learning, Meta, Nvidia, OpenAI, Products 

- Mistral launched its Large 2 model, boasting performance on par with OpenAI and Meta's latest offerings in code generation, mathematics, and reasoning, despite having fewer parameters (123 billion).
- The model is designed to minimize hallucination issues and responds more discerningly, along with improved multilingual support that includes understanding 11 languages and 80 coding languages.
- Large 2 is not fully open source and requires a paid license for commercial use, but is accessible on platforms like Google Vertex AI and Amazon Bedrock for testing.


[Mistral Large 2 is now available in Amazon Bedrock - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Amazon, Big Tech, Gen AI, Language Models, Llms, Machine Learning, Mistral Large 2, Products 

- Mistral AI's Mistral Large 2 foundation model is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, coding, and more.
- The model supports an increased context window of 128,000 tokens, enabling better processing of lengthy texts and improved performance for coding tasks.
- Mistral Large 2 has been trained on over 80 programming languages and offers native JSON output for structured responses, making it versatile for various application needs.


[The secret to better weather forecasts may be a dash of AI - Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS8yMDI0LzA3LzI3L2dvb2dsZV9haV93ZWF0aGVyL9IBPWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS9BTVAvMjAyNC8wNy8yNy9nb29nbGVfYWlfd2VhdGhlci8?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Machine Learning, Science 

- Google and ECMWF developed NeuralGCM, an AI-enhanced climate model that utilizes machine learning to improve the speed and accuracy of weather forecasts by replacing traditional parameterizations with neural networks.
- NeuralGCM demonstrates significant advancements, outperforming existing models by providing quicker forecasts and better predicting small-scale climate processes, while operating on modern hardware like TPUs, making it over 3,500 times faster than traditional models.
- The model's architecture and code have been made publicly available on GitHub, aiming to integrate broader climate factors in future iterations for long-term climate predictions.


[Faulty Nvidia H100 GPUs and HBM3 memory caused half of failures during LLama 3 training  one failure every three hours for Meta's 16,384 GPU training cluster - Reddit](https://www.reddit.com/r/technews/comments/1edzlsp/faulty_nvidia_h100_gpus_and_hbm3_memory_caused/)

Topics: Big Tech, Data, Gen AI, Language Models, Llms, Machine Learning, Meta, Nvidia, Products, Science, Semiconductor Chips, Testing, Training 

- Faulty Nvidia H100 GPUs and HBM3 memory accounted for 50% of failures during the training of LLaMA 3, impacting performance significantly.
- Meta experienced one failure approximately every three hours within a training cluster consisting of 16,384 GPUs.
- The issues highlight reliability concerns in high-performance computing environments utilized for AI model training.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[MSI unveils fleet of new high-end AMD Ryzen AI 300 'Strix Point' APU-powered laptops - Google News](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy50d2Vha3Rvd24uY29tL25ld3MvOTk1NzgvbXNpLXVudmVpbHMtZmxlZXQtb2YtbmV3LWhpZ2gtZW5kLWFtZC1yeXplbi1haS0zMDAtc3RyaXgtcG9pbnQtYXB1LXBvd2VyZWQtbGFwdG9wcy9pbmRleC5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Amd, Amd Ryzen, Big Tech, Gaming, Laptops, Msi, Products 

- MSI has launched a new line of high-end laptops powered by the AMD Ryzen AI 300 series "Strix Point" APUs, aimed at gamers, content creators, and workstation users.
- The flagship model, TITAN 18 PRO Ryzen Edition 2024, features the powerful Ryzen AI 9 370 APU, an NVIDIA GeForce RTX 4090 Laptop GPU, and an 18-inch OLED display.
- Other models introduced include the Alpha 17, Stealth 16 AI+, Creator 16 AI+, and Summit 16 AI+, showcasing advanced specs such as OLED panels and high-performance thermal solutions.


# TSMC AI chip demand surge 

[Nvidia Supplier SK Hynix To Spend $6.8B on South Korea Plant To Meet AI Chip Demand - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chip Plant, Deals, Economics, Funding, Gen AI, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK Hynix is investing $6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips.  
- The facility will be the first of four planned plants in the Yongin Semiconductor Cluster, aligned with South Korea's push to enhance its chip market position.  
- Construction of the first plant is expected to begin next year, with completion targeted for May 2027, fostering collaboration with over 50 small tech companies.


[Nvidia supplier SK hynix to invest $6.8B to build new South Korean chip facility - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9uZXdzLzQxMjg4MjItbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWludmVzdC02OGItaW4tbmV3LXNvdXRoLWtvcmVhbi1jaGlwLWZhY2lsaXR50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Funding, Investment, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK hynix plans to invest $6.8B to construct its first chip plant in Yongin, South Korea, with a focus on next-generation HBM and DRAM production, set to begin construction in March 2024 and finish by May 2027.
- The investment will establish a Global AI semiconductor production base, consisting of four state-of-the-art semiconductor fabs and a cooperation complex for over 50 small local companies.
- SK hynix's broader investment strategy includes a total of $74.8B through 2028, with the majority allocated to HBM chips, enhancing its competitive position in the global semiconductor market.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[Nvidia supplier SK Hynix to build $6.8 billion chip plant in South Korea - Google News](https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjYvbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWJ1aWxkLTZwb2ludDgtYmlsbGlvbi1jaGlwLXBsYW4taW4tc291dGgta29yZWEuaHRtbNIBc2h0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDI0LzA3LzI2L252aWRpYS1zdXBwbGllci1zay1oeW5peC10by1idWlsZC02cG9pbnQ4LWJpbGxpb24tY2hpcC1wbGFuLWluLXNvdXRoLWtvcmVhLmh0bWw?hl=en-US&gl=US&ceid=US:en)

Topics: Chip Plant, Deals, Economics, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK Hynix plans to invest $6.8 billion to build a new semiconductor manufacturing plant in the Yongin Semiconductor Cluster, South Korea.
- The plant will be the first in the cluster, with construction set to start in March 2024 and completion by May 2027.
- This investment is part of a larger strategy by SK Hynix and the South Korean government to strengthen the country's position in the semiconductor industry, particularly for applications in artificial intelligence.


[5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Nvidia, Stocks 

- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from $120 to $140 and Loop Capital from $120 to $175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch.
- Wells Fargo identifies Amazon as the preferred mega-cap stock ahead of its earnings report, predicting an 18% rise in AWS revenue and emphasizing strong investor sentiment compared to competitors like Meta.
- UBS maintains a Sell rating on Tesla, citing concerns about overvaluation and substantial downside risk, as they believe current stock prices reflect inflated expectations around future AI-driven initiatives.


[MSI unveils fleet of new high-end AMD Ryzen AI 300 'Strix Point' APU-powered laptops - Google News](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy50d2Vha3Rvd24uY29tL25ld3MvOTk1NzgvbXNpLXVudmVpbHMtZmxlZXQtb2YtbmV3LWhpZ2gtZW5kLWFtZC1yeXplbi1haS0zMDAtc3RyaXgtcG9pbnQtYXB1LXBvd2VyZWQtbGFwdG9wcy9pbmRleC5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Amd, Amd Ryzen, Big Tech, Gaming, Laptops, Msi, Products 

- MSI has launched a new line of high-end laptops powered by the AMD Ryzen AI 300 series "Strix Point" APUs, aimed at gamers, content creators, and workstation users.
- The flagship model, TITAN 18 PRO Ryzen Edition 2024, features the powerful Ryzen AI 9 370 APU, an NVIDIA GeForce RTX 4090 Laptop GPU, and an 18-inch OLED display.
- Other models introduced include the Alpha 17, Stealth 16 AI+, Creator 16 AI+, and Summit 16 AI+, showcasing advanced specs such as OLED panels and high-performance thermal solutions.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[MSI Unveils High-End AMD Ryzen AI 300 Strix Powered Laptops For Gamers & Creators: Alpha 17 & Titan 18 Pro Detailed - Google News](https://news.google.com/articles/CBMiZ2h0dHBzOi8vd2NjZnRlY2guY29tL21zaS1oaWdoLWVuZC1hbWQtcnl6ZW4tYWktMzAwLXN0cml4LWxhcHRvcHMtZ2FtZXJzLWNyZWF0b3JzLWFscGhhLTE3LXRpdGFuLTE4LXByby_SAWtodHRwczovL3djY2Z0ZWNoLmNvbS9tc2ktaGlnaC1lbmQtYW1kLXJ5emVuLWFpLTMwMC1zdHJpeC1sYXB0b3BzLWdhbWVycy1jcmVhdG9ycy1hbHBoYS0xNy10aXRhbi0xOC1wcm8vYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Amd, Amd Ryzen, Gaming, Laptops, Msi, Products 

- MSI launched a range of high-end laptops powered by AMD's Ryzen AI 300 "Strix" APUs, including the Titan 18 Pro and Alpha 17, aimed at gamers and creators.
- The Titan 18 Pro features top-tier configurations with RTX 4090 GPUs and an 18" OLED screen, while the Alpha 17 targets high-performance gaming in a 17" format.
- Pre-orders for these laptops will begin soon with availability in August, followed by offerings from other manufacturers like Acer, ASUS, and Lenovo.


[NIO unveils new operating system, 5nm chip, and phone at an AI event in Shanghai - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9uZXdzLzQxMjkyMzktbmlvLXVudmVpbHMtbmV3LW9wZXJhdGluZy1zeXN0ZW0tNW5tLWNoaXAtYW5kLXBob25lLWF0LWFuLWFpLWV2ZW50LWluLXNoYW5naGFp0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, China, Nio, Products, Semiconductor Chips 

- Access to the page has been denied, requiring user verification to confirm they are human.
- Users can report issues using Reference ID 5be3f380-4cd7-11ef-bf61-21a3e22fd362 for assistance.
- Feedback options are provided for users experiencing various problems with the page.


# AI search engines 

[OpenAI is testing web search features for ChatGPT, challenging Google - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Gen AI, Google, Llms, OpenAI, Products, Testing 

- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI.
- Concerns from publishers about the potential negative impact on their businesses are rising, as AI-generated answers might undercut traffic from traditional search results, despite OpenAI's efforts to collaborate with media organizations.
- The new search tool aims to provide sources and direct links to information, while OpenAI signals future improvements that may include advertisements, further competing with Google's established business model.


[How to join the SearchGPT waitlist  heres how to get early access - Google News](https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LnRvbXNndWlkZS5jb20vYWkvaG93LXRvLWFjY2Vzcy1zZWFyY2hncHQtaGVyZXMtaG93LXRvLXNpZ24tdXAtZm9yLXRoZS13YWl0bGlzdNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Products 

- OpenAI's SearchGPT, a prototype search engine utilizing GPT-4 AI, offers real-time internet-based information retrieval and organizes search results more effectively than traditional search engines.
- Users can join the SearchGPT waitlist by having a ChatGPT account and signing up through a designated link; initial testing will involve up to 10,000 participants.
- Results from SearchGPT will include in-line attributions and links for further exploration, with plans to eventually integrate its features into ChatGPT.


[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.


[Anthropics crawler is ignoring websites anti-AI scraping policies - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Laws, Policy And Regulation 

- Anthropic's ClaudeBot reportedly scraped iFixit's website nearly a million times in 24 hours, violating its Terms of Use that prohibit unauthorized use of content for AI training.
- iFixit's CEO criticized the excessive crawling for draining their DevOps resources and confirmed that they added crawl-delay measures to the robots.txt file, which led to ClaudeBot halting its scraping activities.
- Other companies, including Read the Docs and Freelancer.com, have also reported aggressive scraping by Anthropic, raising concerns about AI crawlers' impact on web resources and the limitations of robots.txt as a control mechanism.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


[Here's how artificial intelligence can help you sell your home - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LnR1cm50bzIzLmNvbS91cy1uZXdzL2hvdXNpbmcvaGVyZXMtaG93LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNhbi1oZWxwLXlvdS1zZWxsLXlvdXItaG9tZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI 

- Redfin has launched an AI-powered tool called Redfin Redesign to assist home sellers with design choices such as paint colors and furniture placement, making their homes more appealing to buyers.
- As of June, a significant increase in homes remained unsold for over 30 days, largely due to rising prices and mortgage rates, with 64.7% of homes on the market listed for at least 30 days.
- To improve marketability, experts recommend fair pricing, cosmetic repairs, and small curb-appeal enhancements, with the AI tool providing sellers and renovators a way to visualize potential changes.


[Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning 

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software, offering prizes like golden bomber jackets.
- The competition aimed to develop effective prompts for tasks such as managing customer inquiries, invoicing, and business leads to compete with Microsoft's AI capabilities in Office 365.
- Seven winning prompts were selected, including solutions for responding to Google reviews and organizing receipts, with plans to integrate them into future Workspace updates.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[AI start-up Anthropic accused of egregious data scraping - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzA3NjExYjc0LTNkNjktNDU3OS05MDg5LWYyZmMyYWY2MWJhYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Startups 

- Subscription options include Standard Digital and Premium Digital access to content with various features like global news, expert analysis, and exclusive journalism.
- Subscribers receive access to mobile apps, curated newsletters, videos, podcasts, gift articles, and investment insights.
- Additional features include alerts for topics, sharing capabilities, and integration tools for enhanced user experience.


# AI in cybersecurity 

[SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Policy And Regulation, Politics, Science 

- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits.
- The plan includes exploring AI applications in citizen services and workforce training, developing a monitoring system for AI deployment, and ensuring human oversight remains central in government processes.
- Initial implementations of AI technology are already in progress at various state agencies, with future efforts supported by a dedicated Center of Excellence and an AI Advisory Group for collaboration between government and the private sector.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Robots, Science 

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)

Topics: Authors, Writing, Chatgpt, Ethics, Gen AI, Language Models, Llms, Opinion, Society, Culture 

- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility.
- While initially defensive and critical, the author acknowledges that AI tools can assist with mundane tasks, serving as useful aids rather than replacements for human writers.
- Despite the advancements of AI, the author maintains that human creativity and complexity still surpass AI capabilities, particularly in producing nuanced and high-quality written content.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[Unifying gen X, Y, Z and boomers: The overlooked secret to AI success - Google News](https://news.google.com/articles/CBMiXmh0dHBzOi8vdmVudHVyZWJlYXQuY29tL2FpL3VuaWZ5aW5nLWdlbi14LXktei1hbmQtYm9vbWVycy10aGUtb3Zlcmxvb2tlZC1zZWNyZXQtdG8tYWktc3VjY2Vzcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Business, Gen AI, Opinion, Society, Culture 

- Effective implementation of generative AI (gen AI) relies on collaboration between IT veterans and AI-native talent, leveraging their different skills and experiences.
- Organizations face challenges such as resource scarcity and the need for alignment on objectives when integrating gen AI into existing systems.
- Continuous education for IT professionals in data science can enhance understanding and cooperation with AI specialists, fostering successful gen AI initiatives.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


[Apple signs the White House's commitment to AI safety - Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/)

Topics: Apple, Big Tech, Ethics, Gen AI, Policy And Regulation, Politics, Safety, White House 

- Apple has signed a non-binding commitment to AI safety initiated by the White House.
- The signing has prompted skepticism regarding the effectiveness and sincerity of such voluntary commitments.
- Commenters express frustration and disbelief over the motives behind tech companies' commitments to ethics while prioritizing profits.


[Predicting a wildfire's next move? USC researchers using AI to forecast fire's likely path - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Climate, Data, Machine Learning, Science 

- USC researchers are developing an AI model that uses real-time satellite data to predict wildfire behavior, including its path, intensity, and growth rate.
- The AI forecasting tool aims to enhance firefighting efforts amidst an increasing wildfire season, with California experiencing a significant rise in wildfire incidents in 2024.
- The technology is designed to support emergency response operations, aiding in decision-making for evacuation and fire containment without replacing human efforts.


# AI in Healthcare & Precision Medicine 

[With Digital Twins, The Doctor Will See You Now - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LnF1YW50YW1hZ2F6aW5lLm9yZy93aXRoLWRpZ2l0YWwtdHdpbnMtdGhlLWRvY3Rvci13aWxsLXNlZS15b3Utbm93LTIwMjQwNzI2L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Health, Fitness, Healthcare, Science 

- Amanda Randles is developing "digital twins" to create personalized medical simulations, significantly improving non-invasive diagnosis and treatment of diseases by simulating blood flow in patients' circulatory systems.
- Her system can now predict blood dynamics over 700,000 heartbeats, allowing doctors to visualize blood behavior and make informed decisions on medical interventions like medication and stent placements.
- The integration of machine learning with physics-based models has streamlined the predictive process, enabling real-time interactive simulations in clinical settings while addressing challenges related to data accuracy and bias.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


[SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Policy And Regulation, Politics, Science 

- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits.
- The plan includes exploring AI applications in citizen services and workforce training, developing a monitoring system for AI deployment, and ensuring human oversight remains central in government processes.
- Initial implementations of AI technology are already in progress at various state agencies, with future efforts supported by a dedicated Center of Excellence and an AI Advisory Group for collaboration between government and the private sector.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Copilot, Gen AI, Hp, India, Products 

- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life.
- The new PCs are targeted at the SMB segment, promoting upgrades and offering tailored recovery solutions to encourage the adoption of premium devices in a hybrid work environment.
- Competition in the AI PC market is intensifying, with Qualcomm challenging Intel and AMD, and projections indicate AI PCs could represent 50% of the market within two years as affordability and integration improve.


[Unifying gen X, Y, Z and boomers: The overlooked secret to AI success - Google News](https://news.google.com/articles/CBMiXmh0dHBzOi8vdmVudHVyZWJlYXQuY29tL2FpL3VuaWZ5aW5nLWdlbi14LXktei1hbmQtYm9vbWVycy10aGUtb3Zlcmxvb2tlZC1zZWNyZXQtdG8tYWktc3VjY2Vzcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Business, Gen AI, Opinion, Society, Culture 

- Effective implementation of generative AI (gen AI) relies on collaboration between IT veterans and AI-native talent, leveraging their different skills and experiences.
- Organizations face challenges such as resource scarcity and the need for alignment on objectives when integrating gen AI into existing systems.
- Continuous education for IT professionals in data science can enhance understanding and cooperation with AI specialists, fostering successful gen AI initiatives.


[Spectral characterization of intraoperative renal perfusion using hyperspectral imaging and artificial intelligence | Scientific Reports - Google News](https://news.google.com/articles/CBMiMmh0dHBzOi8vd3d3Lm5hdHVyZS5jb20vYXJ0aWNsZXMvczQxNTk4LTAyNC02ODI4MC0z0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Healthcare, Machine Learning, Science 

- Hyperspectral imaging (HSI) combined with machine learning (ML) shows promising potential for accurately assessing renal perfusion states during surgery, distinguishing between various malperfusion scenarios with a prediction accuracy of 97.8%.
- The study evaluated complete, gradual, and partial kidney malperfusion in a porcine model, revealing significant spectral differences in oxygenation and blood flow indices, thereby demonstrating HSI's superiority over conventional techniques like ultrasonic flowmetry.
- Data from this porcine study aligns well with human patient data, indicating transferability and suggesting that HSI could lead to improvements in intraoperative renal surgery outcomes, although further clinical trials are necessary to confirm its efficacy and patient benefits.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


# Autonomous Drones 

[Autonomous delivery startup Nuro is gearing up for a comeback - Reddit](https://www.reddit.com/r/technology/comments/1edr9jq/autonomous_delivery_startup_nuro_is_gearing_up/)

Topics: Autonomous Vehicles, Products, Robots, Startups, Transportation 

- Nuro, an autonomous delivery startup, is planning a comeback in the transportation sector.
- Discussions around AI and robots suggest they might be perceived as a more significant job threat than immigration.
- The comments reflect a blend of humor and skepticism towards the implications of autonomous delivery technology in everyday situations.


[AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Robots, Science 

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


# AI Startup Funding 

[OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses - Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Finance, Funding, Gen AI, OpenAI 

- OpenAI may face bankruptcy within a year, projecting losses of $5 billion due to high operational costs of $7 billion for AI model training and $1.5 billion for staffing.
- Despite generating approximately $3.5 billion in annual revenue, OpenAI's expenses significantly exceed its income, leading to financial strain.
- The company has raised over $11 billion through seven funding rounds but may need additional funding to continue operations while pursuing its goal of artificial general intelligence (AGI).


[OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Economics, Finance, Funding, Gen AI, Llms, OpenAI, Stocks 

- OpenAI may incur losses of up to $5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT.
- The company's expenditures include approximately $4 billion for server capacity from Microsoft and around $3 billion for training AI models with new data.
- The path to profitability is questioned amid increasing competition, raising concerns from investors about OpenAI's unique technology and sustainability in the market.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


[As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Copilot, Gen AI, Hp, India, Products 

- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life.
- The new PCs are targeted at the SMB segment, promoting upgrades and offering tailored recovery solutions to encourage the adoption of premium devices in a hybrid work environment.
- Competition in the AI PC market is intensifying, with Qualcomm challenging Intel and AMD, and projections indicate AI PCs could represent 50% of the market within two years as affordability and integration improve.


[5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Nvidia, Stocks 

- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from $120 to $140 and Loop Capital from $120 to $175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch.
- Wells Fargo identifies Amazon as the preferred mega-cap stock ahead of its earnings report, predicting an 18% rise in AWS revenue and emphasizing strong investor sentiment compared to competitors like Meta.
- UBS maintains a Sell rating on Tesla, citing concerns about overvaluation and substantial downside risk, as they believe current stock prices reflect inflated expectations around future AI-driven initiatives.


[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.


[Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning 

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software, offering prizes like golden bomber jackets.
- The competition aimed to develop effective prompts for tasks such as managing customer inquiries, invoicing, and business leads to compete with Microsoft's AI capabilities in Office 365.
- Seven winning prompts were selected, including solutions for responding to Google reviews and organizing receipts, with plans to integrate them into future Workspace updates.


# Claude AI for Data Analysis 

[The secret to better weather forecasts may be a dash of AI - Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS8yMDI0LzA3LzI3L2dvb2dsZV9haV93ZWF0aGVyL9IBPWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS9BTVAvMjAyNC8wNy8yNy9nb29nbGVfYWlfd2VhdGhlci8?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Machine Learning, Science 

- Google and ECMWF developed NeuralGCM, an AI-enhanced climate model that utilizes machine learning to improve the speed and accuracy of weather forecasts by replacing traditional parameterizations with neural networks.
- NeuralGCM demonstrates significant advancements, outperforming existing models by providing quicker forecasts and better predicting small-scale climate processes, while operating on modern hardware like TPUs, making it over 3,500 times faster than traditional models.
- The model's architecture and code have been made publicly available on GitHub, aiming to integrate broader climate factors in future iterations for long-term climate predictions.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[The Elegant Math of Machine Learning - Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)

Topics: Data, Education, Gen AI, Machine Learning, Science 

- The author recounts their journey of learning machine learning through coding a neural network to predict planetary positions, which sparked their interest in understanding the mathematics of the field.
- Key insights include the transformation of various data types into vectors, allowing algorithms to analyze and recognize patterns in high-dimensional space.
- Certain machine learning algorithms, particularly deep neural networks, are capable of being "universal function approximators," meaning they can theoretically approximate the output for any complex function if given sufficient data and network capacity.


[Mistral Large 2: The David to Big Techs Goliath(s) - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFydGlmaWNpYWxpbnRlbGxpZ2VuY2UtbmV3cy5jb20vbmV3cy9taXN0cmFsLWxhcmdlLTItZGF2aWQtdG8tYmlnLXRlY2gtZ29saWF0aHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Mistral Large 2, Open Source, Opinion, Products, Review 

- Mistral AI's Mistral Large 2 (ML2) model competes with larger models from OpenAI, Meta, and Anthropic, offering impressive capabilities such as a 128,000 token context window and support for numerous languages, while being significantly smaller in size.
- ML2 demonstrates competitive performance in language understanding tests, scoring 84% on the MMLU benchmark, and is designed for higher efficiency and faster response generation on available hardware compared to its larger counterparts.
- The model is released under the Mistral Research License, allowing non-commercial and research use, but requiring a separate license for business applications, indicating a strategic shift in accessibility compared to previous models.


[Mistral AI Unveils Mistral Large 2: A Powerful New Language Model - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, outperforming Llama 3.1 and closely following GPT-4 on benchmarks like HumanEval, MultiPL-E, and the MATH benchmark.
- The model features enhanced code generation, mathematics, and multilingual capabilities, achieving an 84.0% accuracy on the Multilingual MMLU benchmark and surpassing its competitors in multilingual performance.
- Mistral Large 2 is designed for efficiency in single-node inference, emphasizing throughput for long-context applications, and aims to set a new standard for performance-to-cost ratio in AI language models.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[Mistrals Large 2 is its answer to Meta and OpenAIs latest models - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Language Models, Llms, Machine Learning, Meta, Nvidia, OpenAI, Products 

- Mistral launched its Large 2 model, boasting performance on par with OpenAI and Meta's latest offerings in code generation, mathematics, and reasoning, despite having fewer parameters (123 billion).
- The model is designed to minimize hallucination issues and responds more discerningly, along with improved multilingual support that includes understanding 11 languages and 80 coding languages.
- Large 2 is not fully open source and requires a paid license for commercial use, but is accessible on platforms like Google Vertex AI and Amazon Bedrock for testing.


[PRISE: A Unique Machine Learning Method for Learning Multitask Temporal Action Abstractions Using Natural Language Processing (NLP) - Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Language Models, Machine Learning, Science 

- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics.  
- The method simplifies action representation by converting continuous actions into discrete codes, enhancing the performance of behavior cloning by extracting meaningful action primitives.  
- Empirical studies demonstrate PRISE's effectiveness in improving learning efficiency over established baselines in robotic manipulation tasks by enabling the learning of high-level skills.  


[Mistral Large 2 is now available in Amazon Bedrock - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Amazon, Big Tech, Gen AI, Language Models, Llms, Machine Learning, Mistral Large 2, Products 

- Mistral AI's Mistral Large 2 foundation model is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, coding, and more.
- The model supports an increased context window of 128,000 tokens, enabling better processing of lengthy texts and improved performance for coding tasks.
- Mistral Large 2 has been trained on over 80 programming languages and offers native JSON output for structured responses, making it versatile for various application needs.


[What are the chances that all this Marketing Hype? And AI remains as basic as it is now. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/)

Topics: AI Doom, Bubble, Opinion 

- Current AI, particularly LLMs, has potential use cases but fundamentally lacks the ability to create high-quality music, journalism, or cinema, with outputs primarily summarizing existing knowledge.
- The hype surrounding AI advancements is often viewed as marketing rather than reality, with experts agreeing that the field faces significant challenges and technological plateaus within a few years.
- The financial sustainability of AI companies is in jeopardy as they continue to burn through capital without a clear revenue model, raising questions about how long they can persist before funding runs out.


# AI Startup Funding 

[OpenAI could be on the brink of bankruptcy in under 12 months, with projections of $5 billion in losses - Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Finance, Funding, Gen AI, OpenAI 

- OpenAI may face bankruptcy within a year, projecting losses of $5 billion due to high operational costs of $7 billion for AI model training and $1.5 billion for staffing.
- Despite generating approximately $3.5 billion in annual revenue, OpenAI's expenses significantly exceed its income, leading to financial strain.
- The company has raised over $11 billion through seven funding rounds but may need additional funding to continue operations while pursuing its goal of artificial general intelligence (AGI).


[OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Economics, Finance, Funding, Gen AI, Llms, OpenAI, Stocks 

- OpenAI may incur losses of up to $5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT.
- The company's expenditures include approximately $4 billion for server capacity from Microsoft and around $3 billion for training AI models with new data.
- The path to profitability is questioned amid increasing competition, raising concerns from investors about OpenAI's unique technology and sustainability in the market.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


[As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Copilot, Gen AI, Hp, India, Products 

- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life.
- The new PCs are targeted at the SMB segment, promoting upgrades and offering tailored recovery solutions to encourage the adoption of premium devices in a hybrid work environment.
- Competition in the AI PC market is intensifying, with Qualcomm challenging Intel and AMD, and projections indicate AI PCs could represent 50% of the market within two years as affordability and integration improve.


[5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Nvidia, Stocks 

- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from $120 to $140 and Loop Capital from $120 to $175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch.
- Wells Fargo identifies Amazon as the preferred mega-cap stock ahead of its earnings report, predicting an 18% rise in AWS revenue and emphasizing strong investor sentiment compared to competitors like Meta.
- UBS maintains a Sell rating on Tesla, citing concerns about overvaluation and substantial downside risk, as they believe current stock prices reflect inflated expectations around future AI-driven initiatives.


[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.


[Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning 

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software, offering prizes like golden bomber jackets.
- The competition aimed to develop effective prompts for tasks such as managing customer inquiries, invoicing, and business leads to compete with Microsoft's AI capabilities in Office 365.
- Seven winning prompts were selected, including solutions for responding to Google reviews and organizing receipts, with plans to integrate them into future Workspace updates.


# AI Smartphones 

[My new iPhone symbolises stagnation, not innovation  and a similar fate awaits AI - Google News](https://news.google.com/articles/CBMijQFodHRwczovL3d3dy50aGVndWFyZGlhbi5jb20vY29tbWVudGlzZnJlZS9hcnRpY2xlLzIwMjQvanVsLzI3L215LW5ldy1pcGhvbmUtc3ltYm9saXNlcy1zdGFnbmF0aW9uLW5vdC1pbm5vdmF0aW9uLWFuZC1hLXNpbWlsYXItZmF0ZS1hd2FpdHMtYWnSAY0BaHR0cHM6Ly9hbXAudGhlZ3VhcmRpYW4uY29tL2NvbW1lbnRpc2ZyZWUvYXJ0aWNsZS8yMDI0L2p1bC8yNy9teS1uZXctaXBob25lLXN5bWJvbGlzZXMtc3RhZ25hdGlvbi1ub3QtaW5ub3ZhdGlvbi1hbmQtYS1zaW1pbGFyLWZhdGUtYXdhaXRzLWFp?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Apple, Big Tech, Opinion, Society, Culture 

- The author argues that recent advancements in smartphones, including the new iPhone, represent incremental improvements rather than true innovation, mirroring a broader pattern in consumer electronics where technological progress eventually stagnates.
- The same stagnation may apply to AI technology, which has seen rapid development but could be on the verge of plateauing, similar to the smartphone industry after initial breakthroughs.
- There are signs AI is becoming commoditized, with diminishing user engagement and few companies moving from pilot projects to large-scale deployment, suggesting that interest in AI may wane like that of new smartphone models.


[As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Copilot, Gen AI, Hp, India, Products 

- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life.
- The new PCs are targeted at the SMB segment, promoting upgrades and offering tailored recovery solutions to encourage the adoption of premium devices in a hybrid work environment.
- Competition in the AI PC market is intensifying, with Qualcomm challenging Intel and AMD, and projections indicate AI PCs could represent 50% of the market within two years as affordability and integration improve.


[iPhone 16 coming with two new buttons, AI chatbot and surprise color  latest predictions for Apples top... - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LnRoZS1zdW4uY29tL3RlY2gvMTIwMDExMTgvaXBob25lLTE2LXByZWRpY3Rpb25zLXdoYXQtdG8tZXhwZWN0L9IBT2h0dHBzOi8vd3d3LnRoZS1zdW4uY29tL3RlY2gvMTIwMDExMTgvaXBob25lLTE2LXByZWRpY3Rpb25zLXdoYXQtdG8tZXhwZWN0L2FtcC8?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Chatbot, Products 

- The iPhone 16 is expected to launch in September with new features, including Apple’s iOS 18, an AI chatbot integrated with Siri, and customizable Home Screen app icons. 
- Key hardware updates may include an Action Button replacing the mute switch, vertical camera lenses for improved 3D video capabilities, and potentially a new capture button for photos.
- The phone will likely come with the new A18 processor, improved performance and battery life, and a variety of color options, including a rumored new Rose color for the Pro models.


[Nio IN 2024: Autonomous driving chip, SkyOS, 2nd-gen Nio Phone, and more - Google News](https://news.google.com/articles/CBMiQmh0dHBzOi8vY25ldnBvc3QuY29tLzIwMjQvMDcvMjcvbmlvLWluLTIwMjQtY2hpcC1za3lvcy1waG9uZS1tb3JlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Nio, Products, Semiconductor Chips, Transportation 

- Nio unveiled significant advancements, including the Shenji NX9031 autonomous driving chip, SkyOS vehicle operating system, and the second-generation Nio Phone, at its Tech Day event.
- The Shenji NX9031 chip, the first of its kind to use a 5 nm process, boasts performance equivalent to four industry flagship chips and will feature in the upcoming ET9 sedan.
- SkyOS offers low latency communication and enhanced data security, while the second-generation Nio Phone introduces improved interaction features and an AI-powered virtual assistant.


[NIO unveils new operating system, 5nm chip, and phone at an AI event in Shanghai - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9uZXdzLzQxMjkyMzktbmlvLXVudmVpbHMtbmV3LW9wZXJhdGluZy1zeXN0ZW0tNW5tLWNoaXAtYW5kLXBob25lLWF0LWFuLWFpLWV2ZW50LWluLXNoYW5naGFp0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, China, Nio, Products, Semiconductor Chips 

- Access to the page has been denied, requiring user verification to confirm they are human.
- Users can report issues using Reference ID 5be3f380-4cd7-11ef-bf61-21a3e22fd362 for assistance.
- Feedback options are provided for users experiencing various problems with the page.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[New AI, controls make robots easier to program and deploy - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Manufacturing, Products, Robots, Science 

- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao.
- Recent industry news covers the operational costs and applications of Agility Robotics' Digit humanoid, which is designed to enhance automation in facilities like Spanx.
- Skild AI has raised $300 million to create a general-purpose foundation model for robotics, while Helm.ai launched a generative AI model for realistic video sequences aimed at enhancing autonomous driving capabilities.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


[Artificial intelligence breakthroughs create new brain for advanced robots - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50L2JlYTlkZjcxLTM3MWMtNDA0NS05Y2I0LTY0YzIyNzg5YmY3YtIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, Machine Learning, Robots, Science, Singularity 

- Advances in artificial intelligence are leading to the development of a new 'brain' for advanced robots.
- Registration will provide access to exclusive content, demonstrating why over a million readers choose to pay for the Financial Times.
- The subscription model indicates a growing interest in in-depth analysis of technological innovations.


# Big Tech Antitrust Investigations 

[AI start-up Anthropic accused of egregious data scraping - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzA3NjExYjc0LTNkNjktNDU3OS05MDg5LWYyZmMyYWY2MWJhYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Startups 

- Subscription options include Standard Digital and Premium Digital access to content with various features like global news, expert analysis, and exclusive journalism.
- Subscribers receive access to mobile apps, curated newsletters, videos, podcasts, gift articles, and investment insights.
- Additional features include alerts for topics, sharing capabilities, and integration tools for enhanced user experience.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[Xs new AI training opt-out setting draws regulatory scrutiny - Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/)

Topics: Big Tech, Data, Ethics, Gen AI, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Training 

- X introduces an AI training opt-out setting, which has drawn scrutiny from regulators and users express skepticism about its effectiveness.
- Concerns are raised that deleting a Twitter account is the only true way to prevent data use, as opting out may not stop X from leveraging historical posts.
- The conversation reflects broader distrust in AI companies to genuinely respect user privacy and data preferences.


[TikToks algorithm is highly sensitive  and could send you down a hate-filled rabbit hole before you know it - Reddit](https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/)

Topics: AI Doom, Big Tech, Data, Disinformation, Ethics, Machine Learning, Opinion, Recommendation Systems, Society, Culture 

- TikTok and other social media algorithms can quickly lead users down hate-filled content rabbit holes, impacting mental well-being and promoting extreme views.
- Users often encounter unexpected political content from their viewing habits, regardless of original interests, reinforcing biases and directing them towards extremist material.
- The algorithms require active management, such as blocking unwanted channels or marking content as "not interested," but effectiveness varies, leaving many frustrated with their experiences.


[Anthropics crawler is ignoring websites anti-AI scraping policies - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Laws, Policy And Regulation 

- Anthropic's ClaudeBot reportedly scraped iFixit's website nearly a million times in 24 hours, violating its Terms of Use that prohibit unauthorized use of content for AI training.
- iFixit's CEO criticized the excessive crawling for draining their DevOps resources and confirmed that they added crawl-delay measures to the robots.txt file, which led to ClaudeBot halting its scraping activities.
- Other companies, including Read the Docs and Freelancer.com, have also reported aggressive scraping by Anthropic, raising concerns about AI crawlers' impact on web resources and the limitations of robots.txt as a control mechanism.


[OpenAI is testing web search features for ChatGPT, challenging Google - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Gen AI, Google, Llms, OpenAI, Products, Testing 

- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI.
- Concerns from publishers about the potential negative impact on their businesses are rising, as AI-generated answers might undercut traffic from traditional search results, despite OpenAI's efforts to collaborate with media organizations.
- The new search tool aims to provide sources and direct links to information, while OpenAI signals future improvements that may include advertisements, further competing with Google's established business model.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.


# Large Language Models 

[Mistral AI Unveils Mistral Large 2: A Powerful New Language Model - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, outperforming Llama 3.1 and closely following GPT-4 on benchmarks like HumanEval, MultiPL-E, and the MATH benchmark.
- The model features enhanced code generation, mathematics, and multilingual capabilities, achieving an 84.0% accuracy on the Multilingual MMLU benchmark and surpassing its competitors in multilingual performance.
- Mistral Large 2 is designed for efficiency in single-node inference, emphasizing throughput for long-context applications, and aims to set a new standard for performance-to-cost ratio in AI language models.


[Mistral Large 2: The David to Big Techs Goliath(s) - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFydGlmaWNpYWxpbnRlbGxpZ2VuY2UtbmV3cy5jb20vbmV3cy9taXN0cmFsLWxhcmdlLTItZGF2aWQtdG8tYmlnLXRlY2gtZ29saWF0aHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Mistral Large 2, Open Source, Opinion, Products, Review 

- Mistral AI's Mistral Large 2 (ML2) model competes with larger models from OpenAI, Meta, and Anthropic, offering impressive capabilities such as a 128,000 token context window and support for numerous languages, while being significantly smaller in size.
- ML2 demonstrates competitive performance in language understanding tests, scoring 84% on the MMLU benchmark, and is designed for higher efficiency and faster response generation on available hardware compared to its larger counterparts.
- The model is released under the Mistral Research License, allowing non-commercial and research use, but requiring a separate license for business applications, indicating a strategic shift in accessibility compared to previous models.


[Mistrals Large 2 is its answer to Meta and OpenAIs latest models - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Language Models, Llms, Machine Learning, Meta, Nvidia, OpenAI, Products 

- Mistral launched its Large 2 model, boasting performance on par with OpenAI and Meta's latest offerings in code generation, mathematics, and reasoning, despite having fewer parameters (123 billion).
- The model is designed to minimize hallucination issues and responds more discerningly, along with improved multilingual support that includes understanding 11 languages and 80 coding languages.
- Large 2 is not fully open source and requires a paid license for commercial use, but is accessible on platforms like Google Vertex AI and Amazon Bedrock for testing.


[Mistral Large 2 is now available in Amazon Bedrock - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Amazon, Big Tech, Gen AI, Language Models, Llms, Machine Learning, Mistral Large 2, Products 

- Mistral AI's Mistral Large 2 foundation model is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, coding, and more.
- The model supports an increased context window of 128,000 tokens, enabling better processing of lengthy texts and improved performance for coding tasks.
- Mistral Large 2 has been trained on over 80 programming languages and offers native JSON output for structured responses, making it versatile for various application needs.


# AI-Powered Customer Service Agents 

[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.


[A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)

Topics: Authors, Writing, Chatgpt, Ethics, Gen AI, Language Models, Llms, Opinion, Society, Culture 

- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility.
- While initially defensive and critical, the author acknowledges that AI tools can assist with mundane tasks, serving as useful aids rather than replacements for human writers.
- Despite the advancements of AI, the author maintains that human creativity and complexity still surpass AI capabilities, particularly in producing nuanced and high-quality written content.


[Google Exec Shares Latest AI Tools for SMBs - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Products 

- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products.
- The new visual brand profiles enable merchants to consolidate information about their products and brand narratives, which can drive consumer engagement and improve purchase likelihood.
- Small business owners are adopting AI tools to boost digital visibility and efficiency, with 80% reporting that these tools positively impact their operations.


[As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Copilot, Gen AI, Hp, India, Products 

- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life.
- The new PCs are targeted at the SMB segment, promoting upgrades and offering tailored recovery solutions to encourage the adoption of premium devices in a hybrid work environment.
- Competition in the AI PC market is intensifying, with Qualcomm challenging Intel and AMD, and projections indicate AI PCs could represent 50% of the market within two years as affordability and integration improve.


[Here's how artificial intelligence can help you sell your home - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LnR1cm50bzIzLmNvbS91cy1uZXdzL2hvdXNpbmcvaGVyZXMtaG93LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNhbi1oZWxwLXlvdS1zZWxsLXlvdXItaG9tZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI 

- Redfin has launched an AI-powered tool called Redfin Redesign to assist home sellers with design choices such as paint colors and furniture placement, making their homes more appealing to buyers.
- As of June, a significant increase in homes remained unsold for over 30 days, largely due to rising prices and mortgage rates, with 64.7% of homes on the market listed for at least 30 days.
- To improve marketability, experts recommend fair pricing, cosmetic repairs, and small curb-appeal enhancements, with the AI tool providing sellers and renovators a way to visualize potential changes.


[SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Policy And Regulation, Politics, Science 

- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits.
- The plan includes exploring AI applications in citizen services and workforce training, developing a monitoring system for AI deployment, and ensuring human oversight remains central in government processes.
- Initial implementations of AI technology are already in progress at various state agencies, with future efforts supported by a dedicated Center of Excellence and an AI Advisory Group for collaboration between government and the private sector.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.


["AI in Robotics: Revolutionizing the Future of Automation" - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Robotics, Robots, Science 

- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from $12.3 billion in 2023 to $146.8 billion by 2033.
- Key sectors such as manufacturing, healthcare, logistics, and agriculture are experiencing transformative benefits from AI-driven robots, which streamline operations, improve precision, and reduce human error.
- Despite promising advancements, challenges remain in ensuring safety, interpreting data, and addressing workforce implications and ethical concerns as automation continues to evolve.


# AI Ethics and Regulation 

[Xs new AI training opt-out setting draws regulatory scrutiny - Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/)

Topics: Big Tech, Data, Ethics, Gen AI, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Training 

- X introduces an AI training opt-out setting, which has drawn scrutiny from regulators and users express skepticism about its effectiveness.
- Concerns are raised that deleting a Twitter account is the only true way to prevent data use, as opting out may not stop X from leveraging historical posts.
- The conversation reflects broader distrust in AI companies to genuinely respect user privacy and data preferences.


[AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Robots, Science 

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage.


[Apple signs the White House's commitment to AI safety - Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/)

Topics: Apple, Big Tech, Ethics, Gen AI, Policy And Regulation, Politics, Safety, White House 

- Apple has signed a non-binding commitment to AI safety initiated by the White House.
- The signing has prompted skepticism regarding the effectiveness and sincerity of such voluntary commitments.
- Commenters express frustration and disbelief over the motives behind tech companies' commitments to ethics while prioritizing profits.


[AI start-up Anthropic accused of egregious data scraping - Google News](https://news.google.com/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzA3NjExYjc0LTNkNjktNDU3OS05MDg5LWYyZmMyYWY2MWJhYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Policy And Regulation, Privacy, Startups 

- Subscription options include Standard Digital and Premium Digital access to content with various features like global news, expert analysis, and exclusive journalism.
- Subscribers receive access to mobile apps, curated newsletters, videos, podcasts, gift articles, and investment insights.
- Additional features include alerts for topics, sharing capabilities, and integration tools for enhanced user experience.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


[Websites accuse AI startup Anthropic of bypassing their anti-scraping rules and protocol - Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Cybersecurity, Data, Ethics, Gen AI, Intellectual Property, Laws, Llms, Machine Learning, Policy And Regulation, Privacy 

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission.
- Freelancer reported that Anthropic's crawler generated 3.5 million visits in four hours, and iFixit experienced over a million hits in 24 hours, disrupting their services and resources.
- Anthropic claims to respect robots.txt rules and is investigating the incidents, while discussions about potential licensing agreements for content usage are being considered by iFixit.


[Anthropics crawler is ignoring websites anti-AI scraping policies - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Data, Data Scraping, Ethics, Gen AI, Laws, Policy And Regulation 

- Anthropic's ClaudeBot reportedly scraped iFixit's website nearly a million times in 24 hours, violating its Terms of Use that prohibit unauthorized use of content for AI training.
- iFixit's CEO criticized the excessive crawling for draining their DevOps resources and confirmed that they added crawl-delay measures to the robots.txt file, which led to ClaudeBot halting its scraping activities.
- Other companies, including Read the Docs and Freelancer.com, have also reported aggressive scraping by Anthropic, raising concerns about AI crawlers' impact on web resources and the limitations of robots.txt as a control mechanism.


[SheerLuxe shows us AI is harming DE&I progress - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3LnByd2Vlay5jb20vYXJ0aWNsZS8xODgxNTMwL3NoZWVybHV4ZS1zaG93cy11cy1haS1oYXJtaW5nLWRlLWktcHJvZ3Jlc3PSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Ethics, Gen AI, Opinion, Society, Culture 

- Omnicom PR agencies are experiencing layoffs as reported by outgoing staff members.
- Agency founder Robert Marston has passed away at the age of 86.
- Vivendi is preparing to list Havas on Euronext Amsterdam.


[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


# AI Art by Microsoft 

[A Human Writers Thoughts on AI Writing Bots - Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)

Topics: Authors, Writing, Chatgpt, Ethics, Gen AI, Language Models, Llms, Opinion, Society, Culture 

- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility.
- While initially defensive and critical, the author acknowledges that AI tools can assist with mundane tasks, serving as useful aids rather than replacements for human writers.
- Despite the advancements of AI, the author maintains that human creativity and complexity still surpass AI capabilities, particularly in producing nuanced and high-quality written content.


[Forget Nvidia: Billionaire Daniel Loeb Owns $2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Stocks 

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon (11.8%), Microsoft (9.5%), and Meta Platforms (7.7%) as part of his $2.2 billion investment focus in AI.
- Loeb believes the rapid adoption of AI will accelerate cloud usage, viewing companies like Amazon and Microsoft as primary beneficiaries due to their vast cloud infrastructures.
- Meta's advancements in AI, including the development of its Large Language Model, position it as a strong candidate in the AI sector, supported by its extensive user data and recovery in digital advertising.


[What are the chances that all this Marketing Hype? And AI remains as basic as it is now. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/)

Topics: AI Doom, Bubble, Opinion 

- Current AI, particularly LLMs, has potential use cases but fundamentally lacks the ability to create high-quality music, journalism, or cinema, with outputs primarily summarizing existing knowledge.
- The hype surrounding AI advancements is often viewed as marketing rather than reality, with experts agreeing that the field faces significant challenges and technological plateaus within a few years.
- The financial sustainability of AI companies is in jeopardy as they continue to burn through capital without a clear revenue model, raising questions about how long they can persist before funding runs out.


[HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Hp, Opinion 

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities.
- The HP OmniBook Ultra 14 was revealed, boasting advanced AI capabilities that position it as a potential competitor to existing high-performance models, though specific benchmarks were not disclosed.
- Despite the advancements in AI, the text warns of ecological concerns and other issues stemming from the technology while advocating for its use to address real-world challenges, particularly through accessibility initiatives.


[iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Anthropic, Big Tech, Gen AI, Llms, Machine Learning, Opinion 

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training.
- Anthropic's AI, Claude, acknowledged the violation of iFixit's terms of service, advising against using the site's content for training purposes.
- The situation highlights ongoing concerns over AI companies scraping website content without permission, potentially ignoring established terms to access data.


# AI Chip Demand Surge 

[Nvidia Supplier SK Hynix To Spend $6.8B on South Korea Plant To Meet AI Chip Demand - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chip Plant, Deals, Economics, Funding, Gen AI, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK Hynix is investing $6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips.  
- The facility will be the first of four planned plants in the Yongin Semiconductor Cluster, aligned with South Korea's push to enhance its chip market position.  
- Construction of the first plant is expected to begin next year, with completion targeted for May 2027, fostering collaboration with over 50 small tech companies.


[Nvidia Stock Could Surge (Another) 259% by 2030. This Artificial Intelligence (AI) Stock Could Soar Even Higher, According to 1 Wall Street Analyst. - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns.
- The market for AI is rapidly expanding, with a significant shift anticipated from training AI systems to deploying them for tasks, increasing competition and opportunities in the inference market.
- AMD offers a more cost-effective alternative to Nvidia, potentially allowing it to capture market share as AI adoption grows, supported by its lower-priced processors compared to Nvidia's offerings.


[Nvidia supplier SK hynix to invest $6.8B to build new South Korean chip facility - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9uZXdzLzQxMjg4MjItbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWludmVzdC02OGItaW4tbmV3LXNvdXRoLWtvcmVhbi1jaGlwLWZhY2lsaXR50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Funding, Investment, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK hynix plans to invest $6.8B to construct its first chip plant in Yongin, South Korea, with a focus on next-generation HBM and DRAM production, set to begin construction in March 2024 and finish by May 2027.
- The investment will establish a Global AI semiconductor production base, consisting of four state-of-the-art semiconductor fabs and a cooperation complex for over 50 small local companies.
- SK hynix's broader investment strategy includes a total of $74.8B through 2028, with the majority allocated to HBM chips, enhancing its competitive position in the global semiconductor market.


[Nvidia supplier SK Hynix to build $6.8 billion chip plant in South Korea - Google News](https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjYvbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWJ1aWxkLTZwb2ludDgtYmlsbGlvbi1jaGlwLXBsYW4taW4tc291dGgta29yZWEuaHRtbNIBc2h0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDI0LzA3LzI2L252aWRpYS1zdXBwbGllci1zay1oeW5peC10by1idWlsZC02cG9pbnQ4LWJpbGxpb24tY2hpcC1wbGFuLWluLXNvdXRoLWtvcmVhLmh0bWw?hl=en-US&gl=US&ceid=US:en)

Topics: Chip Plant, Deals, Economics, Korea, Manufacturing, Nvidia, Semiconductor Chips, Sk Hynix 

- SK Hynix plans to invest $6.8 billion to build a new semiconductor manufacturing plant in the Yongin Semiconductor Cluster, South Korea.
- The plant will be the first in the cluster, with construction set to start in March 2024 and completion by May 2027.
- This investment is part of a larger strategy by SK Hynix and the South Korean government to strengthen the country's position in the semiconductor industry, particularly for applications in artificial intelligence.


[Real-time neural network training on chip - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS9yZWFsLXRpbWUtbmV1cmFsLW5ldHdvcmstdHJhaW5pbmctb24tY2hpcC0yMzM2ODY4MtIBYWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL2FtcC90ZWNobm9sb2d5L2FydGljbGUvcmVhbC10aW1lLW5ldXJhbC1uZXR3b3JrLXRyYWluaW5nLW9uLWNoaXAtMjMzNjg2ODI?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Machine Learning, Neural Networks, Science, Semiconductor Chips, Training 

- Researchers at TU Eindhoven developed a neuromorphic chip for on-chip neural network training, eliminating the need for model transfers and significantly reducing energy consumption and enhancing efficiency.
- The chip utilizes electrochemical random-access memory (EC-RAM) components to mimic neuron behavior, enabling real-time learning and adaptation.
- Innovations in AI technology are complemented by other advancements, such as biomimetic compound eyes inspired by the praying mantis for improved machine vision and semiconductors that convert waste heat into electricity.


[5 big analyst AI moves: PT hikes for Nvidia, Apple; Tesla downside risk flagged By Investing.com - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Nvidia, Stocks 

- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from $120 to $140 and Loop Capital from $120 to $175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch.
- Wells Fargo identifies Amazon as the preferred mega-cap stock ahead of its earnings report, predicting an 18% rise in AWS revenue and emphasizing strong investor sentiment compared to competitors like Meta.
- UBS maintains a Sell rating on Tesla, citing concerns about overvaluation and substantial downside risk, as they believe current stock prices reflect inflated expectations around future AI-driven initiatives.


[As HPs new Copilot Plus AI PCs reach India, it sets its sights on businesses to break into the market - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Copilot, Gen AI, Hp, India, Products 

- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life.
- The new PCs are targeted at the SMB segment, promoting upgrades and offering tailored recovery solutions to encourage the adoption of premium devices in a hybrid work environment.
- Competition in the AI PC market is intensifying, with Qualcomm challenging Intel and AMD, and projections indicate AI PCs could represent 50% of the market within two years as affordability and integration improve.


[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.


[2 Monster AI Growth Stocks to Buy Before They Join Microsoft and Apple as $3 Trillion Companies - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Finance, Gen AI, Investment, Stocks 

- Nvidia is projected to reach a market capitalization of $3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure, with Wall Street expecting a 34% annual growth in earnings per share over the next three years.
- Alphabet could achieve a $3 trillion valuation by the end of 2025, bolstered by its position as a leading adtech company and third-largest cloud provider, with a strong focus on AI innovations that enhance ad and cloud services.
- Both companies have significantly outperformed the S&P 500 in the past five years, and analyst expectations suggest continued growth, making them attractive investments ahead of potential entry into the elite $3 trillion club.


[NIO unveils new operating system, 5nm chip, and phone at an AI event in Shanghai - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9uZXdzLzQxMjkyMzktbmlvLXVudmVpbHMtbmV3LW9wZXJhdGluZy1zeXN0ZW0tNW5tLWNoaXAtYW5kLXBob25lLWF0LWFuLWFpLWV2ZW50LWluLXNoYW5naGFp0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, China, Nio, Products, Semiconductor Chips 

- Access to the page has been denied, requiring user verification to confirm they are human.
- Users can report issues using Reference ID 5be3f380-4cd7-11ef-bf61-21a3e22fd362 for assistance.
- Feedback options are provided for users experiencing various problems with the page.


# Text-To-Image Diffusion Models 

# ChatGPT in AI Dominance 

[What's something you use ChatGPT for that you're sure no one else does? - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/)

Topics: Chatgpt, Opinion, Reddit 

- Users share unique and creative ways they utilize ChatGPT, including real-time chatbots, interactive art installations, and simulations for decision-making.
- Several participants highlight using ChatGPT for tasks like summarizing legal documents, providing recipe enhancements, and generating personalized book or movie recommendations.
- Some individuals experiment with humor and creativity, such as creating fictional scenarios, generating personalized poems, or even conducting debates between ChatGPT and other AI models.


[How to join the SearchGPT waitlist  heres how to get early access - Google News](https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LnRvbXNndWlkZS5jb20vYWkvaG93LXRvLWFjY2Vzcy1zZWFyY2hncHQtaGVyZXMtaG93LXRvLXNpZ24tdXAtZm9yLXRoZS13YWl0bGlzdNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Products 

- OpenAI's SearchGPT, a prototype search engine utilizing GPT-4 AI, offers real-time internet-based information retrieval and organizes search results more effectively than traditional search engines.
- Users can join the SearchGPT waitlist by having a ChatGPT account and signing up through a designated link; initial testing will involve up to 10,000 participants.
- Results from SearchGPT will include in-line attributions and links for further exploration, with plans to eventually integrate its features into ChatGPT.


[OpenAI is testing web search features for ChatGPT, challenging Google - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Gen AI, Google, Llms, OpenAI, Products, Testing 

- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI.
- Concerns from publishers about the potential negative impact on their businesses are rising, as AI-generated answers might undercut traffic from traditional search results, despite OpenAI's efforts to collaborate with media organizations.
- The new search tool aims to provide sources and direct links to information, while OpenAI signals future improvements that may include advertisements, further competing with Google's established business model.


[Chat-GPT Inaccuracy - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edl1yt/chatgpt_inaccuracy/)

Topics: Big Tech, Chatgpt, Disinformation, Ethics, Gen AI, Language Models, Llms, Machine Learning, OpenAI 

- A user reported that 60% of the information obtained from ChatGPT over 24 hours was inaccurate or fabricated, raising concerns about the model's reliability.
- The user emphasizes the need for constant fact-checking, which undermines the convenience of using the AI tool for queries.
- Another user mentions Myko Assistant as a preferable alternative for complex queries, stating it provides more accurate and complete information.


[ChatGPT Is Not Providing the Accurate Information I Need. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/)

Topics: Chatgpt, Customer Service, Disinformation, Gen AI, Language Models, Llms, OpenAI, Opinion, Review 

- Users are experiencing issues with ChatGPT's accuracy, particularly in obtaining detailed professional profiles and accurate industry trends, leading to challenges in their work.
- Suggestions for alternatives include using Myko Assistant, which is reported to provide more reliable information without complicated setup processes.
- ChatGPT is primarily a language generator and needs specific prompts to yield accurate results; it does not inherently understand truth and can create incorrect information instead of admitting limitations.


[What are the chances that all this Marketing Hype? And AI remains as basic as it is now. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/)

Topics: AI Doom, Bubble, Opinion 

- Current AI, particularly LLMs, has potential use cases but fundamentally lacks the ability to create high-quality music, journalism, or cinema, with outputs primarily summarizing existing knowledge.
- The hype surrounding AI advancements is often viewed as marketing rather than reality, with experts agreeing that the field faces significant challenges and technological plateaus within a few years.
- The financial sustainability of AI companies is in jeopardy as they continue to burn through capital without a clear revenue model, raising questions about how long they can persist before funding runs out.


[Sci-fi ChatGPT Voice Mode that lets you talk to AI like a human lands next week  trick to try it before... - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Entertainment, Gen AI, Language Models, Llms, OpenAI, Products, Science, Voice Mode 

- OpenAI's upgraded Voice Mode for ChatGPT, powered by the new GPT-4o model, will enable more humanlike spoken conversations and is set to launch next week for Plus subscribers.
- The new Voice Mode features significantly reduced response times, averaging around 320 milliseconds compared to 5.4 seconds with the previous generation, allowing for smoother interactions.
- GPT-4o will also support real-time translation, enabling conversations between users speaking different languages, and offers the ability to adjust speaking tones and pace for personalized interactions.


[Unifying gen X, Y, Z and boomers: The overlooked secret to AI success - Google News](https://news.google.com/articles/CBMiXmh0dHBzOi8vdmVudHVyZWJlYXQuY29tL2FpL3VuaWZ5aW5nLWdlbi14LXktei1hbmQtYm9vbWVycy10aGUtb3Zlcmxvb2tlZC1zZWNyZXQtdG8tYWktc3VjY2Vzcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Business, Gen AI, Opinion, Society, Culture 

- Effective implementation of generative AI (gen AI) relies on collaboration between IT veterans and AI-native talent, leveraging their different skills and experiences.
- Organizations face challenges such as resource scarcity and the need for alignment on objectives when integrating gen AI into existing systems.
- Continuous education for IT professionals in data science can enhance understanding and cooperation with AI specialists, fostering successful gen AI initiatives.


[Elon Musks X Default Setting Lets Posts Train His AI Chatbot Grok, But Users Can Opt Out - Google News](https://news.google.com/articles/CBMiRGh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvZWxvbi1tdXNrLXgtYWktY2hhdGJvdC1ncm9rLTEyMzYwMjQwNzcv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatbot, Data, Elon Musk, Gen AI, Language Models, Llms, Machine Learning, Privacy, Products, X 

- Elon Musk's X platform has introduced a default setting that uses user posts to help train the AI chatbot Grok, allowing for better fine-tuning of the assistant.
- Users have the option to opt out of this setting and can make their accounts private to prevent their content from being used for training.
- Grok, linked to Musk's AI startup xAI, was rolled out to premium subscribers and is designed similarly to OpenAI's ChatGPT, despite Musk's previous calls for caution regarding AI development.


[OpenAI, Home Of ChatGPT, May Lose $5B This Year  Report - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Economics, Finance, Funding, Gen AI, Llms, OpenAI, Stocks 

- OpenAI may incur losses of up to $5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT.
- The company's expenditures include approximately $4 billion for server capacity from Microsoft and around $3 billion for training AI models with new data.
- The path to profitability is questioned amid increasing competition, raising concerns from investors about OpenAI's unique technology and sustainability in the market.




In [86]:
docid_list

[27,
 60,
 72,
 41,
 24,
 77,
 14,
 13,
 49,
 46,
 53,
 18,
 5,
 40,
 77,
 41,
 14,
 49,
 35,
 60,
 50,
 74,
 24,
 40,
 49,
 58,
 17,
 13,
 27,
 14,
 12,
 18,
 23,
 22,
 50,
 40,
 41,
 77,
 35,
 51,
 49,
 23,
 22,
 18,
 40,
 65,
 35,
 75,
 47,
 21,
 77,
 18,
 23,
 22,
 60,
 77,
 40,
 21,
 47,
 48,
 41,
 11,
 41,
 10,
 57,
 18,
 60,
 77,
 21,
 47,
 5,
 34,
 70,
 65,
 48,
 75,
 46,
 77,
 27,
 71,
 57,
 49,
 24,
 70,
 74,
 75,
 48,
 60,
 76,
 47,
 8,
 34,
 77,
 6,
 66,
 8,
 7,
 17,
 30,
 41,
 18,
 37,
 11,
 10,
 41,
 9,
 51,
 37,
 50,
 77,
 68,
 64,
 0,
 36,
 5,
 48,
 57,
 46,
 4,
 47,
 14,
 21,
 27,
 60,
 74,
 14,
 24,
 77,
 72,
 49,
 34,
 13,
 69,
 14,
 49,
 27,
 18,
 40,
 32,
 72,
 54,
 60,
 20,
 74,
 40,
 23,
 22,
 18,
 40,
 50,
 57,
 32,
 51,
 5,
 4,
 17,
 18,
 53,
 66,
 6,
 60,
 8,
 58,
 7,
 76,
 23,
 22,
 18,
 40,
 50,
 57,
 32,
 51,
 5,
 4,
 56,
 32,
 61,
 38,
 64,
 14,
 40,
 57,
 35,
 21,
 50,
 47,
 60,
 70,
 15,
 48,
 0,
 57,
 75,
 6,
 66,
 8,
 7,
 75,
 24,
 57,
 32,
 46,
 27,
 

In [87]:
# write sections individually

mail_md_str = ""

for current_topic, cat in enumerate(my_cats):

    section_prompt = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input.

Input:
I will provide a markdown list of today's news articles on the topic: {my_cats[current_topic]}.
The input will be in the format
[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Instructions:

Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Write an engaging summary consisting of a title and at least 1 and no more than 5 bullet points.
Use as few bullet points as you need to provide the most significant facts.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.
DO NOT REPEAT LINKS FROM PREVIOUS BULLET POINTS.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.

Please check carefully that you only use information provided in the following input, and that any bullet point
does not repeat information or links prevously provided.

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging title

- bullet point a - [site name a](site url a)
- bullet point b - [site name b ](site url b)

Input:

{doc_list[current_topic]}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", "content": section_prompt
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\$")
    mail_md_str += response_str + " \n\n"
    display(Markdown(response_str))


2024-07-28 07:52:53,432 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

2024-07-28 07:52:55,472 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Advances in Machine Learning for Structural Biology

- The journey of learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- Insights into transforming various data types into vectors for pattern recognition in high-dimensional space are discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- The potential of deep neural networks as "universal function approximators" is examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).

2024-07-28 07:53:05,381 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

# AI Chip Demand Surge

- SK Hynix is investing \$6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips - [Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Researchers at TU Eindhoven developed a neuromorphic chip for on-chip neural network training, eliminating the need for model transfers and significantly reducing energy consumption and enhancing efficiency - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS9yZWFsLXRpbWUtbmV1cmFsLW5ldHdvcmstdHJhaW5pbmctb24tY2hpcC0yMzM2ODY4MtIBYWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL2FtcC90ZWNobm9sb2d5L2FydGljbGUvcmVhbC10aW1lLW5ldXJhbC1uZXR3b3JrLXRyYWluaW5nLW9uLWNoaXAtMjMzNjg2ODI?hl=en-US&gl=US&ceid=US:en).
- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from \$120 to \$140 and Loop Capital from \$120 to \$175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch - [Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is projected to reach a market capitalization of \$3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en).

2024-07-28 07:56:26,768 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Advances in Text-To-Image Diffusion Models

- [Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

- [Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

2024-07-28 07:56:36,198 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# ChatGPT in AI Dominance

- Users share unique and creative ways they utilize ChatGPT, including real-time chatbots, interactive art installations, and simulations for decision-making - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/).
- OpenAI's SearchGPT, a prototype search engine utilizing GPT-4 AI, offers real-time internet-based information retrieval and organizes search results more effectively than traditional search engines - [Google News](https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LnRvbXNndWlkZS5jb20vYWkvaG93LXRvLWFjY2Vzcy1zZWFyY2hncHQtaGVyZXMtaG93LXRvLXNpZ24tdXAtZm9yLXRoZS13YWl0bGlzdNIBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI - [Washington Post](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en).
- A user reported that 60% of the information obtained from ChatGPT over 24 hours was inaccurate or fabricated, raising concerns about the model's reliability - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1edl1yt/chatgpt_inaccuracy/).
- OpenAI's upgraded Voice Mode for ChatGPT, powered by the new GPT-4o model, will enable more humanlike spoken conversations and is set to launch next week for Plus subscribers - [The Sun](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en).
- Elon Musk's X platform has introduced a default setting that uses user posts to help train the AI chatbot Grok, allowing for better fine-tuning of the assistant - [Deadline](https://news.google.com/articles/CBMiRGh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvZWxvbi1tdXNrLXgtYWktY2hhdGJvdC1ncm9rLTEyMzYwMjQwNzcv0gEA?hl=en-US&gl=US&ceid=US:en).
- OpenAI may incur losses of up to \$5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT - [Deadline](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en).

In [88]:
print(mail_md_str)

# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 2

In [89]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(mail_md_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-28 07:56:36,218 - AInewsbot - INFO - Sending full summary email 


# Final Summary

In [90]:
# summarize by just giving selected stories in semantic order and hinting how to write it
AIdf.loc[AIdf['id'].isin(set(docid_list))]

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,OpenAI is testing web search features for Chat...,https://news.google.com/articles/CBMiUmh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiUmh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Gen AI, Google, Llms, OpenA...",OpenAI is testing web search features for Chat...,14,0,ChatGPT and AI Innovations,htmldata/OpenAI_is_testing_web_search_features...
2,2,Google News,Sci-fi ChatGPT Voice Mode that lets you talk t...,https://news.google.com/articles/CBMiZGh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiZGh0dHBzO...,news.google.com,Google News,"Big Tech, Chatgpt, Entertainment, Gen AI, Lang...",Sci-fi ChatGPT Voice Mode that lets you talk t...,43,0,ChatGPT and AI Innovations,htmldata/Sci-fi_ChatGPT_Voice_Mode_that_lets_y...
4,4,Google News,Google dangles bomber jackets and spot bonuses...,https://news.google.com/articles/CBMiSWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiSWh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Google, Jobs, Careers, Labor...",Google dangles bomber jackets and spot bonuses...,25,1,Google and AI Innovations,htmldata/Google_dangles_bomber_jackets_and_spo...
5,5,Google News,Google restates commitment to empowering busin...,https://news.google.com/articles/CBMiUGh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiUGh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Google",Google restates commitment to empowering busin...,29,1,Google and AI Innovations,htmldata/Google_restates_commitment_to_empower...
6,6,Google News,Mistral AI Unveils Mistral Large 2: A Powerful...,https://news.google.com/articles/CBMiZGh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiZGh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Language Models, Llms, Machi...",Mistral AI Unveils Mistral Large 2: A Powerful...,52,2,Mistral Large 2 Language Model,htmldata/Mistral_AI_Unveils_Mistral_Large_2__A...
7,7,Google News,Mistral Large 2 is now available in Amazon Bed...,https://news.google.com/articles/CBMiYWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiYWh0dHBzO...,news.google.com,Google News,"Amazon, Big Tech, Gen AI, Language Models, Llm...",Mistral Large 2 is now available in Amazon Bed...,61,2,Mistral Large 2 Language Model,htmldata/Mistral_Large_2_is_now_available_in_A...
8,8,Google News,Mistrals Large 2 is its answer to Meta and Ope...,https://news.google.com/articles/CBMiUWh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiUWh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Google, Language Models, Llm...",Mistrals Large 2 is its answer to Meta and Ope...,62,2,Mistral Large 2 Language Model,htmldata/Mistrals_Large_2_is_its_answer_to_Met...
9,9,Google News,Nvidia supplier SK Hynix to build $6.8 billion...,https://news.google.com/articles/CBMib2h0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMib2h0dHBzO...,news.google.com,Google News,"Chip Plant, Deals, Economics, Korea, Manufactu...",Nvidia supplier SK Hynix to build $6.8 billion...,3,3,Nvidia and Semiconductor Manufacturing Investm...,htmldata/Nvidia_supplier_SK_Hynix_to_build__6_...
10,10,Google News,Nvidia supplier SK hynix to invest $6.8B to bu...,https://news.google.com/articles/CBMibmh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMibmh0dHBzO...,news.google.com,Google News,"Deals, Economics, Funding, Investment, Korea, ...",Nvidia supplier SK hynix to invest $6.8B to bu...,50,3,Nvidia and Semiconductor Manufacturing Investm...,htmldata/Nvidia_supplier_SK_hynix_to_invest__6...
11,11,Google News,Nvidia Supplier SK Hynix To Spend $6.8B on Sou...,https://news.google.com/articles/CBMiemh0dHBzO...,True,2024-07-28,https://news.google.com/articles/CBMiemh0dHBzO...,news.google.com,Google News,"Big Tech, Chip Plant, Deals, Economics, Fundin...",Nvidia Supplier SK Hynix To Spend $6.

In [91]:
docs[16].page_content

"[ChatGPT Is Not Providing the Accurate Information I Need. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edq7am/chatgpt_is_not_providing_the_accurate_information/)\n\nTopics: Chatgpt, Customer Service, Disinformation, Gen AI, Language Models, Llms, OpenAI, Opinion, Review \n\n- Users are experiencing issues with ChatGPT's accuracy, particularly in obtaining detailed professional profiles and accurate industry trends, leading to challenges in their work.\n- Suggestions for alternatives include using Myko Assistant, which is reported to provide more reliable information without complicated setup processes.\n- ChatGPT is primarily a language generator and needs specific prompts to yield accurate results; it does not inherently understand truth and can create incorrect information instead of admitting limitations.\n\n"

In [92]:
# make text for email and also collect data for vector store
markdown_str = ''
print()

for i, row in enumerate(AIdf.loc[AIdf['id'].isin(set(docid_list))].itertuples()):
    mdstr = docs[row.id].page_content
    display(Markdown(mdstr.replace('$', '\\$')))
    markdown_str += mdstr
    

[OpenAI is testing web search features for ChatGPT, challenging Google - Google News](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Gen AI, Google, Llms, OpenAI, Products, Testing 

- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI.
- Concerns from publishers about the potential negative impact on their businesses are rising, as AI-generated answers might undercut traffic from traditional search results, despite OpenAI's efforts to collaborate with media organizations.
- The new search tool aims to provide sources and direct links to information, while OpenAI signals future improvements that may include advertisements, further competing with Google's established business model.



[Sci-fi ChatGPT Voice Mode that lets you talk to AI like a human lands next week  trick to try it before... - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Entertainment, Gen AI, Language Models, Llms, OpenAI, Products, Science, Voice Mode 

- OpenAI's upgraded Voice Mode for ChatGPT, powered by the new GPT-4o model, will enable more humanlike spoken conversations and is set to launch next week for Plus subscribers.
- The new Voice Mode features significantly reduced response times, averaging around 320 milliseconds compared to 5.4 seconds with the previous generation, allowing for smoother interactions.
- GPT-4o will also support real-time translation, enabling conversations between users speaking different languages, and offers the ability to adjust speaking tones and pace for personalized interactions.



[Google dangles bomber jackets and spot bonuses in search for 'Golden Prompts' for AI, leaked documents show - Google News](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning 

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software, offering prizes like golden bomber jackets.
- The competition aimed to develop effective prompts for tasks such as managing customer inquiries, invoicing, and business leads to compete with Microsoft's AI capabilities in Office 365.
- Seven winning prompts were selected, including solutions for responding to Google reviews and organizing receipts, with plans to integrate them into future Workspace updates.



[Google restates commitment to empowering businesses with AI - Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Google 

- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations.
- Gemini offers a collaborative creative partnership that enhances decision-making and performance, with intuitive prompt interactions that differ from traditional search methods.
- The model supports multisensory experiences by delivering information in various formats such as visuals, sounds, and videos, aimed at helping businesses tackle challenges effectively.



[Mistral AI Unveils Mistral Large 2: A Powerful New Language Model - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, outperforming Llama 3.1 and closely following GPT-4 on benchmarks like HumanEval, MultiPL-E, and the MATH benchmark.
- The model features enhanced code generation, mathematics, and multilingual capabilities, achieving an 84.0% accuracy on the Multilingual MMLU benchmark and surpassing its competitors in multilingual performance.
- Mistral Large 2 is designed for efficiency in single-node inference, emphasizing throughput for long-context applications, and aims to set a new standard for performance-to-cost ratio in AI language models.



...

[Unifying gen X, Y, Z and boomers: The overlooked secret to AI success - Google News](https://news.google.com/articles/CBMiXmh0dHBzOi8vdmVudHVyZWJlYXQuY29tL2FpL3VuaWZ5aW5nLWdlbi14LXktei1hbmQtYm9vbWVycy10aGUtb3Zlcmxvb2tlZC1zZWNyZXQtdG8tYWktc3VjY2Vzcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Business, Gen AI, Opinion, Society, Culture 

- Effective implementation of generative AI (gen AI) relies on collaboration between IT veterans and AI-native talent, leveraging their different skills and experiences.
- Organizations face challenges such as resource scarcity and the need for alignment on objectives when integrating gen AI into existing systems.
- Continuous education for IT professionals in data science can enhance understanding and cooperation with AI specialists, fostering successful gen AI initiatives.



[AI is making robots smarter. Theyll need boundaries. - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Robots, Science 

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability.
- The integration of AI with robotics raises significant ethical and safety concerns, particularly regarding autonomous machines in potentially lethal roles, necessitating clear regulations and human oversight.
- As the U.S. faces growing competition in the robotics sector, there is an urgent need to establish a robust domestic supply chain and enhance safety measures for robots to ensure responsible usage.



[Mark Zuckerberg says there will be more AI agents than people as businesses and individuals creates AI agents that reflect their values and interact with the world on their behalf - Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

Topics: Big Tech, Business, Gen AI, Llms, Mark Zuckerberg 

- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values.
- The conversation raises concerns about AI agents loosely interfacing as representatives, replacing traditional roles like call center agents with automated systems.
- Many commenters express skepticism about the effectiveness and authenticity of these AI agents, likening them to existing problematic online behaviors.



[What are the chances that all this Marketing Hype? And AI remains as basic as it is now. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/)

Topics: AI Doom, Bubble, Opinion 

- Current AI, particularly LLMs, has potential use cases but fundamentally lacks the ability to create high-quality music, journalism, or cinema, with outputs primarily summarizing existing knowledge.
- The hype surrounding AI advancements is often viewed as marketing rather than reality, with experts agreeing that the field faces significant challenges and technological plateaus within a few years.
- The financial sustainability of AI companies is in jeopardy as they continue to burn through capital without a clear revenue model, raising questions about how long they can persist before funding runs out.



[The hot AI trade is entering a danger zone: Morgan Stanley chief investment officer [Video] - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Bubble, Economics, Finance, Gen AI, Investment, Opinion, Stocks 

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers.
- Recent disappointments from companies like Salesforce and ServiceNow suggest that the anticipated revenue boosts from AI tools have not materialized, with experts projecting delays in significant monetization efforts.
- While there is high demand for AI-related technology, exemplified by Nvidia’s peak market valuation, Wilson indicates that expectations for AI growth may have outpaced the actual timeline for its commercialization.



In [93]:
# clean up the list of topics in human in the loop workflow
# loop though each topic and summarize, then 
# then combine the summaries for a final prompt

In [94]:
TESTPROMPT = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input. You are able to categorize information, 
and identify trends from large volumes of news.

Objective: 
I will provide the text of today's news articles about AI and summary bullet points in markdown format.
Bullet points will contain a title and URL, a list of topics discussed, and a bullet-point summary of
the article. You are tasked with identifying and summarizing the most important news, recurring themes,
common facts and items. Your job is to create a concise summary of today's topics and developments.
You will write an engaging summary of today's news encompassing the most important and frequently 
mentioned topics and themes.
You will write in the professional but engaging, narrative style of a tech reporter for a national publication.
You will be balanced, professional, informative, providing accurate, clear, concise summaries in a neutral tone.
You will group stories into related topics

Input Format Template:

[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging-topic-title-1

- bullet-point-1a - [site-name-1a](site-url-1a)
- bullet-point-1b - [site-name-1b](site-url-1b)

# Engaging-topic-title-2

- bullet-point-2a - [site-name-2a](site-url-2a)
- bullet-point-2b - [site-name-2b](site-url-2b)

Instructions:

Read the input closely.
Very important: USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.

Please check carefully that you only use information provided in the following input, that you include
all links in the input, and that any bullet point does not repeat information or links prevously provided.

Input:

"""



In [95]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TESTPROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)

response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


2024-07-28 07:57:12,397 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI and Big Tech Developments

## OpenAI's Innovations

- OpenAI is introducing a new web search tool for ChatGPT, challenging Google's search dominance and aiming for a more interactive web experience through AI - [Washington Post](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)
- OpenAI's upgraded Voice Mode for ChatGPT will enable more humanlike spoken conversations and is set to launch next week - [The Sun](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en)
- OpenAI may incur losses of up to \$5 billion this year due to high operational and training costs - [Deadline](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

## Google and AI

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software - [Business Insider](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model - [Punchng](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

## Mistral AI's New Model

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, aiming to set a new standard for performance-to-cost ratio in AI language models - [Maginative](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en)
- Mistral Large 2 is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, coding, and more - [Amazon](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en)

## AI in Manufacturing and Robotics

- SK Hynix plans to invest \$6.8 billion to build a new semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips - [CNBC](https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjYvbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWJ1aWxkLTZwb2ludDgtYmlsbGlvbi1jaGlwLXBsYW4taW4tc291dGgta29yZWEuaHRtbNIBc2h0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDI0LzA3LzI2L252aWRpYS1zdXBwbGllci1zay1oeW5peC10by1idWlsZC02cG9pbnQ4LWJpbGxpb24tY2hpcC1wbGFuLWluLXNvdXRoLWtvcmVhLmh0bWw?hl=en-US&gl=US&ceid=US:en)
- Researchers at TU Eindhoven developed a neuromorphic chip for on-chip neural network training, eliminating the need for model transfers and significantly reducing energy consumption - [Mid-Day](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS9yZWFsLXRpbWUtbmV1cmFsLW5ldHdvcmstdHJhaW5pbmctb24tY2hpcC0yMzM2ODY4MtIBYWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL2FtcC90ZWNobm9sb2d5L2FydGljbGUvcmVhbC10aW1lLW5ldXJhbC1uZXR3b3JrLXRyYWluaW5nLW9uLWNoaXAtMjMzNjg2ODI?hl=en-US&gl=US&ceid=US:en)

## AI in Science and Healthcare

- USC researchers are developing an AI model that uses real-time satellite data to predict wildfire behavior, enhancing firefighting efforts - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- Hyperspectral imaging (HSI) combined with machine learning (ML) shows promising potential for accurately assessing renal perfusion states during surgery, distinguishing between various malperfusion scenarios - [Nature](https://news.google.com/articles/CBMiMmh0dHBzOi8vd3d3Lm5hdHVyZS5jb20vYXJ0aWNsZXMvczQxNTk4LTAyNC02ODI4MC0z0gEA?hl=en-US&gl=US&ceid=US:en)

## AI in Consumer Electronics

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge for users with disabilities - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)
- MSI has launched a new line of high-end laptops powered by the AMD Ryzen AI 300 series "Strix Point" APUs, aimed at gamers, content creators, and workstation users - [TweakTown](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy50d2Vha3Rvd24uY29tL25ld3MvOTk1NzgvbXNpLXVudmVpbHMtZmxlZXQtb2YtbmV3LWhpZ2gtZW5kLWFtZC1yeXplbi1haS0zMDAtc3RyaXgtcG9pbnQtYXB1LXBvd2VyZWQtbGFwdG9wcy9pbmRleC5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

## AI and Ethics

- TikTok and other social media algorithms can quickly lead users down hate-filled content rabbit holes, impacting mental well-being and promoting extreme views - [Reddit](https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/)
- X's new AI training opt-out setting has drawn scrutiny from regulators and users express skepticism about its effectiveness - [Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/)

## AI and Finance

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \$2.2 billion investment focus in AI - [Fool](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

## AI and Policy

- South Carolina's Department of Administration announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits - [Post and Courier](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- Apple has signed a non-binding commitment to AI safety initiated by the White House - [Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/)

## AI and Society

- The author argues that recent advancements in smartphones, including the new iPhone, represent incremental improvements rather than true innovation, mirroring a broader pattern in consumer electronics - [The Guardian](https://news.google.com/articles/CBMijQFodHRwczovL3d3dy50aGVndWFyZGlhbi5jb20vY29tbWVudGlzZnJlZS9hcnRpY2xlLzIwMjQvanVsLzI3L215LW5ldy1pcGhvbmUtc3ltYm9saXNlcy1zdGFnbmF0aW9uLW5vdC1pbm5vdmF0aW9uLWFuZC1hLXNpbWlsYXItZmF0ZS1hd2FpdHMtYWnSAY0BaHR0cHM6Ly9hbXAudGhlZ3VhcmRpYW4uY29tL2NvbW1lbnRpc2ZyZWUvYXJ0aWNsZS8yMDI0L2p1bC8yNy9teS1uZXctaXBob25lLXN5bWJvbGlzZXMtc3RhZ25hdGlvbi1ub3QtaW5ub3ZhdGlvbi1hbmQtYS1zaW1pbGFyLWZhdGUtYXdhaXRzLWFp?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

In [96]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


# AI and Big Tech Developments

## OpenAI's Innovations

- OpenAI is introducing a new web search tool for ChatGPT, challenging Google's search dominance and aiming for a more interactive web experience through AI - [Washington Post](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en)
- OpenAI's upgraded Voice Mode for ChatGPT will enable more humanlike spoken conversations and is set to launch next week - [The Sun](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en)
- OpenAI may incur losses of up to \$5 billion this year due to high operational and training costs - [Deadline](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en)

## Google and AI

- Google held an internal competition for employees to create "golden prompts" to enhance AI features within its Workspace productivity software - [Business Insider](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model - [Punchng](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)

## Mistral AI's New Model

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, aiming to set a new standard for performance-to-cost ratio in AI language models - [Maginative](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en)
- Mistral Large 2 is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, coding, and more - [Amazon](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en)

## AI in Manufacturing and Robotics

- SK Hynix plans to invest \$6.8 billion to build a new semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips - [CNBC](https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjYvbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLWJ1aWxkLTZwb2ludDgtYmlsbGlvbi1jaGlwLXBsYW4taW4tc291dGgta29yZWEuaHRtbNIBc2h0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDI0LzA3LzI2L252aWRpYS1zdXBwbGllci1zay1oeW5peC10by1idWlsZC02cG9pbnQ4LWJpbGxpb24tY2hpcC1wbGFuLWluLXNvdXRoLWtvcmVhLmh0bWw?hl=en-US&gl=US&ceid=US:en)
- Researchers at TU Eindhoven developed a neuromorphic chip for on-chip neural network training, eliminating the need for model transfers and significantly reducing energy consumption - [Mid-Day](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS9yZWFsLXRpbWUtbmV1cmFsLW5ldHdvcmstdHJhaW5pbmctb24tY2hpcC0yMzM2ODY4MtIBYWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL2FtcC90ZWNobm9sb2d5L2FydGljbGUvcmVhbC10aW1lLW5ldXJhbC1uZXR3b3JrLXRyYWluaW5nLW9uLWNoaXAtMjMzNjg2ODI?hl=en-US&gl=US&ceid=US:en)

## AI in Science and Healthcare

- USC researchers are developing an AI model that uses real-time satellite data to predict wildfire behavior, enhancing firefighting efforts - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- Hyperspectral imaging (HSI) combined with machine learning (ML) shows promising potential for accurately assessing renal perfusion states during surgery, distinguishing between various malperfusion scenarios - [Nature](https://news.google.com/articles/CBMiMmh0dHBzOi8vd3d3Lm5hdHVyZS5jb20vYXJ0aWNsZXMvczQxNTk4LTAyNC02ODI4MC0z0gEA?hl=en-US&gl=US&ceid=US:en)

## AI in Consumer Electronics

- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge for users with disabilities - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)
- MSI has launched a new line of high-end laptops powered by the AMD Ryzen AI 300 series "Strix Point" APUs, aimed at gamers, content creators, and workstation users - [TweakTown](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy50d2Vha3Rvd24uY29tL25ld3MvOTk1NzgvbXNpLXVudmVpbHMtZmxlZXQtb2YtbmV3LWhpZ2gtZW5kLWFtZC1yeXplbi1haS0zMDAtc3RyaXgtcG9pbnQtYXB1LXBvd2VyZWQtbGFwdG9wcy9pbmRleC5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

## AI and Ethics

- TikTok and other social media algorithms can quickly lead users down hate-filled content rabbit holes, impacting mental well-being and promoting extreme views - [Reddit](https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/)
- X's new AI training opt-out setting has drawn scrutiny from regulators and users express skepticism about its effectiveness - [Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/)

## AI and Finance

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \$2.2 billion investment focus in AI - [Fool](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

## AI and Policy

- South Carolina's Department of Administration announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits - [Post and Courier](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- Apple has signed a non-binding commitment to AI safety initiated by the White House - [Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/)

## AI and Society

- The author argues that recent advancements in smartphones, including the new iPhone, represent incremental improvements rather than true innovation, mirroring a broader pattern in consumer electronics - [The Guardian](https://news.google.com/articles/CBMijQFodHRwczovL3d3dy50aGVndWFyZGlhbi5jb20vY29tbWVudGlzZnJlZS9hcnRpY2xlLzIwMjQvanVsLzI3L215LW5ldy1pcGhvbmUtc3ltYm9saXNlcy1zdGFnbmF0aW9uLW5vdC1pbm5vdmF0aW9uLWFuZC1hLXNpbWlsYXItZmF0ZS1hd2FpdHMtYWnSAY0BaHR0cHM6Ly9hbXAudGhlZ3VhcmRpYW4uY29tL2NvbW1lbnRpc2ZyZWUvYXJ0aWNsZS8yMDI0L2p1bC8yNy9teS1uZXctaXBob25lLXN5bWJvbGlzZXMtc3RhZ25hdGlvbi1ub3QtaW5ub3ZhdGlvbi1hbmQtYS1zaW1pbGFyLWZhdGUtYXdhaXRzLWFp?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)

In [97]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-28 07:57:12,531 - AInewsbot - INFO - Sending full summary email 


In [98]:
log("Finished")


2024-07-28 07:57:13,583 - AInewsbot - INFO - Finished


In [99]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at refining and enhancing written materials, specializing in
ensuring clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Edit the markdown newsletter provided below by removing any redundant
sentences or bullet points that restate previous points and contain the same link.
Leave intact bullet points that are unique and provide distinct information.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information or provide identical links.
Remove all redundant sentences and bullet points that do not contribute new information or unique links.
Ensure that the remaining content flows logically and maintains the intended message and tone of the newsletter.
Double-check the final edited version for any inconsistencies or errors introduced during the editing process.
Take a deep breath and work on this problem step-by-step.
"""

'You will act like a professional editor with expertise in content optimization.\nYou are skilled at refining and enhancing written materials, specializing in\nensuring clarity, conciseness, and coherence in various types of documents,\nincluding newsletters.\n\nObjective: Edit the markdown newsletter provided below by removing any redundant\nsentences or bullet points that restate previous points and contain the same link.\nLeave intact bullet points that are unique and provide distinct information.\n\nStep-by-step instructions:\n\nCarefully read through the entire newsletter to understand the overall structure and content.\nIdentify sentences and bullet points that repeat information or provide identical links.\nRemove all redundant sentences and bullet points that do not contribute new information or unique links.\nEnsure that the remaining content flows logically and maintains the intended message and tone of the newsletter.\nDouble-check the final edited version for any inconsiste

In [100]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below and advise on ways to improve it.
Note any links which are repeated, any sections which are similar and could be combined,
and any copy edits. You will only provide suggestions, and not rewrite the copy.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information and provide identical links and should be removed.
Identify any sections which could be combined because they contain similar but not identical content.
Suggest improvements to any sections which are not clear, concise, and coherent.
Take a deep breath and work on this problem step-by-step.
"""

'You will act like a professional editor with expertise in content optimization.\nYou are skilled at reviewing and enhancing written materials, specializing in\nhelping improve clarity, conciseness, and coherence in various types of documents,\nincluding newsletters.\n\nObjective: Review the markdown newsletter provided below and advise on ways to improve it.\nNote any links which are repeated, any sections which are similar and could be combined,\nand any copy edits. You will only provide suggestions, and not rewrite the copy.\n\nStep-by-step instructions:\n\nCarefully read through the entire newsletter to understand the overall structure and content.\nIdentify sentences and bullet points that repeat information and provide identical links and should be removed.\nIdentify any sections which could be combined because they contain similar but not identical content.\nSuggest improvements to any sections which are not clear, concise, and coherent.\nTake a deep breath and work on this prob

In [101]:
mail_md_str

'# AI in Drug Discovery: Today\'s Key Developments\n\n- South Carolina\'s Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)\n- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)\n- Nvidia\'s stock is projected to su

In [102]:
display(Markdown(mail_md_str.replace("$", "\\$")))


# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en) 

# Advances in Machine Learning for Structural Biology

- The journey of learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- Insights into transforming various data types into vectors for pattern recognition in high-dimensional space are discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- The potential of deep neural networks as "universal function approximators" is examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/). 

# nference and Takeda AI Partnership

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en) 

# Ant-inspired AI for Drones

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility - [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from \\$12.3 billion in 2023 to \\$146.8 billion by 2033 - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)
- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics - [Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en) 

# Fei-Fei Li's \\$1B AI Startup

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast discusses new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en) 

# Artificial Agency's \\$16M Funding

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission - [Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en) 

# CTERA Secures \\$80M in Funding

- CTERA has raised \\$80 million in a new funding round - [TechCrunch](https://techcrunch.com)
- The funding will be used to expand CTERA's global operations and enhance its cloud storage solutions - [VentureBeat](https://venturebeat.com)
- This investment round was led by a mix of existing and new investors, highlighting strong market confidence - [Forbes](https://forbes.com) 

# Anthropic's \\$100M AI Fund: Key Developments

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en).

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs and staffing expenses - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en).

- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training - [Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).

- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en).

- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission - [Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en). 

# Samsung's AI Image Generation

- SK Hynix is investing \\$6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips - [Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en). 

# EU Antitrust Probe on AI Deals

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en).
- iFixit CEO Kyle Wiens criticized AI company Anthropic for making a million hits on their servers within 24 hours, causing resource strain despite their terms of service prohibiting such actions for AI training - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [AOL](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en).
- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission - [Engadget](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Punchng](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en). 

# Salesforce AI Service Agent: Today's Highlights

- Mark Zuckerberg predicts AI agents will surpass the human population, raising concerns about their role in replacing traditional jobs - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/).
- Redfin's AI-powered tool, Redfin Redesign, helps home sellers with design choices to make properties more appealing - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LnR1cm50bzIzLmNvbS91cy1uZXdzL2hvdXNpbmcvaGVyZXMtaG93LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNhbi1oZWxwLXlvdS1zZWxsLXlvdXItaG9tZdIBAA?hl=en-US&gl=US&ceid=US:en).
- Morgan Stanley's Mike Wilson warns that the AI trade may be entering a dangerous phase, with revenue boosts from AI tools like Salesforce not yet materializing - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en).
- South Carolina's Department of Administration announces an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en).
- Google's new AI tools in Product Studio help small businesses create compelling ads and improve digital visibility - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en). 

# Meta's AI Regulatory Issues

- X's new AI training opt-out setting faces regulatory scrutiny and user skepticism about its effectiveness - [Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/).
- Anthropic's ClaudeBot violated iFixit's anti-scraping policies, causing significant resource strain - [Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en).
- Mark Zuckerberg predicts AI agents will outnumber humans, raising concerns about their roles and authenticity - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/).
- Apple signs the White House's non-binding commitment to AI safety, prompting skepticism about tech companies' motives - [Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/).
- Morgan Stanley's Mike Wilson warns that the AI trade is entering a dangerous phase, with revenue boosts from AI tools yet to materialize - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en). 

# Nvidia and Mistral AI Model: Key Developments

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, outperforming Llama 3.1 and closely following GPT-4 on benchmarks like HumanEval and MATH - [Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en).
- Mistral Large 2 competes with larger models from OpenAI, Meta, and Anthropic, offering a 128,000 token context window and support for numerous languages - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFydGlmaWNpYWxpbnRlbGxpZ2VuY2UtbmV3cy5jb20vbmV3cy9taXN0cmFsLWxhcmdlLTItZGF2aWQtdG8tYmlnLXRlY2gtZ29saWF0aHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Mistral Large 2 is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, and coding - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en).
- Faulty Nvidia H100 GPUs and HBM3 memory caused 50% of failures during the training of LLaMA 3, impacting Meta's 16,384 GPU training cluster - [Reddit](https://www.reddit.com/r/technews/comments/1edzlsp/faulty_nvidia_h100_gpus_and_hbm3_memory_caused/).
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en). 

# TSMC AI Chip Demand Surge

- SK Hynix is investing \\$6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips - [Investopedia](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Analysts have significantly raised the price targets for Nvidia, driven by strong business trends and the anticipated growth from the Blackwell architecture launch - [Investing.com](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en).
- MSI has launched a new line of high-end laptops powered by the AMD Ryzen AI 300 series "Strix Point" APUs, aimed at gamers, content creators, and workstation users - [Tweaktown](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy50d2Vha3Rvd24uY29tL25ld3MvOTk1NzgvbXNpLXVudmVpbHMtZmxlZXQtb2YtbmV3LWhpZ2gtZW5kLWFtZC1yeXplbi1haS0zMDAtc3RyaXgtcG9pbnQtYXB1LXBvd2VyZWQtbGFwdG9wcy9pbmRleC5odG1s0gEA?hl=en-US&gl=US&ceid=US:en).
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [AOL](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en). 

# AI Search Engines: Today's Key Developments

- OpenAI is testing a new web search tool for ChatGPT, challenging Google's search dominance and aiming for a more interactive web experience - [Washington Post](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en).
- Users can join the SearchGPT waitlist for early access to OpenAI's prototype search engine, which utilizes GPT-4 AI for real-time information retrieval - [Tom's Guide](https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LnRvbXNndWlkZS5jb20vYWkvaG93LXRvLWFjY2Vzcy1zZWFyY2hncHQtaGVyZXMtaG93LXRvLXNpZ24tdXAtZm9yLXRoZS13YWl0bGlzdNIBAA?hl=en-US&gl=US&ceid=US:en).
- Google emphasized its commitment to empowering businesses with AI, showcasing the Gemini AI model's capabilities at the AI in Action event - [Punchng](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en).
- Anthropic's ClaudeBot has been accused of violating anti-scraping protocols by aggressively scraping websites like iFixit and Freelancer, leading to significant disruptions - [The Verge](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjUvMjQyMDU5NDMvYW50aHJvcGljLWFpLXdlYi1jcmF3bGVyLWNsYXVkZWJvdC1pZml4aXQtc2NyYXBpbmctdHJhaW5pbmctZGF0YdIBAA?hl=en-US&gl=US&ceid=US:en).
- Google held an internal competition to create "golden prompts" for enhancing AI features in Workspace, with winning prompts to be integrated into future updates - [Business Insider](https://news.google.com/articles/CBMiSWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vZ29vZ2xlLXdvcmtzcGFjZS1haS1wcm9tcHRzLWdlbWluaS0yMDI0LTfSAQA?hl=en-US&gl=US&ceid=US:en). 

# AI in Cybersecurity: Today's Highlights

- South Carolina's Department of Administration has launched a comprehensive AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en).
- iFixit CEO Kyle Wiens criticized Anthropic for making a million hits on their servers within 24 hours, causing resource strain - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- AI is enhancing the capabilities of robots in manufacturing and logistics, raising significant ethical and safety concerns - [Japan Times](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en).
- USC researchers are developing an AI model that uses real-time satellite data to predict wildfire behavior, enhancing firefighting efforts - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en). 

# AI in Healthcare & Precision Medicine

- Amanda Randles is developing "digital twins" to create personalized medical simulations, significantly improving non-invasive diagnosis and treatment of diseases by simulating blood flow in patients' circulatory systems - [With Digital Twins, The Doctor Will See You Now - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LnF1YW50YW1hZ2F6aW5lLm9yZy93aXRoLWRpZ2l0YWwtdHdpbnMtdGhlLWRvY3Rvci13aWxsLXNlZS15b3Utbm93LTIwMjQwNzI2L9IBAA?hl=en-US&gl=US&ceid=US:en).
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [HP may have plenty of AI software and hardware to show off, but accessibility is AI's best and only future - Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en).
- South Carolina's Department of Administration has announced a comprehensive artificial intelligence strategy to enhance government operations, focusing on transparency, fairness, and citizen benefits - [SC state government enters the artificial intelligence era | Palmetto Politics - Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en).
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Forget Nvidia: Billionaire Daniel Loeb Owns \\$2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Hyperspectral imaging (HSI) combined with machine learning (ML) shows promising potential for accurately assessing renal perfusion states during surgery, distinguishing between various malperfusion scenarios with a prediction accuracy of 97.8% - [Spectral characterization of intraoperative renal perfusion using hyperspectral imaging and artificial intelligence | Scientific Reports - Google News](https://news.google.com/articles/CBMiMmh0dHBzOi8vd3d3Lm5hdHVyZS5jb20vYXJ0aWNsZXMvczQxNTk4LTAyNC02ODI4MC0z0gEA?hl=en-US&gl=US&ceid=US:en). 

# Autonomous Drones: Innovations and Ethical Considerations

- Nuro, an autonomous delivery startup, is planning a comeback in the transportation sector - [Reddit](https://www.reddit.com/r/technology/comments/1edr9jq/autonomous_delivery_startup_nuro_is_gearing_up/)
- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Skild AI has raised \\$300 million to create a general-purpose foundation model for robotics - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en) 

# AI Startup Funding News

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Skild AI has raised \\$300 million to create a general-purpose foundation model for robotics - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en) 

# Latest Developments in Claude AI for Data Analysis

- Google and ECMWF's NeuralGCM model enhances weather forecasts using machine learning, achieving over 3,500 times faster performance than traditional models - [The secret to better weather forecasts may be a dash of AI - Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS8yMDI0LzA3LzI3L2dvb2dsZV9haV93ZWF0aGVyL9IBPWh0dHBzOi8vd3d3LnRoZXJlZ2lzdGVyLmNvbS9BTVAvMjAyNC8wNy8yNy9nb29nbGVfYWlfd2VhdGhlci8?hl=en-US&gl=US&ceid=US:en).
- Billionaire Daniel Loeb invests \\$2.2 billion in AI stocks like Amazon, Microsoft, and Meta, anticipating AI-driven cloud usage growth - [Forget Nvidia: Billionaire Daniel Loeb Owns \\$2.2 Billion Worth of These 3 Brilliant Artificial Intelligence (AI) Stocks Instead - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Mistral AI's Mistral Large 2 model, featuring a 128,000 token context window and multilingual support, is now available on Amazon Bedrock - [Mistral Large 2 is now available in Amazon Bedrock - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en).
- iFixit CEO criticizes Anthropic for overloading their servers, highlighting issues with AI companies scraping content without permission - [iFixit CEO takes shots at Anthropic for 'hitting our servers a million times in 24 hours' and even the AI company's own chatbot disapproves - Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- PRISE, a new machine learning method, uses NLP techniques to improve multitask temporal action abstractions in robotics - [PRISE: A Unique Machine Learning Method for Learning Multitask Temporal Action Abstractions Using Natural Language Processing (NLP) - Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en). 

# AI Startup Funding: Key Developments

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire Daniel Loeb favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Skild AI has raised \\$300 million to create a general-purpose foundation model for robotics - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en) 

# Today's AI Smartphone Innovations

- The author argues that recent advancements in smartphones, including the new iPhone, represent incremental improvements rather than true innovation, mirroring a broader pattern in consumer electronics where technological progress eventually stagnates - [The Guardian](https://news.google.com/articles/CBMijQFodHRwczovL3d3dy50aGVndWFyZGlhbi5jb20vY29tbWVudGlzZnJlZS9hcnRpY2xlLzIwMjQvanVsLzI3L215LW5ldy1pcGhvbmUtc3ltYm9saXNlcy1zdGFnbmF0aW9uLW5vdC1pbm5vdmF0aW9uLWFuZC1hLXNpbWlsYXItZmF0ZS1hd2FpdHMtYWnSAY0BaHR0cHM6Ly9hbXAudGhlZ3VhcmRpYW4uY29tL2NvbW1lbnRpc2ZyZWUvYXJ0aWNsZS8yMDI0L2p1bC8yNy9teS1uZXctaXBob25lLXN5bWJvbGlzZXMtc3RhZ25hdGlvbi1ub3QtaW5ub3ZhdGlvbi1hbmQtYS1zaW1pbGFyLWZhdGUtYXdhaXRzLWFp?hl=en-US&gl=US&ceid=US:en)
- HP is launching two new AI-focused PCs, the EliteBook Ultra and OmniBook X, designed for businesses, featuring Qualcomm's Snapdragon X Elite processor for improved productivity and longer battery life - [Indian Express](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)
- The iPhone 16 is expected to launch in September with new features, including Apple’s iOS 18, an AI chatbot integrated with Siri, and customizable Home Screen app icons - [The Sun](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LnRoZS1zdW4uY29tL3RlY2gvMTIwMDExMTgvaXBob25lLTE2LXByZWRpY3Rpb25zLXdoYXQtdG8tZXhwZWN0L9IBT2h0dHBzOi8vd3d3LnRoZS1zdW4uY29tL3RlY2gvMTIwMDExMTgvaXBob25lLTE2LXByZWRpY3Rpb25zLXdoYXQtdG8tZXhwZWN0L2FtcC8?hl=en-US&gl=US&ceid=US:en)
- Nio unveiled significant advancements, including the Shenji NX9031 autonomous driving chip, SkyOS vehicle operating system, and the second-generation Nio Phone, at its Tech Day event - [CNEV Post](https://news.google.com/articles/CBMiQmh0dHBzOi8vY25ldnBvc3QuY29tLzIwMjQvMDcvMjcvbmlvLWluLTIwMjQtY2hpcC1za3lvcy1waG9uZS1tb3JlL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google's new AI tools in Product Studio enhance small businesses' online marketing by allowing them to create compelling ads, including converting still images into videos and creating 3D views of products - [US Chamber](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en) 

# Big Tech Antitrust Investigations

- AI startup Anthropic is accused of violating anti-scraping protocols by aggressively scraping websites like iFixit and Freelancer, causing significant disruptions - [Engadget](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbnRpLXNjcmFwaW5nLXJ1bGVzLWFuZC1wcm90b2NvbC0xMzMwMjI3NTYuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Nvidia and Alphabet are projected to join the \\$3 trillion market cap club, driven by their dominance in AI and cloud services - [Fool](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en).
- X's new AI training opt-out setting faces regulatory scrutiny, with users skeptical about its effectiveness in protecting their data - [Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/).
- OpenAI's new web search tool for ChatGPT challenges Google's search dominance, raising concerns among publishers about potential traffic loss - [Washington Post](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en).
- TikTok's algorithm can quickly lead users to hate-filled content, raising concerns about its impact on mental well-being and the spread of extreme views - [Reddit](https://www.reddit.com/r/technology/comments/1eduwfi/tiktoks_algorithm_is_highly_sensitive_and_could/). 

# Mistral AI's Mistral Large 2: A New Contender in Language Models

- Mistral AI introduced Mistral Large 2, a 123 billion-parameter language model, outperforming Llama 3.1 and closely following GPT-4 on benchmarks like HumanEval, MultiPL-E, and the MATH benchmark - [Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3Lm1hZ2luYXRpdmUuY29tL2FydGljbGUvbWlzdHJhbC1haS11bnZlaWxzLW1pc3RyYWwtbGFyZ2UtMi1hLXBvd2VyZnVsLW5ldy1sYW5ndWFnZS1tb2RlbC_SAQA?hl=en-US&gl=US&ceid=US:en).
- ML2 demonstrates competitive performance in language understanding tests, scoring 84% on the MMLU benchmark, and is designed for higher efficiency and faster response generation on available hardware compared to its larger counterparts - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFydGlmaWNpYWxpbnRlbGxpZ2VuY2UtbmV3cy5jb20vbmV3cy9taXN0cmFsLWxhcmdlLTItZGF2aWQtdG8tYmlnLXRlY2gtZ29saWF0aHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Mistral Large 2 is designed to minimize hallucination issues and responds more discerningly, along with improved multilingual support that includes understanding 11 languages and 80 coding languages - [Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9taXN0cmFsLXJlbGVhc2VzLWxhcmdlLTItbWV0YS1vcGVuYWktYWktbW9kZWxzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Mistral AI's Mistral Large 2 foundation model is now available on Amazon Bedrock, featuring significant enhancements in multilingual capabilities, reasoning, coding, and more - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vYXdzLmFtYXpvbi5jb20vYmxvZ3MvbWFjaGluZS1sZWFybmluZy9taXN0cmFsLWxhcmdlLTItaXMtbm93LWF2YWlsYWJsZS1pbi1hbWF6b24tYmVkcm9jay_SAQA?hl=en-US&gl=US&ceid=US:en). 

# AI-Powered Customer Service Agents: Today's Highlights

- Mark Zuckerberg predicts AI agents will surpass human population, raising concerns about their role in replacing traditional jobs - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
- Google's new AI tools for SMBs enhance online marketing with features like converting images to videos and creating 3D product views - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd3d3LnVzY2hhbWJlci5jb20vY28vZ29vZC1jb21wYW55L3RoZS1sZWFwL2dvb2dsZS1haS1jdXN0b21lci1zZXJ2aWNlLXRvb2xz0gEA?hl=en-US&gl=US&ceid=US:en)
- HP's new AI PCs, EliteBook Ultra and OmniBook X, aim to boost productivity for businesses with advanced processors and longer battery life - [Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L3RlY2gtbmV3cy10ZWNobm9sb2d5L2hwLW5ldy1jb3BpbG90LXBsdXMtYWktcGNzLXJlYWNoLWluZGlhLTk0NzUzNTgv0gEA?hl=en-US&gl=US&ceid=US:en)
- South Carolina's Department of Administration launches an AI strategy to enhance government operations with a focus on transparency and fairness - [Google News](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticizes Anthropic for server strain caused by excessive hits, highlighting issues with AI companies scraping content without permission - [Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en) 

# Key Developments in AI Ethics and Regulation

- X's new AI training opt-out setting faces regulatory scrutiny and user skepticism regarding its effectiveness - [Reddit](https://www.reddit.com/r/technology/comments/1edwi2o/xs_new_ai_training_optout_setting_draws/).
- AI's integration with robotics raises ethical and safety concerns, necessitating clear regulations and human oversight - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Apple signs the White House's non-binding commitment to AI safety, prompting skepticism about the sincerity of such voluntary commitments - [Reddit](https://www.reddit.com/r/technews/comments/1edzau5/apple_signs_the_white_houses_commitment_to_ai/).
- iFixit CEO criticizes Anthropic for excessive server hits, highlighting ongoing concerns over AI companies scraping website content without permission - [Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, with AI's impact not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en). 

# AI Art by Microsoft: Today's Highlights

- The author of [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots) expresses skepticism towards AI writing tools, noting their potential to harm human writers' credibility while acknowledging their utility for mundane tasks.
- Billionaire Daniel Loeb prefers AI stocks like Amazon, Microsoft, and Meta Platforms over Nvidia, as reported by [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en).
- [Reddit](https://www.reddit.com/r/ChatGPT/comments/1edkg15/what_are_the_chances_that_all_this_marketing_hype/) discusses the limitations of current AI, particularly LLMs, and questions the sustainability of AI companies amid significant financial challenges.
- HP emphasizes AI accessibility with tools like Cephable and OmniBridge, as highlighted by [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en).
- iFixit CEO criticized Anthropic for excessive server hits, with Anthropic's AI, Claude, acknowledging the violation, as reported by [Google News](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en). 

# AI Chip Demand Surge

- SK Hynix is investing \\$6.8 billion to build a semiconductor manufacturing plant in South Korea to meet the rising demand for AI chips - [Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmludmVzdG9wZWRpYS5jb20vbnZpZGlhLXN1cHBsaWVyLXNrLWh5bml4LXRvLXNwZW5kLXVzZDYtOGItb24tc291dGgta29yZWEtcGxhbnQtdG8tbWVldC1haS1jaGlwLWRlbWFuZC04Njg0Mjc50gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Researchers at TU Eindhoven developed a neuromorphic chip for on-chip neural network training, eliminating the need for model transfers and significantly reducing energy consumption and enhancing efficiency - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS9yZWFsLXRpbWUtbmV1cmFsLW5ldHdvcmstdHJhaW5pbmctb24tY2hpcC0yMzM2ODY4MtIBYWh0dHBzOi8vd3d3Lm1pZC1kYXkuY29tL2FtcC90ZWNobm9sb2d5L2FydGljbGUvcmVhbC10aW1lLW5ldXJhbC1uZXR3b3JrLXRyYWluaW5nLW9uLWNoaXAtMjMzNjg2ODI?hl=en-US&gl=US&ceid=US:en).
- Analysts have significantly raised the price targets for Nvidia, with Piper Sandler increasing it from \\$120 to \\$140 and Loop Capital from \\$120 to \\$175, driven by strong business trends and the anticipated growth from the Blackwell architecture launch - [Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5pbnZlc3RpbmcuY29tL25ld3Mvc3RvY2stbWFya2V0LW5ld3MvNS1iaWctYW5hbHlzdC1haS1tb3Zlcy1wdC1oaWtlcy1mb3ItbnZpZGlhLWFwcGxlLXRlc2xhLWRvd25zaWRlLXJpc2stZmxhZ2dlZC0zNTQwMzQy0gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs and heightened demand for AI infrastructure - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en). 

# Advances in Text-To-Image Diffusion Models

- [Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

- [Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3 

# ChatGPT in AI Dominance

- Users share unique and creative ways they utilize ChatGPT, including real-time chatbots, interactive art installations, and simulations for decision-making - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1edrdk3/whats_something_you_use_chatgpt_for_that_youre/).
- OpenAI's SearchGPT, a prototype search engine utilizing GPT-4 AI, offers real-time internet-based information retrieval and organizes search results more effectively than traditional search engines - [Google News](https://news.google.com/articles/CBMiWmh0dHBzOi8vd3d3LnRvbXNndWlkZS5jb20vYWkvaG93LXRvLWFjY2Vzcy1zZWFyY2hncHQtaGVyZXMtaG93LXRvLXNpZ24tdXAtZm9yLXRoZS13YWl0bGlzdNIBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI is introducing a new web search tool for ChatGPT that directly challenges Google's search dominance and envisions a more interactive web experience through AI - [Washington Post](https://news.google.com/articles/CBMiUmh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjUvb3BlbmFpLXNlYXJjaC1nb29nbGUtY2hhdGdwdC_SAQA?hl=en-US&gl=US&ceid=US:en).
- A user reported that 60% of the information obtained from ChatGPT over 24 hours was inaccurate or fabricated, raising concerns about the model's reliability - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1edl1yt/chatgpt_inaccuracy/).
- OpenAI's upgraded Voice Mode for ChatGPT, powered by the new GPT-4o model, will enable more humanlike spoken conversations and is set to launch next week for Plus subscribers - [The Sun](https://news.google.com/articles/CBMiZGh0dHBzOi8vd3d3LnRoZXN1bi5jby51ay90ZWNoLzI5NDk5MDQzL2NoYXRncHQtdm9pY2UtbW9kZS1ncHQtNG8tYWktYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtY2hhdGJvdC_SAWhodHRwczovL3d3dy50aGVzdW4uY28udWsvdGVjaC8yOTQ5OTA0My9jaGF0Z3B0LXZvaWNlLW1vZGUtZ3B0LTRvLWFpLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWNoYXRib3QvYW1wLw?hl=en-US&gl=US&ceid=US:en).
- Elon Musk's X platform has introduced a default setting that uses user posts to help train the AI chatbot Grok, allowing for better fine-tuning of the assistant - [Deadline](https://news.google.com/articles/CBMiRGh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvZWxvbi1tdXNrLXgtYWktY2hhdGJvdC1ncm9rLTEyMzYwMjQwNzcv0gEA?hl=en-US&gl=US&ceid=US:en).
- OpenAI may incur losses of up to \\$5 billion this year due to high operational and training costs associated with its AI products, particularly ChatGPT - [Deadline](https://news.google.com/articles/CBMiUmh0dHBzOi8vZGVhZGxpbmUuY29tLzIwMjQvMDcvb3BlbmFpLWNoYXRncHQtaG9tZS1tYXktbG9zZS01Yi10aGlzLXllYXItMTIzNjAyNDcwNC_SAQA?hl=en-US&gl=US&ceid=US:en). 



In [103]:
edit_prompt1 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below.
It consists of a series of sections, each of which contains several bullet points.
For each section, review each bullet point and advise if it should be moved to a different section.
You will only provide suggestions, and not rewrite the newsletter or provide other comments except
instructions regarding moving bullet points between sections.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections.
Identify sentences and bullet points that should be moved to a different section. Write the
bullet point and the section in should be moved to.
If no bullet points should be moved for a given section, state that no action is required for that section.

Check carefully to make sure all similar bullet points end up grouped together in the same section.

Take a deep breath and work on this problem step-by-step.

Newsletter to edit: 
{mail_md_str}
"""
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt1
              }],
    n=1,   
    temperature=0.2
)

response_str1 = response.choices[0].message.content
display(Markdown(response_str1.replace("$", "\\$")))


2024-07-28 07:57:56,734 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en) 

No action is required for this section.

# Advances in Machine Learning for Structural Biology

- The journey of learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- Insights into transforming various data types into vectors for pattern recognition in high-dimensional space are discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- The potential of deep neural networks as "universal function approximators" is examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/). 

No action is required for this section.

# nference and Takeda AI Partnership

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en) 

- Move "Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns" to "AI in Drug Discovery: Today's Key Developments".

# Ant-inspired AI for Drones

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility - [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from \\$12.3 billion in 2023 to \\$146.8 billion by 2033 - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)
- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics - [Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en) 

No action is required for this section.

# Fei-Fei Li's \\$1B AI Startup

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast discusses new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en) 

- Move "The Robot Report Podcast discusses new robotics technologies, including Jacobi Robotics' innovative control architecture" to "Ant-inspired AI for Drones".

# Artificial Agency's \\$16M Funding

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtl

In [104]:
edit_prompt2 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Below are editing instructions followed by a markdown newsletter.
Carefully review the editing instructions and the markdown newsletter provided below.
The newsletter consists of a series of sections, each of which contains several bullet points.
Move bullet points according to the editing instructions below from one section to another 
If there is no change to a specific section, include it unchanged in the response as it appears in the input.
Respond with the updated newsletter in markdown format.

Editing instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections. Then make only the following changes:
{response_str1}

Newsletter to edit: 
{mail_md_str}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt2
              }],
    n=1,   
    temperature=0.2
)
response_str2 = response.choices[0].message.content
display(Markdown(response_str2.replace("$", "\\$")))


2024-07-28 07:58:43,360 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)

# Advances in Machine Learning for Structural Biology

- The journey of learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- Insights into transforming various data types into vectors for pattern recognition in high-dimensional space are discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/).
- The potential of deep neural networks as "universal function approximators" is examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/). 

# nference and Takeda AI Partnership

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en) 

# Ant-inspired AI for Drones

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility - [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from \\$12.3 billion in 2023 to \\$146.8 billion by 2033 - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)
- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics - [Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en) 

# Fei-Fei Li's \\$1B AI Startup

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en) 

# Artificial Agency's \\$16M Funding

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission - [Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbn

In [105]:
edit_prompt3 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Carefully review each section of the markdown newsletter provided below. 
Each section consists of several bullet points. 

For each section, identify and combine redundant bullet points:

Instructions: 
For each section, identify bullet points containing identical URLs to other bullet points in the same section 
Rewrite the section, combining these similar bullet points to eliminate duplication.
Do not duplicate any URLs within a section.
Check the response carefully and ensure that no links are duplicated within a section.

Newsletter to edit: 
{response_str2}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt3
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-07-28 07:59:24,293 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

# Advances in Machine Learning for Structural Biology

- The journey of learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
- Insights into transforming various data types into vectors for pattern recognition in high-dimensional space are discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
- The potential of deep neural networks as "universal function approximators" is examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)

# nference and Takeda AI Partnership

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

# Ant-inspired AI for Drones

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility - [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from \\$12.3 billion in 2023 to \\$146.8 billion by 2033 - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)
- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics - [Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en)

# Fei-Fei Li's \\$1B AI Startup

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

# Artificial Agency's \\$16M Funding

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission - [Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbn
```


In [106]:
display(Markdown(response_str3[11:].replace("$", "\\$")))



# AI in Drug Discovery: Today's Key Developments

- South Carolina's Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

# Advances in Machine Learning for Structural Biology

- The journey of learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
- Insights into transforming various data types into vectors for pattern recognition in high-dimensional space are discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
- The potential of deep neural networks as "universal function approximators" is examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)

# nference and Takeda AI Partnership

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \\$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

# Ant-inspired AI for Drones

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility - [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from \\$12.3 billion in 2023 to \\$146.8 billion by 2033 - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)
- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics - [Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en)

# Fei-Fei Li's \\$1B AI Startup

- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \\$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

# Artificial Agency's \\$16M Funding

- OpenAI may face bankruptcy within a year, projecting losses of \\$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Billionaire investor Daniel Loeb significantly favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for data without permission - [Google News](https://news.google.com/articles/CBMigAFodHRwczovL3d3dy5lbmdhZGdldC5jb20vd2Vic2l0ZXMtYWNjdXNlLWFpLXN0YXJ0dXAtYW50aHJvcGljLW9mLWJ5cGFzc2luZy10aGVpci1hbn
```


In [107]:
response_str3.replace("$", "\\$")

'```markdown\n# AI in Drug Discovery: Today\'s Key Developments\n\n- South Carolina\'s Department of Administration has launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)\n- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)\n- Nvidia\'s stock is pr

In [108]:
PROMPT = f"""You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging technology trends, and the ability to 
produce, edit, and curate high-quality content that engages and informs readers. You are 
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness, 
and coherence, and ensuring its accuracy and relevance.

Objective: Carefully review each section of the markdown newsletter provided below, which
contains several sections consistint of bullet points. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'Good' if no changes
are needed.

Instructions: 
For each section, review the title and edit it to be as short and engaging, and as consistent with the bullets
in the section as possible
Remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible with facts.
Respond with the updated newsletter only in markdown format, without editorial comment, or the word 'OK' 
if no changes are recommended.

Newsletter to edit: 
{mail_md_str}
"""


response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": PROMPT
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-07-28 08:00:10,802 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI in Drug Discovery: Key Updates

- South Carolina's Department of Administration launched an AI strategy to enhance government operations, focusing on transparency and citizen benefits - [Palmetto Politics](https://news.google.com/articles/CBMilgFodHRwczovL3d3dy5wb3N0YW5kY291cmllci5jb20vcG9saXRpY3Mvc291dGgtY2Fyb2xpbmEtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtdGVjaG5vbG9neS1nb3Zlcm5tZW50L2FydGljbGVfMDI1ODliNjAtNDljZS0xMWVmLWE4YjUtNmJmODUwNDUwOWEwLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)
- iFixit CEO criticized Anthropic for overloading their servers, highlighting concerns over AI companies scraping content without permission - [PC Gamer](https://news.google.com/articles/CBMirQFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9zb2Z0d2FyZS9haS9pZml4aXQtY2VvLXRha2VzLXNob3RzLWF0LWFudGhyb3BpYy1mb3ItaGl0dGluZy1vdXItc2VydmVycy1hLW1pbGxpb24tdGltZXMtaW4tMjQtaG91cnMtYW5kLWV2ZW4tdGhlLWFpLWNvbXBhbnlzLW93bi1jaGF0Ym90LWRpc2FwcHJvdmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's stock is projected to surge by 259% by 2030 due to its strong AI market position, with AMD potentially achieving even higher returns - [Fool](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- USC researchers are using AI to predict wildfire behavior, enhancing firefighting efforts with real-time satellite data - [ABC7](https://news.google.com/articles/CBMiW2h0dHBzOi8vYWJjNy5jb20vcG9zdC9wcmVkaWN0aW5nLXdpbGRmaXJlcy1tb3ZlLXVzYy1yZXNlYXJjaGVycy11c2luZy1haS1mb3JlY2FzdC8xNTEwNTQ1OC_SAQA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes AI accessibility, showcasing tools for users with disabilities and highlighting ecological concerns - [TechRadar](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

# Machine Learning in Structural Biology

- Learning machine learning through coding neural networks is explored, highlighting its application in predicting planetary positions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
- Transforming various data types into vectors for pattern recognition in high-dimensional space is discussed - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)
- Deep neural networks as "universal function approximators" are examined, emphasizing their capability to approximate outputs for complex functions - [Nautilus](https://nautil.us/the-elegant-math-of-machine-learning-727842/)

# nference and Takeda AI Partnership

- Daniel Loeb favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms as part of his \$2.2 billion investment focus in AI - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Google emphasized its commitment to empowering businesses through AI at the AI in Action event, highlighting the benefits of its Gemini AI model for optimizing operations - [Google News](https://news.google.com/articles/CBMiUGh0dHBzOi8vcHVuY2huZy5jb20vZ29vZ2xlLXJlc3RhdGVzLWNvbW1pdG1lbnQtdG8tZW1wb3dlcmluZy1idXNpbmVzc2VzLXdpdGgtYWkv0gFUaHR0cHM6Ly9wdW5jaG5nLmNvbS9nb29nbGUtcmVzdGF0ZXMtY29tbWl0bWVudC10by1lbXBvd2VyaW5nLWJ1c2luZXNzZXMtd2l0aC1haS8_YW1w?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer Mike Wilson warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is predicted to see a 259% stock price increase by 2030 due to its strong position in the AI market, though competitor AMD may achieve even higher returns - [Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L252aWRpYS0yMDMwLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrL9IBAA?hl=en-US&gl=US&ceid=US:en)
- HP emphasizes that the future of AI should focus on accessibility, showcasing tools like Cephable and OmniBridge that cater to users with disabilities - [Google News](https://news.google.com/articles/CBMipAFodHRwczovL3d3dy50ZWNocmFkYXIuY29tL2NvbXB1dGluZy9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9ocC1tYXktaGF2ZS1wbGVudHktb2YtYWktc29mdHdhcmUtYW5kLWhhcmR3YXJlLXRvLXNob3ctb2ZmLWJ1dC1hY2Nlc3NpYmlsaXR5LWlzLWFpcy1iZXN0LWFuZC1vbmx5LWZ1dHVyZdIBAA?hl=en-US&gl=US&ceid=US:en)

# Ant-inspired AI for Drones

- AI is enhancing the capabilities of robots in manufacturing and logistics, enabling them to perform more complex tasks and increasing their adaptability - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3LmphcGFudGltZXMuY28uanAvY29tbWVudGFyeS8yMDI0LzA3LzIzL3dvcmxkL2FpLXNtYXJ0ZXItcm9ib3RzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The author expresses skepticism and frustration towards AI writing tools like Jasper AI, describing them as producing subpar content and raising concerns about their reliability and potential to harm human writers' credibility - [Hacker Noon](https://hackernoon.com/a-human-writers-thoughts-on-ai-writing-bots)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture led by CEO Max Cao - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- The integration of AI into robotics is significantly enhancing their capabilities, allowing robots to perform complex tasks with improved efficiency and adaptability across various industries, projected to grow from \$12.3 billion in 2023 to \$146.8 billion by 2033 - [Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmRjdmVsb2NpdHkuY29tL2FydGljbGVzLzYxMzcyLWFpLWluLXJvYm90aWNzLXJldm9sdXRpb25pemluZy10aGUtZnV0dXJlLW9mLWF1dG9tYXRpb27SAQA?hl=en-US&gl=US&ceid=US:en)
- PRISE (Primitive Sequence Encoding) is a novel method that applies NLP techniques, specifically byte pair encoding (BPE), to learn multitask temporal action abstractions in continuous action spaces for robotics - [Google News](https://news.google.com/articles/CBMiqQFodHRwczovL3d3dy5tYXJrdGVjaHBvc3QuY29tLzIwMjQvMDcvMjYvcHJpc2UtYS11bmlxdWUtbWFjaGluZS1sZWFybmluZy1tZXRob2QtZm9yLWxlYXJuaW5nLW11bHRpdGFzay10ZW1wb3JhbC1hY3Rpb24tYWJzdHJhY3Rpb25zLXVzaW5nLW5hdHVyYWwtbGFuZ3VhZ2UtcHJvY2Vzc2luZy1ubHAv0gGtAWh0dHBzOi8vd3d3Lm1hcmt0ZWNocG9zdC5jb20vMjAyNC8wNy8yNi9wcmlzZS1hLXVuaXF1ZS1tYWNoaW5lLWxlYXJuaW5nLW1ldGhvZC1mb3ItbGVhcm5pbmctbXVsdGl0YXNrLXRlbXBvcmFsLWFjdGlvbi1hYnN0cmFjdGlvbnMtdXNpbmctbmF0dXJhbC1sYW5ndWFnZS1wcm9jZXNzaW5nLW5scC8_YW1w?hl=en-US&gl=US&ceid=US:en)

# Fei-Fei Li's \$1B AI Startup

- Daniel Loeb favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI may face bankruptcy within a year, projecting losses of \$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is projected to reach a market capitalization of \$3 trillion by the end of 2024, driven by its dominance in data center GPUs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4LzItbW9uc3Rlci1haS1ncm93dGgtc3RvY2tzLWJ1eS1iZWZvcmUtMy10cmlsbGlvbi_SAQA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast discusses new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Morgan Stanley's chief investment officer warns that the hot AI trade may be entering a dangerous phase, as AI's impact is not fully reflected in companies' revenue and earnings numbers - [Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3LmFvbC5jb20vZmluYW5jZS9ob3QtYWktdHJhZGUtZW50ZXJpbmctZGFuZ2VyLTE2MDAwODAzMi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

# Artificial Agency's \$16M Funding

- OpenAI may face bankruptcy within a year, projecting losses of \$5 billion due to high operational costs - [Google News](https://news.google.com/articles/CBMimAFodHRwczovL3d3dy53aW5kb3dzY2VudHJhbC5jb20vc29mdHdhcmUtYXBwcy9vcGVuYWktY291bGQtYmUtb24tdGhlLWJyaW5rLW9mLWJhbmtydXB0Y3ktaW4tdW5kZXItMTItbW9udGhzLXdpdGgtcHJvamVjdGlvbnMtb2YtZG9sbGFyNS1iaWxsaW9uLWluLWxvc3Nlc9IBAA?hl=en-US&gl=US&ceid=US:en)
- Daniel Loeb favors AI stocks over Nvidia, holding major positions in Amazon, Microsoft, and Meta Platforms - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI4L2ZvcmdldC1udmlkaWEtYmlsbGlvbmFpcmUtZGFuaWVsLWxvZWItb3ducy0yMi1iaWxsL9IBAA?hl=en-US&gl=US&ceid=US:en)
- The Robot Report Podcast episode features discussions on new robotics technologies, including Jacobi Robotics' innovative control architecture - [Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LnRoZXJvYm90cmVwb3J0LmNvbS9haS1jb250cm9scy1tYWtlLXJvYm90cy1lYXNpZXItcHJvZ3JhbS1kZXBsb3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- Mark Zuckerberg predicts that the number of AI agents will surpass the human population as they increasingly reflect individual and business values - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1ee1eli/mark_zuckerberg_says_there_will_be_more_ai_agents/)
- iFixit and Freelancer have accused AI startup Anthropic of violating their "do not crawl" protocols by aggressively scraping their websites for

In [109]:
mail_md_str = response_str3

In [110]:
z = """
# AI in Drug Discovery

- SLAC and Stanford develop AI method to enhance materials discovery - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vcGh5cy5vcmcvbmV3cy8yMDI0LTA3LWFpLWFwcHJvYWNoLW1hdGVyaWFscy1kaXNjb3Zlcnktc3RhZ2UuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Intel and IOC create Athlete365, an AI chatbot for Olympic athletes - [Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmludGMuY29tL25ld3MtZXZlbnRzL3ByZXNzLXJlbGVhc2VzL2RldGFpbC8xNzAyL2Zyb20tYXRobGV0ZXMtdG8tZ2VuYWktZGV2ZWxvcGVycy1pbnRlbC10YWNrbGVzLXJlYWwtd29ybGTSAQA?hl=en-US&gl=US&ceid=US:en).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI and Broadcom discuss new AI chip to reduce GPU reliance - [Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9vcGVuYWktaG9sZHMtdGFsa3MtYnJvYWRjb20tZGV2ZWxvcGluZy0yMTAwMzEwODkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
# Machine Learning in Structural Biology

- MIT uses machine learning to enhance high-entropy materials design - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vdGVjaHhwbG9yZS5jb20vbmV3cy8yMDI0LTA3LW1hY2hpbmUtc2VjcmV0cy1hZHZhbmNlZC1hbGxveXMuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).

# OpenAI and Broadcom Partnership

- OpenAI and Broadcom to develop new AI chip - [Financial Times](https://www.ft.com/content/496a0c33-1af3-4dbf-977f-04d6804a8d28).
- Broadcom projects over \$11B in AI sales for FY24 - [TipRanks](https://www.tipranks.com/news/broadcom-nasdaqavgo-eyes-ai-chip-collaboration-with-openai).
- OpenAI hires ex-Google employees for AI server chip development - [Yahoo Finance](https://finance.yahoo.com/news/openai-holds-talks-broadcom-developing-210031089.html).

# Ant-inspired AI for Drones

- Army tests Black Hornet 3 drones for squad deployment - [Fox 7 Austin](https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZNIBZGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZC5hbXA?hl=en-US&gl=US&ceid=US:en).
- Drone warfare in Ukraine shifts military helicopter tactics - [Defense News](https://news.google.com/articles/CBMie2h0dHBzOi8vd3d3LmRlZmVuc2VuZXdzLmNvbS9nbG9iYWwvZXVyb3BlLzIwMjQvMDcvMTkvZHJvbmUtd2FyZmFyZS1pbi11a3JhaW5lLXByb21wdHMtZnJlc2gtdGhpbmtpbmctaW4taGVsaWNvcHRlci10YWN0aWNzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Ukraine's use of AI-driven unmanned systems raises ethical concerns - [American Magazine](https://news.google.com/articles/CBMigwFodHRwczovL3d3dy5hbWVyaWNhbWFnYXppbmUub3JnL3BvbGl0aWNzLXNvY2lldHkvMjAyNC8wNy8xOC91a3JhaW5lLWxldGhhbC1hdXRvbm9tb3VzLXdlYXBvbnMtc3lzdGVtcy1wb3BlLWZyYW5jaXMtdW4taW50ZXJuYXRpb25hbNIBAA?hl=en-US&gl=US&ceid=US:en).
- U.S. and China discuss AI risks in military contexts - [Foreign Policy](https://news.google.com/articles/CBMiT2h0dHBzOi8vZm9yZWlnbnBvbGljeS5jb20vMjAyNC8wNy8xOC9jaGluYS1taWxpdGFyeS1haS1hcnRpZmljaWFsLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- Pentagon emphasizes "human-machine teaming" in military - [IEEE Spectrum](https://spectrum.ieee.org/robot-dog-vacuum).

# Fei-Fei Li's AI Startup

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- OpenAI developing AI chip to reduce Nvidia reliance - [The Verge](https://www.theverge.com/2024/7/19/24201737/openai-wants-in-on-the-ai-chip-business).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- Major tech companies face financial risks in AI investments - [Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9taWNyb3NvZnQtbWV0YS1hbWF6b24tYW5kLWdvb2dsZS1mYWNlLXRoaXMtZ3Jvd2luZy1yaXNrLWFyb3VuZC1haS0yOGJjYTRhN9IBcGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9hbXAvc3RvcnkvbWljcm9zb2Z0LW1ldGEtYW1hem9uLWFuZC1nb29nbGUtZmFjZS10aGlzLWdyb3dpbmctcmlzay1hcm91bmQtYWktMjhiY2E0YTc?hl=en-US&gl=US&ceid=US:en).

# Artificial Agency's Funding

- Saronic raises \$175M for autonomous military boats, valued at over \$1B - [Forbes](https://www.forbes.com/sites/davidjeans/2024/07/18/andreessen-horowitz-saronic-funding/).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en).

# CTERA's \$80M Funding

- CTERA raises \$80M in Series D funding led by Red Dot Capital Partners - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- Funds to accelerate product development, expand global sales, and enhance customer support - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- CTERA to innovate cloud storage solutions and strengthen enterprise market position - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).

# Anthropic's \$100M AI Fund

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en)
"""


display(Markdown(z))



# AI in Drug Discovery

- SLAC and Stanford develop AI method to enhance materials discovery - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vcGh5cy5vcmcvbmV3cy8yMDI0LTA3LWFpLWFwcHJvYWNoLW1hdGVyaWFscy1kaXNjb3Zlcnktc3RhZ2UuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Intel and IOC create Athlete365, an AI chatbot for Olympic athletes - [Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmludGMuY29tL25ld3MtZXZlbnRzL3ByZXNzLXJlbGVhc2VzL2RldGFpbC8xNzAyL2Zyb20tYXRobGV0ZXMtdG8tZ2VuYWktZGV2ZWxvcGVycy1pbnRlbC10YWNrbGVzLXJlYWwtd29ybGTSAQA?hl=en-US&gl=US&ceid=US:en).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI and Broadcom discuss new AI chip to reduce GPU reliance - [Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9vcGVuYWktaG9sZHMtdGFsa3MtYnJvYWRjb20tZGV2ZWxvcGluZy0yMTAwMzEwODkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
# Machine Learning in Structural Biology

- MIT uses machine learning to enhance high-entropy materials design - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vdGVjaHhwbG9yZS5jb20vbmV3cy8yMDI0LTA3LW1hY2hpbmUtc2VjcmV0cy1hZHZhbmNlZC1hbGxveXMuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).

# OpenAI and Broadcom Partnership

- OpenAI and Broadcom to develop new AI chip - [Financial Times](https://www.ft.com/content/496a0c33-1af3-4dbf-977f-04d6804a8d28).
- Broadcom projects over \$11B in AI sales for FY24 - [TipRanks](https://www.tipranks.com/news/broadcom-nasdaqavgo-eyes-ai-chip-collaboration-with-openai).
- OpenAI hires ex-Google employees for AI server chip development - [Yahoo Finance](https://finance.yahoo.com/news/openai-holds-talks-broadcom-developing-210031089.html).

# Ant-inspired AI for Drones

- Army tests Black Hornet 3 drones for squad deployment - [Fox 7 Austin](https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZNIBZGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZC5hbXA?hl=en-US&gl=US&ceid=US:en).
- Drone warfare in Ukraine shifts military helicopter tactics - [Defense News](https://news.google.com/articles/CBMie2h0dHBzOi8vd3d3LmRlZmVuc2VuZXdzLmNvbS9nbG9iYWwvZXVyb3BlLzIwMjQvMDcvMTkvZHJvbmUtd2FyZmFyZS1pbi11a3JhaW5lLXByb21wdHMtZnJlc2gtdGhpbmtpbmctaW4taGVsaWNvcHRlci10YWN0aWNzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Ukraine's use of AI-driven unmanned systems raises ethical concerns - [American Magazine](https://news.google.com/articles/CBMigwFodHRwczovL3d3dy5hbWVyaWNhbWFnYXppbmUub3JnL3BvbGl0aWNzLXNvY2lldHkvMjAyNC8wNy8xOC91a3JhaW5lLWxldGhhbC1hdXRvbm9tb3VzLXdlYXBvbnMtc3lzdGVtcy1wb3BlLWZyYW5jaXMtdW4taW50ZXJuYXRpb25hbNIBAA?hl=en-US&gl=US&ceid=US:en).
- U.S. and China discuss AI risks in military contexts - [Foreign Policy](https://news.google.com/articles/CBMiT2h0dHBzOi8vZm9yZWlnbnBvbGljeS5jb20vMjAyNC8wNy8xOC9jaGluYS1taWxpdGFyeS1haS1hcnRpZmljaWFsLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- Pentagon emphasizes "human-machine teaming" in military - [IEEE Spectrum](https://spectrum.ieee.org/robot-dog-vacuum).

# Fei-Fei Li's AI Startup

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- OpenAI developing AI chip to reduce Nvidia reliance - [The Verge](https://www.theverge.com/2024/7/19/24201737/openai-wants-in-on-the-ai-chip-business).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- Major tech companies face financial risks in AI investments - [Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9taWNyb3NvZnQtbWV0YS1hbWF6b24tYW5kLWdvb2dsZS1mYWNlLXRoaXMtZ3Jvd2luZy1yaXNrLWFyb3VuZC1haS0yOGJjYTRhN9IBcGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9hbXAvc3RvcnkvbWljcm9zb2Z0LW1ldGEtYW1hem9uLWFuZC1nb29nbGUtZmFjZS10aGlzLWdyb3dpbmctcmlzay1hcm91bmQtYWktMjhiY2E0YTc?hl=en-US&gl=US&ceid=US:en).

# Artificial Agency's Funding

- Saronic raises \$175M for autonomous military boats, valued at over \$1B - [Forbes](https://www.forbes.com/sites/davidjeans/2024/07/18/andreessen-horowitz-saronic-funding/).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en).

# CTERA's \$80M Funding

- CTERA raises \$80M in Series D funding led by Red Dot Capital Partners - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- Funds to accelerate product development, expand global sales, and enhance customer support - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- CTERA to innovate cloud storage solutions and strengthen enterprise market position - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).

# Anthropic's \$100M AI Fund

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en)


In [111]:
from openbb import obb
import datetime

# Get today's date
today = datetime.date.today().strftime("%Y-%m-%d")

# log in
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

# Search for AI news from today
results = obb.news.company("META, MSFT, GOOG, AAPL, AMZN, NVDA, TSLA", provider='yfinance', limit=20).to_df()


In [112]:
results


,title,images,url,symbols,source
date,,,,,
2024-07-26 19:40:22,"Apple, Microsoft, Meta Set to Report During Bu...",[{'url': 'https://s.yimg.com/uu/api/res/1.2/Fw...,https://finance.yahoo.com/m/91f40042-bfac-352a...,"AMZN,MSFT,AAPL,META",The Wall Street Journal
2024-07-26 21:12:09,"Big Tech earnings, jobs report, Fed decision: ...",[{'url': 'https://s.yimg.com/uu/api/res/1.2/zw...,https://finance.yahoo.com/video/big-tech-earni...,"MSFT,QCOM,AAPL,TM,TOYOF,MCD,PYPL,PFE,INTC,META...",Yahoo Finance Video
2024-07-26 21:46:05,"Dow Jones Stocks Apple, Microsoft Lead Earning...",[{'url': 'https://s.yimg.com/uu/api/res/1.2/o1...,https://finance.yahoo.com/m/d8a4aed5-d3df-3fd4...,"MSFT,AAPL,^GSPC,META,AMZN,AMD",Investor's Business Daily
2024-07-26 23:09:00,Mag 7 Earnings Loom: A Closer Look,[{'url': 'https://s.yimg.com/uu/api/res/1.2/ke...,https://finance.yahoo.com/news/mag-7-earnings-...,"AMZN,AAPL,META",Zacks
2024-07-27 08:30:00,1 Growth Stock Down 59% to Buy Right Now,[{'url': 'https://s.yimg.com/uu/api/res/1.2/Yk...,https://finance.yahoo.com/m/6b5325c4-f0b6-3bb9...,"SHOP,AMZN,META",Motley Fool
2024-07-27 12:00:52,Tesla's volume growth will 'have to wait quite...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/gY...,https://finance.yahoo.com/video/teslas-volume-...,TSLA,Yahoo Finance Video
2024-07-27 14:03:00,No One Really Knows What Tesla Stock Is Worth,[{'url': 'https://s.yimg.com/uu/api/res/1.2/T....,https://finance.yahoo.com/m/dfa6b332-3e6e-32a2...,TSLA,Motley Fool
2024-07-27 15:18:02,"Dow Jones Futures: Huge AI Earnings, Fed Pivot...",[{'url': 'https://s.yimg.com/uu/api/res/1.2/g0...,https://finance.yahoo.com/m/b8b5f4a5-8ba6-3515...,"^GSPC,MSFT,^DJI,AAPL,COMP,META,NVDA,GOOG,TSLA",Investor's Business Daily
2024-07-27 15:18:02,"Dow Jones Futures: Huge AI Earnings, Fed Pivot...",[{'url': 'https://s.yimg.com/uu/api/res/1.2/g0...,https://finance.yahoo.com/m/b8b5f4a5-8ba6-3515...,"^GSPC,MSFT,^DJI,AAPL,COMP,META,NVDA,GOOG,TSLA",Investor's Business Daily
